In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path("/Users/yenkopro/Desktop/Metadata_Yelp")
PATH = BASE / "11_biz_merged_clean.parquet"


if not PATH.exists():
    print("Parquet file not found at:", PATH)
    print("Parquet files in the folder:")
    for p in BASE.glob("*.parquet"):
        print("  -", p.name)
    raise FileNotFoundError(PATH)


df = pd.read_parquet(PATH)  # or: pd.read_parquet(PATH, engine="pyarrow")

print(df.shape)
df.head()


In [ ]:
# --- imports ---
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.api.types import is_numeric_dtype

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# --- load your business-level table (with business_id) ---
PATH = "/Users/yenkopro/Desktop/Metadata_Yelp/11_biz_merged_clean.parquet"
df = pd.read_parquet(PATH)

# drop columns that are entirely missing (avoids imputer warnings like attr_Smoking)
all_null = [c for c in df.columns if df[c].isna().all()]
if all_null:
    df = df.drop(columns=all_null)

# columns we should NEVER use as features
ban = {
    "business_id", "name", "address", "postal_code", "phone",
    # any labels / leak-prone targets if present
    "avg_stars_2019", "avg_stars", "stars", "rev_count_2019",
    "y_cls", "label", "target", "y_reg", "label_reg",
    # free-text or raw nested if present
    "categories", "hours"
}
feat_cols = [c for c in df.columns if c not in ban]

# split numeric vs categorical automatically
num_cols = [c for c in feat_cols if is_numeric_dtype(df[c])]
cat_cols = [c for c in feat_cols if c not in num_cols]

print(f"Rows: {len(df):,}  | feature columns: {len(feat_cols)} "
      f"(num={len(num_cols)}, cat={len(cat_cols)})")

# build the sparse preprocessing pipeline
pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc",  StandardScaler(with_mean=False)),
        ]), num_cols),

        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            # NOTE: scikit-learn >=1.2 uses 'sparse_output', not 'sparse'
            ("oh",  OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
        ]), cat_cols),
    ],
    sparse_threshold=1.0,                 # keep overall output sparse
    verbose_feature_names_out=False
)

# this is the full feature frame we’ll feed to 'pre'
X_all = df[feat_cols]


In [ ]:
!pip install scikit-optimize

In [ ]:
%pip install kmodes gower scikit-learn-extra


In [ ]:
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import Normalizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# --- Build a compact dense embedding once (sparse -> SVD -> L2 normalize) ---
Z_sparse = pre.fit_transform(X_all)                   # sparse matrix
svd = TruncatedSVD(n_components=80, random_state=42)
Z80 = svd.fit_transform(Z_sparse)                     # dense (n x 80)
Z80 = Normalizer(copy=False).fit_transform(Z80)       # spherical (L2=1)

def scan_k(Z, ks=(3,4,5,6,7,8,10,12), sample=6000, seed=42):
    rng = np.random.default_rng(seed)
    idx = rng.choice(Z.shape[0], size=min(sample, Z.shape[0]), replace=False)
    Zs  = Z[idx]
    rows = []
    for k in ks:
        print(f"fitting k={k} ...", end="")
        km = MiniBatchKMeans(
            n_clusters=k, batch_size=2048, max_iter=100,
            n_init="auto", random_state=seed, verbose=0
        )
        lbl = km.fit_predict(Zs)
        sil = silhouette_score(
            Zs, lbl, metric="cosine",
            sample_size=min(2000, len(Zs)), random_state=seed
        )
        rows.append((k, sil, km.inertia_))
        print(f"  silhouette={sil:.4f}")
    return pd.DataFrame(rows, columns=["k","silhouette","inertia"]).sort_values("k")

scan_results = scan_k(Z80)
display(scan_results)

best_k = int(scan_results.sort_values("silhouette", ascending=False).iloc[0]["k"])
print("Chosen k =", best_k)

# --- Fit final model on ALL rows ---
km_final = MiniBatchKMeans(
    n_clusters=best_k, batch_size=2048, max_iter=200,
    n_init="auto", random_state=42
)
labels = km_final.fit_predict(Z80)
df["cluster_mbk"] = labels

# --- Quick 2D plot for the report ---
XY = PCA(n_components=2, random_state=42).fit_transform(Z80)
plt.figure(figsize=(7,6))
sc = plt.scatter(XY[:,0], XY[:,1], c=df["cluster_mbk"], s=6, alpha=0.35, cmap="tab10")
plt.title(f"MiniBatch Spherical K-Means (k={best_k})")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.legend(*sc.legend_elements(title="cluster"), loc="best", fontsize=8)
plt.tight_layout(); plt.show()

# --- Optional: cluster summary if avg_stars_2019 exists ---
if "avg_stars_2019" in df.columns:
    summ = (df.assign(y=(df["avg_stars_2019"]>=4).astype(int))
              .groupby("cluster_mbk")
              .agg(n=("business_id","size"),
                   avg_stars=("avg_stars_2019","mean"),
                   pct_4plus=("y","mean"))
              .sort_index())
    display(summ)

# Save cluster assignments for later supervised blending
df[["business_id","cluster_mbk"]].to_csv("clusters_mbkmeans.csv", index=False)



In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, normalize
from sklearn.impute import SimpleImputer
from sklearn.decomposition import TruncatedSVD



# -------- Label for external eval (optional) --------
label_col = None
for cand in ["avg_stars_2019", "avg_stars", "stars", "y_cls", "label", "target"]:
    if cand in df.columns:
        label_col = cand
        break

y = None
if label_col is not None:
    # binary ≥4★
    if df[label_col].dropna().isin([0,1]).all():
        y = df[label_col].astype(int)
    else:
        y = (df[label_col] >= 4.0).astype(int)
    print("Using label:", label_col, " (binary target constructed)")

# -------- Feature table --------
ban = {
    "business_id", "checkin_count", "rev_count_2019", "stars", "avg_stars",
    "avg_stars_2019", "y_cls", "label", "target"
}
# drop all-NA columns to avoid imputer warnings
all_na_cols = [c for c in df.columns if df[c].isna().all()]
if all_na_cols:
    print("Dropping all-NA columns:", all_na_cols)
use_cols = [c for c in df.columns if c not in ban and c not in all_na_cols]

Xfeat = df[use_cols].copy()

# split by dtype
num_cols = Xfeat.select_dtypes(include=[np.number, "boolean", "bool"]).columns.tolist()
cat_cols = Xfeat.columns.difference(num_cols).tolist()

# Keep common continuous columns up front if present (not required, just nice)
preferred_num = [
    "latitude","longitude","review_count_log1p","review_count",
    "total_weekly_hours","days_open","weekend_hours","avg_daily_hours",
    "attr_RestaurantsPriceRange2"
]
num_cols = [c for c in preferred_num if c in num_cols] + [c for c in num_cols if c not in preferred_num]

print(f"num={len(num_cols)}  cat={len(cat_cols)}  total={len(use_cols)}")

# -------- Sparse preprocessing (same pattern as earlier) --------
pre_unsup = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("to_float", FunctionTransformer(lambda X: X.astype(float), feature_names_out="one-to-one")),
            ("imp", SimpleImputer(strategy="median")),
            ("sc", StandardScaler(with_mean=False)),
        ]), num_cols),

        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
        ]), cat_cols),
    ],
    sparse_threshold=1.0,
    verbose_feature_names_out=False
)

Xs = pre_unsup.fit_transform(Xfeat)   # sparse matrix
print("Xs shape (sparse):", Xs.shape)

# -------- SVD → 80D + L2 row-normalize (spherical) --------
svd = TruncatedSVD(n_components=80, random_state=42)
Z = svd.fit_transform(Xs)             # dense (n, 80)
from sklearn.preprocessing import normalize as sk_normalize
Zs = sk_normalize(Z, norm="l2", axis=1)  # L2 row normalization
print("Zs shape:", Zs.shape, "  explained_var(80)≈", svd.explained_variance_ratio_.sum())


In [ ]:
#MiniBatch “spherical” K-Means

In [ ]:
import numpy as np, pandas as pd
from sklearn.preprocessing import normalize
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, confusion_matrix, classification_report, roc_auc_score, f1_score
from scipy.optimize import linear_sum_assignment

# Zs: your 80-D embedding (already computed)
# y : your binary label (0/1), index-aligned with df

# 1) L2-normalize rows (spherical K-Means)
Z_unit = normalize(Zs)

# 2) Search k and fit
k_grid = [3,4,5,6,7]
results, models = [], {}
for k in k_grid:
    mb = MiniBatchKMeans(
        n_clusters=k, init="k-means++", n_init=10,
        batch_size=2048, max_iter=100, reassignment_ratio=0.01,
        random_state=42
    )
    labels = mb.fit_predict(Z_unit)
    sil = silhouette_score(Z_unit, labels, metric="cosine")
    results.append((k, sil, float(mb.inertia_)))
    models[k] = (mb, labels)

res_df = pd.DataFrame(results, columns=["k","silhouette","inertia"]).sort_values("k")
best_k = int(res_df.loc[res_df.silhouette.idxmax(), "k"])
mbkm, labels = models[best_k]
df["cluster_sph"] = labels  # keep for later

print("MiniBatch spherical K-Means — chosen k =", best_k)
print(res_df)

# 3) External metrics by mapping clusters → classes (Hungarian assignment)
c = pd.Series(labels, index=df.index, name="cluster_sph")
cm_tc = confusion_matrix(y, c)                        # rows=true (0/1), cols=clusters
cost = cm_tc.max() - cm_tc
rows, cols = linear_sum_assignment(cost)
cluster_to_class = {cols[i]: rows[i] for i in range(len(rows))}
majority = int(y.mode()[0])
y_pred  = c.map(cluster_to_class).fillna(majority).astype(int)

# calibrated score: per-cluster positive rate
p_pos   = y.groupby(c).mean()
y_proba = c.map(p_pos).fillna(y.mean()).astype(float)

acc = float((y_pred == y).mean())
f1m = float(f1_score(y, y_pred, average="macro"))
auc = float(roc_auc_score(y, y_proba))

print("\naccuracy=%.3f  macro-F1=%.3f  ROC-AUC=%.3f" % (acc, f1m, auc))
print("\nConfusion matrix (mapped):\n", confusion_matrix(y, y_pred))
print("\nClassification report:\n", classification_report(y, y_pred, digits=3))

# 4) Handy dicts to paste into the paper
sph_params = {
    "svd_components": Zs.shape[1], "row_normalize": "l2",
    "k": best_k, "batch_size": 2048, "n_init": 10,
    "max_iter": 100, "reassignment_ratio": 0.01, "random_state": 42
}
sph_scores = {
    "silhouette": float(res_df.loc[res_df.k==best_k, "silhouette"].iloc[0]),
    "inertia": float(mbkm.inertia_),
    "accuracy": acc, "macro_f1": f1m, "roc_auc": auc
}
print("\nHYPERPARAMS:", sph_params)
print("SCORES:", sph_scores)


In [ ]:
#Fix the metrics (use mapped predictions)
#confusion matrix + classification report are using raw cluster IDs (0..6), 
#not the binary mapped predictions.

In [ ]:
import numpy as np, pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
from scipy.optimize import linear_sum_assignment

# c: raw cluster labels from your chosen model
c = pd.Series(df["cluster_sph"], index=df.index, name="cluster_sph")

# --- raw audit: true class (rows) x cluster (cols) ---
cm_raw = pd.crosstab(y, c)
print("Raw confusion (true class x cluster):\n", cm_raw)

# --- optimal mapping cluster -> {0,1} via Hungarian ---
cm_tc = confusion_matrix(y, c)  # rows=true labels (0/1), cols=clusters
cost = cm_tc.max() - cm_tc
rows, cols = linear_sum_assignment(cost)
cluster_to_class = {cols[i]: rows[i] for i in range(len(rows))}
majority = int(y.mode()[0])

# mapped hard predictions
y_pred = c.map(cluster_to_class).fillna(majority).astype(int)

# calibrated “probability” = per-cluster positive rate
p_pos = y.groupby(c).mean()
y_proba = c.map(p_pos).fillna(y.mean()).astype(float)

# --- binary metrics ---
acc = float((y_pred == y).mean())
f1m = float(f1_score(y, y_pred, average="macro"))
auc = float(roc_auc_score(y, y_proba))

print("\nMapped confusion (true x predicted):\n", confusion_matrix(y, y_pred))
print("\nClassification report (binary):\n", classification_report(y, y_pred, digits=3))
print("accuracy=%.3f  macro-F1=%.3f  ROC-AUC=%.3f" % (acc, f1m, auc))


In [ ]:
#Small improvements you can try on the spherical K-Means (quick wins)

In [ ]:
# Sweep SVD dimensionality and k
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

svd_dims = [60, 80, 120]
k_grid   = [5, 6, 7, 8, 9]
records = []

Xs_norm = Xs  # your sparse matrix from ColumnTransformer
for d in svd_dims:
    svd = TruncatedSVD(n_components=d, random_state=42)
    Z   = svd.fit_transform(Xs_norm)
    Z   = normalize(Z)
    for k in k_grid:
        mb = MiniBatchKMeans(n_clusters=k, init="k-means++", n_init=10,
                             batch_size=2048, max_iter=100, reassignment_ratio=0.01,
                             random_state=42)
        lab = mb.fit_predict(Z)
        sil = silhouette_score(Z, lab, metric="cosine")
        records.append((d, k, sil, mb.inertia_))

pd.DataFrame(records, columns=["svd_dim","k","silhouette","inertia"]).sort_values(["silhouette"], ascending=False)


In [ ]:
# k search table
res_df.to_csv("spherical_kmeans_k_sweep.csv", index=False)

# cluster assignments
df[["business_id","cluster_sph"]].to_csv("cluster_sph_assignments.csv", index=False)

# hyperparams & scores you already printed (paste these in Methods/Results)
# HYPERPARAMS: {'svd_components': 80, 'row_normalize': 'l2', 'k': 7, ...}
# SCORES: {'silhouette': 0.0926, 'inertia': 5320.82, 'accuracy': ..., 'macro_f1': ..., 'roc_auc': ...}


In [ ]:
#2 K-Modes: search k, fit best, and evaluate vs. your binary label

In [ ]:
# --- 1) Re-embed with SVD=60 and L2-normalize ---
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, confusion_matrix, classification_report, roc_auc_score, f1_score
from sklearn.metrics.pairwise import cosine_distances
from scipy.optimize import linear_sum_assignment
import numpy as np, pandas as pd

svd_dim = 60
svd = TruncatedSVD(n_components=svd_dim, random_state=42)
Zs = svd.fit_transform(Xs)          # Xs is your sparse matrix from ColumnTransformer
Zs = normalize(Zs)                  # spherical (cosine) geometry

# --- 2) Fit MiniBatch K-Means (spherical via normalized Zs) ---
k = 7
mbkm = MiniBatchKMeans(
    n_clusters=k, init="k-means++", n_init=10,
    batch_size=2048, max_iter=100, reassignment_ratio=0.01,
    random_state=42
)
labels = mbkm.fit_predict(Zs)

sil = float(silhouette_score(Zs, labels, metric="cosine"))
inertia = float(mbkm.inertia_)
print(f"MiniBatch spherical K-Means (svd_dim={svd_dim}, k={k})  silhouette={sil:.4f}  inertia={inertia:.1f}")

# Persist cluster ids on df
df["cluster_sph"] = labels

# --- 3) External mapping to binary label (ONLY if y exists) ---
cm_true_vs_cluster = confusion_matrix(y, labels)
cost = cm_true_vs_cluster.max() - cm_true_vs_cluster
rows, cols = linear_sum_assignment(cost)
cluster_to_class = {cols[i]: rows[i] for i in range(len(rows))}
majority = int(y.mode()[0])

y_pred  = pd.Series(labels, index=df.index).map(cluster_to_class).fillna(majority).astype(int)
p_pos   = y.groupby(labels).mean()
y_proba = pd.Series(labels, index=df.index).map(p_pos).fillna(y.mean()).astype(float)

acc = float((y_pred == y).mean())
f1m = float(f1_score(y, y_pred, average="macro"))
auc = float(roc_auc_score(y, y_proba))

print("\nMapped confusion (true x predicted):\n", confusion_matrix(y, y_pred))
print("\nClassification report (binary):\n", classification_report(y, y_pred, digits=3))
print(f"accuracy={acc:.3f}  macro-F1={f1m:.3f}  ROC-AUC={auc:.3f}")

# --- 4) Optional: distances for later hybrid features (cosine distance to centroids) ---
C = normalize(mbkm.cluster_centers_)
dist_cos = cosine_distances(Zs, C)            # shape: (n_samples, k)
dist_cols = [f"sphcosdist_{i}" for i in range(k)]
dist_df = pd.DataFrame(dist_cos, index=df.index, columns=dist_cols)
df[dist_cols] = dist_df

# --- 5) Save artifacts for the paper ---
pd.DataFrame({"business_id": df["business_id"], "cluster_sph": df["cluster_sph"]}).to_csv("cluster_sph_assignments.csv", index=False)
pd.DataFrame({"svd_dim":[svd_dim], "k":[k], "silhouette":[sil], "inertia":[inertia],
              "acc":[acc], "macro_f1":[f1m], "roc_auc":[auc]}).to_csv("spherical_kmeans_summary.csv", index=False)

print("\nHYPERPARAMS:", {"svd_components": svd_dim, "row_normalize": "l2", "k": k,
                          "batch_size": 2048, "n_init": 10, "max_iter": 100,
                          "reassignment_ratio": 0.01, "random_state": 42})
print("SCORES:", {"silhouette": sil, "inertia": inertia, "accuracy": acc, "macro_f1": f1m, "roc_auc": auc})


In [ ]:
#K-Modes (categoricals only): sweep k, pick best, report metrics

In [ ]:
# --- K-Modes on raw categoricals only ---
from kmodes.kmodes import KModes
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
from scipy.optimize import linear_sum_assignment
import numpy as np, pandas as pd

# helper: align clusters to binary label and compute metrics
def eval_clusters_vs_binary(labels, y, verbose=True):
    cm = confusion_matrix(y, labels)
    cost = cm.max() - cm
    rows, cols = linear_sum_assignment(cost)
    mapping = {cols[i]: rows[i] for i in range(len(rows))}
    majority = int(y.mode()[0])
    y_pred  = pd.Series(labels, index=y.index).map(mapping).fillna(majority).astype(int)
    p_pos   = y.groupby(labels).mean()
    y_proba = pd.Series(labels, index=y.index).map(p_pos).fillna(y.mean()).astype(float)
    acc = float((y_pred==y).mean())
    f1m = float(f1_score(y, y_pred, average="macro"))
    auc = float(roc_auc_score(y, y_proba))
    if verbose:
        print("\nConfusion (mapped):\n", confusion_matrix(y, y_pred))
        print("\nReport:\n", classification_report(y, y_pred, digits=3))
    return acc, f1m, auc, mapping

# pick the categorical columns (those *not* numeric in your current use_cols)
Xcat = df[[c for c in use_cols if c not in num_cols]].copy()

# coerce dtypes so imputation/string conversion is safe
Xcat = Xcat.replace({pd.NA: np.nan})
for c in Xcat.columns:
    # ensure extension booleans etc. don't block fillna
    Xcat[c] = Xcat[c].astype("object")
Xcat = Xcat.fillna("missing").astype(str)

results, best = [], None
for k in [3,4,5,6,7,8,9]:
    km = KModes(n_clusters=k, init="Huang", n_init=5, max_iter=50, random_state=42, verbose=0)
    labels_km = km.fit_predict(Xcat)
    acc, f1m, auc, _ = eval_clusters_vs_binary(labels_km, y, verbose=False)
    results.append({"algo":"KModes","k":k,"init":"Huang","n_init":5,"max_iter":50,
                    "cost": float(km.cost_), "accuracy":acc, "macro_f1":f1m, "roc_auc":auc})
    if best is None or f1m > best["macro_f1"]:
        best = {"k":k, "km":km, "labels":labels_km, "accuracy":acc, "macro_f1":f1m, "roc_auc":auc}

pd.DataFrame(results).sort_values(["macro_f1","accuracy"], ascending=False)
print(f"\nBest K-Modes (k={best['k']})  acc={best['accuracy']:.3f}  macro-F1={best['macro_f1']:.3f}  ROC-AUC={best['roc_auc']:.3f}")
_ = eval_clusters_vs_binary(best["labels"], y, verbose=True)

# persist assignments for hybrid
df["cluster_kmodes"] = best["labels"]


In [ ]:
#Hybrid (spherical K-Means + K-Modes → supervised logistic)

In [ ]:
# --- 1) Build clean hybrid feature table -------------------------------------
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score

# extra features from clustering
extra_num = [c for c in df.columns if c.startswith("sphcosdist_")]         # cosine dists
extra_cat = [c for c in ["cluster_sph", "cluster_kmodes"] if c in df]      # cluster IDs

hyb_cols = list(dict.fromkeys(use_cols + extra_num + extra_cat))           # keep order, drop dups
Xh = df[hyb_cols].copy()

# Treat cluster IDs as categoricals (so they go through the OHE branch)
for c in extra_cat:
    Xh[c] = Xh[c].astype("object")

# ---- critical sanitization: remove pandas extension dtypes / pd.NA ----------
# (a) numeric extension -> plain float64 so np.nan is valid
num_ext = Xh.select_dtypes(include=["Int64", "UInt64", "Float64"]).columns
if len(num_ext):
    Xh[num_ext] = Xh[num_ext].astype("float64")

# (b) booleans / strings -> plain object
bool_ext = Xh.select_dtypes(include=["boolean"]).columns
if len(bool_ext):
    Xh[bool_ext] = Xh[bool_ext].astype("object")

str_ext = Xh.select_dtypes(include=["string"]).columns
if len(str_ext):
    Xh[str_ext] = Xh[str_ext].astype("object")

# (c) replace *all* pd.NA with np.nan, then ensure object cols only carry np.nan
Xh = Xh.replace({pd.NA: np.nan})
obj_cols_all = Xh.select_dtypes(include=["object", "category"]).columns
for c in obj_cols_all:
    # .isna() handles pd.NA masks safely; assignment writes real np.nan scalars
    mask = Xh[c].isna()
    if mask.any():
        Xh.loc[mask, c] = np.nan

# Re-detect numeric vs categorical after coercions
num_cols_h = list(Xh.select_dtypes(include=[np.number]).columns)
cat_cols_h = [c for c in Xh.columns if c not in num_cols_h]

# --- 2) Preprocess + model ----------------------------------------------------
pre_h = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc",  StandardScaler(with_mean=False)),
        ]), num_cols_h),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh",  OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
        ]), cat_cols_h),
    ],
    sparse_threshold=1.0,
    verbose_feature_names_out=False,
)

Xtr, Xte, ytr, yte = train_test_split(
    Xh, y, test_size=0.20, random_state=42, stratify=y
)

logit = LogisticRegression(
    solver="saga", penalty="elasticnet", l1_ratio=0.2, C=0.3,
    max_iter=2000, class_weight="balanced", n_jobs=-1
)

pipe_hybrid = Pipeline([("pre", pre_h), ("clf", logit)])
pipe_hybrid.fit(Xtr, ytr)

# --- 3) Metrics ---------------------------------------------------------------
proba = pipe_hybrid.predict_proba(Xte)[:, 1]
pred  = (proba >= 0.50).astype(int)

acc = float((pred == yte).mean())
f1m = float(f1_score(yte, pred, average="macro"))
auc = float(roc_auc_score(yte, proba))

print(f"\nHYBRID Logistic — acc={acc:.3f}  macro-F1={f1m:.3f}  ROC-AUC={auc:.3f}")
print("\nConfusion matrix @0.5:\n", confusion_matrix(yte, pred))
print("\nClassification report:\n", classification_report(yte, pred, digits=3))


In [ ]:
# --- clean hybrid feature table (safe dtypes + safe NaNs) ---
Xh = df[hyb_cols].copy(deep=True)

# cluster ids should be categorical
for c in extra_cat:
    if c in Xh:
        Xh[c] = Xh[c].astype("object")

# 1) extension numerics -> float64 (so np.nan works)
to_float = Xh.select_dtypes(include=["Int64","UInt64","Float64"]).columns
if len(to_float):
    Xh[to_float] = Xh[to_float].astype("float64")

# 2) extension categoricals -> plain object (string/boolean/category)
to_obj = Xh.select_dtypes(include=["string","boolean","category"]).columns
if len(to_obj):
    Xh[to_obj] = Xh[to_obj].astype("object")

# 3) unify missing values WITHOUT per-cell assignment
Xh = Xh.replace({pd.NA: np.nan}).infer_objects(copy=False)  # silences the FutureWarning

# 4) re-detect column sets
num_cols_h = list(Xh.select_dtypes(include=[np.number]).columns)
cat_cols_h = [c for c in Xh.columns if c not in num_cols_h]

print(f"num={len(num_cols_h)}  cat={len(cat_cols_h)}  total={len(Xh.columns)}")


In [ ]:
#Fit & evaluate the Hybrid (clusters-as-features) → Logistic (elastic-net)

In [ ]:
# --- Train/test split (uses your existing binary y) ---
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, f1_score
)

Xtr, Xte, ytr, yte = train_test_split(
    Xh, y, test_size=0.20, random_state=42, stratify=y
)

# --- Preprocess (sparse-safe) ---
pre_h = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc",  StandardScaler(with_mean=False)),
        ]), num_cols_h),
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh",  OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
        ]), cat_cols_h),
    ],
    sparse_threshold=1.0,
    verbose_feature_names_out=False,
)

# --- Classifier (your best params from earlier) ---
logit = LogisticRegression(
    solver="saga", penalty="elasticnet",
    l1_ratio=0.2, C=0.3, max_iter=2000,
    class_weight="balanced", n_jobs=-1
)

pipe_hybrid = Pipeline([("pre", pre_h), ("clf", logit)])

pipe_hybrid.fit(Xtr, ytr)

# --- Metrics (holdout) ---
proba = pipe_hybrid.predict_proba(Xte)[:, 1]
pred  = (proba >= 0.50).astype(int)

acc = float((pred == yte).mean())
f1m = float(f1_score(yte, pred, average="macro"))
auc = float(roc_auc_score(yte, proba))

print(f"\nHYBRID Logistic — acc={acc:.3f}  macro-F1={f1m:.3f}  ROC-AUC={auc:.3f}")
print("\nConfusion matrix @0.50:\n", confusion_matrix(yte, pred))
print("\nClassification report:\n", classification_report(yte, pred, digits=3))

# hyperparams 
hyb_params = {
    "model": "Hybrid(LogReg-elasticnet)",
    "C": 0.3, "l1_ratio": 0.2, "solver": "saga",
    "max_iter": 2000, "class_weight": "balanced",
    "pre_num_imp": "median", "pre_num_scaler": "StandardScaler(with_mean=False)",
    "pre_cat_imp": "most_frequent", "pre_cat_enc": "OHE(ignore)",
    "features_num": len(num_cols_h), "features_cat": len(cat_cols_h),
}
hyb_scores = {"accuracy": acc, "macro_f1": f1m, "roc_auc": auc}
print("\nHYPERPARAMS:", hyb_params)
print("SCORES:", hyb_scores)


In [ ]:
#5-fold CV

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"acc": "accuracy", "f1m": "f1_macro", "auc": "roc_auc"}

cv_res = cross_validate(
    pipe_hybrid, Xh, y, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False
)
print("\nCV means (±std):",
      f"acc={cv_res['test_acc'].mean():.3f}±{cv_res['test_acc'].std():.3f}  ",
      f"f1m={cv_res['test_f1m'].mean():.3f}±{cv_res['test_f1m'].std():.3f}  ",
      f"auc={cv_res['test_auc'].mean():.3f}±{cv_res['test_auc'].std():.3f}")


In [ ]:

#DO NOT RUN.... TAKES 5-6 hrs




from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import pandas as pd

param_dist = {
    "clf__C": np.logspace(-2, 1, 30),             # 0.01 … 10
    "clf__l1_ratio": np.linspace(0.0, 1.0, 21),   # 0 (pure L2) … 1 (pure L1)
}

rs = RandomizedSearchCV(
    estimator=pipe_hybrid,
    param_distributions=param_dist,
    n_iter=15,               
    cv=cv,
    scoring={"acc":"accuracy", "f1m":"f1_macro", "auc":"roc_auc"},
    refit="f1m",
    n_jobs=-1,
    verbose=1,
    random_state=42
)

rs.fit(Xh, y)

print("Best params:", rs.best_params_)
print("Best CV macro-F1:", round(rs.best_score_, 4))

# tidy search results table for the appendix
r = pd.DataFrame(rs.cv_results_)
cols = [
    "param_clf__C", "param_clf__l1_ratio",
    "mean_test_acc", "std_test_acc",
    "mean_test_f1m", "std_test_f1m",
    "mean_test_auc", "std_test_auc",
    "rank_test_f1m"
]
r = r[cols].sort_values("rank_test_f1m")
r.to_csv("artifacts/hybrid_logreg_randomsearch_results.csv", index=False)
print("Saved: artifacts/hybrid_logreg_randomsearch_results.csv")

best_hybrid = rs.best_estimator_


In [ ]:
param_dist = {
    "clf__C": np.logspace(-1, 0.5, 15),        # narrower, 0.1 … ~3.16
    "clf__l1_ratio": np.linspace(0.2, 0.8, 13)
}

rs = RandomizedSearchCV(
    estimator=pipe_hybrid,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring="f1_macro",
    refit=True,
    n_jobs=4,                  # reduce oversubscription
    pre_dispatch="2*n_jobs",
    verbose=2,
    random_state=42
).fit(Xh, y)


In [ ]:
#This focuses the space around your previously good params (C≈0.3, l1_ratio≈0.2), raises max_iter, relaxes tol, and keeps things stable/fast.

In [ ]:
# assumes Xtr, Xte, ytr, yte, pre_h are already in memory (from earlier cells)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

logit = LogisticRegression(
    solver="saga",
    penalty="elasticnet",
    class_weight="balanced",
    max_iter=4000,        # more room to converge
    tol=1e-3,             # slightly looser for speed
    n_jobs=-1             # harmless on binary; leave set
)
pipe_hybrid = Pipeline([("pre", pre_h), ("clf", logit)])

param_dist_trim = {
    "clf__C": np.logspace(-1.3, 0.2, 12),       # ~0.05 … 1.58, focuses near 0.3
    "clf__l1_ratio": np.linspace(0.15, 0.35, 9) # focuses near 0.2
}

rs_trim = RandomizedSearchCV(
    estimator=pipe_hybrid,
    param_distributions=param_dist_trim,
    n_iter=15,
    cv=3,
    scoring="f1_macro",
    refit=True,           # refit best on Xtr
    n_jobs=4,
    pre_dispatch="2*n_jobs",
    verbose=1,
    random_state=42
)
rs_trim.fit(Xtr, ytr)

print("Best params:", rs_trim.best_params_)
print("Best CV macro-F1:", rs_trim.best_score_)
best_hybrid = rs_trim.best_estimator_


In [ ]:
#lock Results

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from joblib import dump
import json, pandas as pd

print("Best params (CV):", rs_trim.best_params_)
print("Best CV macro-F1:", rs_trim.best_score_)

# 1) Finalize the tuned model and evaluate on hold-out
best_hybrid = rs_trim.best_estimator_

proba_te = best_hybrid.predict_proba(Xte)[:, 1]
pred05   = (proba_te >= 0.50).astype(int)

acc05 = accuracy_score(yte, pred05)
f1m05 = f1_score(yte, pred05, average="macro")
auc   = roc_auc_score(yte, proba_te)
cm05  = confusion_matrix(yte, pred05)

print(f"\nHYBRID(LogReg-enet) — hold-out @0.50  acc={acc05:.3f}  macro-F1={f1m05:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @0.50:\n", cm05)
print("\nReport @0.50:\n", classification_report(yte, pred05, digits=3))

# 2) (Optional) pick threshold that maximizes macro-F1 and re-report
ths = np.linspace(0.1, 0.9, 81)
f1s = [f1_score(yte, (proba_te >= t).astype(int), average="macro") for t in ths]
t_star = float(ths[int(np.argmax(f1s))])

pred_star = (proba_te >= t_star).astype(int)
acc_star  = accuracy_score(yte, pred_star)
f1m_star  = f1_score(yte, pred_star, average="macro")
cm_star   = confusion_matrix(yte, pred_star)

print(f"\nBest threshold for macro-F1: {t_star:.3f}")
print(f"Hold-out @{t_star:.3f}  acc={acc_star:.3f}  macro-F1={f1m_star:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @t*:\n", cm_star)

# 3) Save artifacts for the report/repro
dump(best_hybrid, "hybrid_logreg_enet_best.joblib")

summary = {
    "search": {"type": "RandomizedSearchCV", "n_iter": 15, "cv": 3},
    "best_params": rs_trim.best_params_,
    "cv_best_macro_f1": float(rs_trim.best_score_),
    "holdout_thr_0.50": {
        "accuracy": float(acc05), "macro_f1": float(f1m05),
        "roc_auc": float(auc), "confusion": cm05.tolist()
    },
    "holdout_thr_star": {
        "threshold": t_star, "accuracy": float(acc_star),
        "macro_f1": float(f1m_star), "roc_auc": float(auc),
        "confusion": cm_star.tolist()
    }
}
json.dump(summary, open("hybrid_logreg_enet_summary.json","w"), indent=2)

cv_df = pd.DataFrame(rs_trim.cv_results_)
cv_df.to_csv("hybrid_logreg_enet_cv_results.csv", index=False)

# 4) Emit a one-row table for your doc
row = {
    "Model": "Hybrid(LogReg-enet)",
    "Search": "RandomizedSearch",
    "n_iter": 15,
    "cv": 3,
    "C": rs_trim.best_params_["clf__C"],
    "l1_ratio": rs_trim.best_params_["clf__l1_ratio"],
    "Accuracy@0.50": acc05,
    "MacroF1@0.50": f1m05,
    "ROC-AUC": auc,
    "BestThr": t_star,
    "Accuracy@BestThr": acc_star,
    "MacroF1@BestThr": f1m_star,
    "CM@0.50": cm05.tolist(),
    "CM@BestThr": cm_star.tolist(),
}
pd.DataFrame([row]).to_csv("hybrid_logreg_enet_table_row.csv", index=False)
print("\nSaved: hybrid_logreg_enet_best.joblib, hybrid_logreg_enet_summary.json, hybrid_logreg_enet_cv_results.csv, hybrid_logreg_enet_table_row.csv")


In [ ]:
# === Save hybrid visuals & table row (no refit required) ===
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

# ---- 0) Where to save ----
OUT_DIR = Path("report_artifacts_hybrid")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- 1) Helpers ----
def compute_at_threshold(y_true, proba, t):
    pred = (proba >= t).astype(int)
    acc  = float((pred == y_true).mean())
    f1m  = float(f1_score(y_true, pred, average="macro"))
    auc  = float(roc_auc_score(y_true, proba))
    cm   = confusion_matrix(y_true, pred)
    return pred, acc, f1m, auc, cm

def best_threshold_f1_macro(y_true, proba, grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.95, 181)
    f1s = [f1_score(y_true, (proba >= t).astype(int), average="macro") for t in grid]
    i   = int(np.argmax(f1s))
    return float(grid[i]), float(f1s[i])

def plot_confusion(cm, title, path):
    fig, ax = plt.subplots(figsize=(4, 3.6), dpi=150)
    im = ax.imshow(cm, aspect="auto")
    ax.set_title(title)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, f"{int(cm[i,j])}", ha="center", va="center")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout()
    fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.close(fig)

def plot_bars(values, labels, title, path):
    fig, ax = plt.subplots(figsize=(5.5, 3.6), dpi=150)
    x = np.arange(len(values))
    ax.bar(x, values)
    ax.set_xticks(x); ax.set_xticklabels(labels, rotation=0)
    ax.set_ylim(0, 1)
    ax.set_title(title)
    for i, v in enumerate(values):
        ax.text(i, v + (0.02 if v < 0.95 else -0.05), f"{v:.3f}", ha="center",
                va="bottom" if v < 0.95 else "top")
    fig.tight_layout()
    fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.close(fig)

# ---- 2) Inputs expected in memory: yte (Series/array), proba (np.array of positive-class probs)
y_true = np.asarray(yte)
p_pos  = np.asarray(proba)

# Baseline @0.50
pred50, acc50, f1m50, auc50, cm50 = compute_at_threshold(y_true, p_pos, 0.50)
# Best macro-F1 threshold
t_star, f1m_star_grid = best_threshold_f1_macro(y_true, p_pos)
preds, accs, f1ms, aucs, cms = compute_at_threshold(y_true, p_pos, t_star)

# ---- 3) Save figures ----
plot_confusion(cm50, "Hybrid — Confusion Matrix (hold-out @0.50)",
               OUT_DIR / "hybrid_confusion_050.png")
plot_confusion(cms,  f"Hybrid — Confusion Matrix (hold-out @{t_star:.3f})",
               OUT_DIR / "hybrid_confusion_bestthr.png")

plot_bars([acc50, f1m50, auc50],
          ["Acc(0.50)", "F1m(0.50)", "AUC"],
          "Hold-out metrics @0.50",
          OUT_DIR / "hybrid_metrics_050.png")

plot_bars([accs, f1ms, aucs],
          [f"Acc({t_star:.3f})", f"F1m({t_star:.3f})", "AUC"],
          "Hold-out metrics @t* (best F1m)",
          OUT_DIR / "hybrid_metrics_tstar.png")

# ---- 4) Save a one-line table row for the report ----
try:
    C  = float(pipe_hybrid.named_steps["clf"].C)
    L1 = float(pipe_hybrid.named_steps["clf"].l1_ratio)
except Exception:
    C  = np.nan
    L1 = np.nan

cv_f1m = float(rs.best_score_) if "rs" in locals() else np.nan

row = pd.DataFrame([{
    "Model": "Hybrid(LogReg-enet)",
    "C": C, "l1_ratio": L1,
    "CV_macroF1": cv_f1m,
    "thr_050_acc": acc50, "thr_050_macroF1": f1m50, "thr_050_auc": auc50,
    "thr_star": t_star, "thr_star_acc": accs, "thr_star_macroF1": f1ms, "thr_star_auc": aucs
}])
row.to_csv(OUT_DIR / "hybrid_report_table_row.csv", index=False)

print("Saved:", OUT_DIR.resolve())



In [ ]:
# === Inline visuals for Hybrid(LogReg-enet) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import confusion_matrix, roc_curve, auc, f1_score, roc_auc_score

# Expect these to exist from your previous steps:
#   yte  -> hold-out labels (array-like, 0/1)
#   proba -> predicted proba for positive class (np.array)
model_name = "Hybrid(LogReg-enet)"

# ----------------- helpers -----------------
def best_threshold_f1_macro(y, p, grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.95, 181)
    f1s = [f1_score(y, (p >= t).astype(int), average="macro") for t in grid]
    i = int(np.argmax(f1s))
    return float(grid[i]), float(f1s[i])

def metrics_at_t(y, p, t):
    pred = (p >= t).astype(int)
    acc  = float((pred == y).mean())
    f1m  = float(f1_score(y, pred, average="macro"))
    aucv = float(roc_auc_score(y, p))
    cm   = confusion_matrix(y, pred)
    return pred, acc, f1m, aucv, cm

def show_confusion(cm, title):
    fig, ax = plt.subplots(figsize=(5.0, 4.2), dpi=150)
    im = ax.imshow(cm, aspect="auto")
    ax.set_title(title)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, f"{int(cm[i,j])}", ha="center", va="center")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout()
    plt.show()

def show_metric_bars(values, labels, subtitle):
    fig, ax = plt.subplots(figsize=(5.6, 3.6), dpi=150)
    x = np.arange(len(values))
    ax.bar(x, values)
    ax.set_xticks(x); ax.set_xticklabels(labels)
    ax.set_ylim(0, 1)
    ax.set_title(f"{model_name} — {subtitle}")
    for i, v in enumerate(values):
        ax.text(i, v + (0.02 if v < 0.95 else -0.05), f"{v:.3f}",
                ha="center", va=("bottom" if v < 0.95 else "top"))
    fig.tight_layout()
    plt.show()

# ----------------- compute + show -----------------
y_true = np.asarray(yte)
p_pos  = np.asarray(proba)

# Confusions @ 0.50 and @ t*
t_star, _ = best_threshold_f1_macro(y_true, p_pos)
_, acc50, f1m50, auc50, cm50 = metrics_at_t(y_true, p_pos, 0.50)
_, accs,  f1ms,  aucs,  cms  = metrics_at_t(y_true, p_pos, t_star)

show_confusion(cm50, f"{model_name} — Confusion Matrix @0.50")
show_confusion(cms,  f"{model_name} — Confusion Matrix @{t_star:.3f} (best F1m)")

# ROC curve
fpr, tpr, _ = roc_curve(y_true, p_pos)
rocA = auc(fpr, tpr)
fig, ax = plt.subplots(figsize=(5.0, 4.2), dpi=150)
ax.plot(fpr, tpr, label=f"AUC = {rocA:.3f}")
ax.plot([0,1], [0,1], linestyle="--")
ax.set_title(f"{model_name} — ROC Curve")
ax.set_xlabel("False Positive Rate"); ax.set_ylabel("True Positive Rate")
ax.legend(loc="lower right")
fig.tight_layout()
plt.show()

# Metric bars (two separate figures so each is easy to screenshot)
show_metric_bars([acc50, f1m50, auc50], ["Accuracy","Macro-F1","ROC-AUC"], "Hold-out @0.50")
show_metric_bars([accs,  f1ms,  aucs],  ["Accuracy","Macro-F1","ROC-AUC"], f"Hold-out @{t_star:.3f}")

# Small summary table (renders inline)
summary = pd.DataFrame({
    "Threshold": ["0.50", f"{t_star:.3f} (best F1m)"],
    "Accuracy":  [acc50, accs],
    "Macro-F1":  [f1m50, f1ms],
    "ROC-AUC":   [auc50, aucs]
})
summary

# (Optional) also save the figures to your folder for consistency with earlier step
OUT_DIR = Path("report_artifacts_hybrid"); OUT_DIR.mkdir(exist_ok=True, parents=True)
# Re-plot quickly for saving:
for title, cm, fname in [
    (f"{model_name} — Confusion Matrix @0.50", cm50, "hybrid_confusion_050.png"),
    (f"{model_name} — Confusion Matrix @{t_star:.3f}", cms, "hybrid_confusion_bestthr.png"),
]:
    fig, ax = plt.subplots(figsize=(5.0, 4.2), dpi=150)
    im = ax.imshow(cm, aspect="auto")
    ax.set_title(title); ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, f"{int(cm[i,j])}", ha="center", va="center")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    fig.tight_layout(); fig.savefig(OUT_DIR/fname, dpi=220, bbox_inches="tight"); plt.close(fig)

# Save a one-line CSV row you can drop into a table later
try:
    C  = float(pipe_hybrid.named_steps["clf"].C)
    L1 = float(pipe_hybrid.named_steps["clf"].l1_ratio)
except Exception:
    C, L1 = np.nan, np.nan

cv_f1m = float(rs.best_score_) if "rs" in globals() else np.nan
pd.DataFrame([{
    "Model": model_name, "C": C, "l1_ratio": L1, "CV_macroF1": cv_f1m,
    "thr_050_acc": acc50, "thr_050_macroF1": f1m50, "thr_050_auc": auc50,
    "thr_star": t_star, "thr_star_acc": accs, "thr_star_macroF1": f1ms, "thr_star_auc": aucs
}]).to_csv(OUT_DIR / "hybrid_report_table_row.csv", index=False)


In [ ]:
# === UNSUPERVISED VISUALS: spherical K-Means (k=7) & K-Modes (k=3) ===
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, roc_curve, auc

# ---------- Resolve target ----------
if 'y' in globals():
    y_true = pd.Series(y).astype(int)
elif 'df' in globals() and 'avg_stars_2019' in df.columns:
    y_true = df['avg_stars_2019'].astype(int)
else:
    raise RuntimeError("Couldn't find y or df['avg_stars_2019']. Please define the binary target first.")

# ---------- Helper: evaluate clusters via majority vote + prob from pos-rate ----------
def eval_clusters(cluster_like, y_true, name="clusters"):
    s = pd.Series(cluster_like, name="cluster")
    # try to align on index; if impossible, fall back to positional
    if not s.index.equals(y_true.index):
        s = s.reindex(y_true.index)
        if s.isna().any():
            s = s.reset_index(drop=True)
            y_aligned = y_true.reset_index(drop=True)
        else:
            y_aligned = y_true
    else:
        y_aligned = y_true

    s = s.astype(int)
    ct = pd.crosstab(y_aligned, s)  # rows: true {0,1}, cols: clusters
    # cluster-wise positive rate (used as probability for ROC; NaN->0)
    pos_rate = (ct.loc[1] / ct.sum(axis=0)).fillna(0.0)
    pred = s.map(lambda g: 1 if pos_rate.get(g, 0.0) >= 0.5 else 0).astype(int)
    proba = s.map(pos_rate).fillna(0.0).to_numpy()

    cm = confusion_matrix(y_aligned, pred)
    acc = float((pred == y_aligned).mean())
    f1m = float(f1_score(y_aligned, pred, average="macro"))
    aucv = float(roc_auc_score(y_aligned, proba))

    return dict(name=name, clusters=s, table=ct, pos_rate=pos_rate,
                pred=pred, proba=proba, cm=cm, acc=acc, f1m=f1m, auc=aucv)

# ---------- Plot helpers ----------
def plot_confusion(cm, title, path=None):
    fig, ax = plt.subplots(figsize=(5.2, 4.6), dpi=120)
    im = ax.imshow(cm, interpolation="nearest")
    ax.set_title(title)
    ax.set_xlabel("Predicted label"); ax.set_ylabel("True label")
    ax.set_xticks([0,1]); ax.set_xticklabels(["0","1"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["0","1"])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, f"{int(cm[i,j])}", ha="center", va="center")
    fig.tight_layout()
    if path: fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.show()

def plot_cluster_sizes(ct, title, path=None):
    sizes = ct.sum(axis=0).sort_index()
    fig, ax = plt.subplots(figsize=(6.8, 3.8), dpi=120)
    ax.bar(sizes.index.astype(str), sizes.values)
    ax.set_title(title)
    ax.set_xlabel("Cluster ID"); ax.set_ylabel("Count")
    fig.tight_layout()
    if path: fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.show()

def plot_pos_rate(pos_rate, title, path=None):
    pr = pos_rate.sort_index()
    fig, ax = plt.subplots(figsize=(6.8, 3.8), dpi=120)
    ax.bar(pr.index.astype(str), pr.values)
    ax.set_ylim(0, 1.0)
    ax.set_title(title)
    ax.set_xlabel("Cluster ID"); ax.set_ylabel("Positive rate P(y=1|cluster)")
    fig.tight_layout()
    if path: fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.show()

def plot_roc_curve(y_true, proba, title, path=None):
    fpr, tpr, _ = roc_curve(y_true, proba)
    roc_auc = auc(fpr, tpr)
    fig, ax = plt.subplots(figsize=(5.2, 4.6), dpi=120)
    ax.plot(fpr, tpr, lw=2, label=f"AUC={roc_auc:.3f}")
    ax.plot([0, 1], [0, 1], linestyle="--", lw=1)
    ax.set_title(title); ax.set_xlabel("False Positive Rate"); ax.set_ylabel("True Positive Rate")
    ax.legend(loc="lower right")
    fig.tight_layout()
    if path: fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.show()

def plot_scatter_2d(embed2, clusters, title, path=None, sample=12000, s=6):
    # subsample for speed/visibility
    n = len(clusters)
    idx = np.random.RandomState(42).choice(n, size=min(sample, n), replace=False)
    x, y = embed2[idx, 0], embed2[idx, 1]
    lab = pd.Series(clusters).to_numpy()[idx]
    fig, ax = plt.subplots(figsize=(6.6, 5.6), dpi=120)
    sc = ax.scatter(x, y, c=lab, s=s)
    ax.set_title(title); ax.set_xticks([]); ax.set_yticks([])
    fig.tight_layout()
    if path: fig.savefig(path, dpi=220, bbox_inches="tight")
    plt.show()

# ---------- Make output folder ----------
outdir = Path("outputs_unsup"); outdir.mkdir(exist_ok=True)

# ---------- Evaluate & plot: SPHERICAL K-MEANS ----------
if 'df' in globals() and 'cluster_sph' in df.columns:
    res_sph = eval_clusters(df['cluster_sph'], y_true, name="spherical-kmeans")
    print(f"[Spherical K-Means] acc={res_sph['acc']:.3f}  macro-F1={res_sph['f1m']:.3f}  ROC-AUC={res_sph['auc']:.3f}")
    plot_confusion(res_sph['cm'],
                   "Spherical K-Means — Confusion (majority mapping)",
                   outdir / "sph_confusion.png")
    plot_cluster_sizes(res_sph['table'],
                       "Spherical K-Means — Cluster sizes",
                       outdir / "sph_sizes.png")
    plot_pos_rate(res_sph['pos_rate'],
                  "Spherical K-Means — Positive rate by cluster",
                  outdir / "sph_posrate.png")
    plot_roc_curve(y_true, res_sph['proba'],
                   "Spherical K-Means — ROC (prob=cluster positive rate)",
                   outdir / "sph_roc.png")

    # Optional 2-D scatter if an embedding is available
    embed2 = None
    try:
        if 'Zs' in globals():
            from sklearn.decomposition import TruncatedSVD
            embed2 = TruncatedSVD(n_components=2, random_state=42).fit_transform(Zs)
        elif 'Xs' in globals():
            from sklearn.decomposition import TruncatedSVD
            embed2 = TruncatedSVD(n_components=2, random_state=42).fit_transform(Xs)
    except Exception as e:
        embed2 = None
        print("2-D scatter skipped (no embedding available).")

    if embed2 is not None:
        plot_scatter_2d(embed2, df['cluster_sph'],
                        "Spherical K-Means — 2-D embedding (sampled)",
                        outdir / "sph_scatter2d.png")
else:
    print("⚠️ 'cluster_sph' not found in df; skipping spherical K-Means visuals.")

# ---------- Evaluate & plot: K-MODES ----------
if 'df' in globals() and 'cluster_kmodes' in df.columns:
    res_km = eval_clusters(df['cluster_kmodes'], y_true, name="k-modes")
    print(f"[K-Modes] acc={res_km['acc']:.3f}  macro-F1={res_km['f1m']:.3f}  ROC-AUC={res_km['auc']:.3f}")
    plot_confusion(res_km['cm'],
                   "K-Modes — Confusion (majority mapping)",
                   outdir / "kmodes_confusion.png")
    plot_cluster_sizes(res_km['table'],
                       "K-Modes — Cluster sizes",
                       outdir / "kmodes_sizes.png")
    plot_pos_rate(res_km['pos_rate'],
                  "K-Modes — Positive rate by cluster",
                  outdir / "kmodes_posrate.png")
    plot_roc_curve(y_true, res_km['proba'],
                   "K-Modes — ROC (prob=cluster positive rate)",
                   outdir / "kmodes_roc.png")
else:
    print("⚠️ 'cluster_kmodes' not found in df; skipping K-Modes visuals.")

# ---------- Optional: silhouette/inertia over k (will plot if a grid is present) ----------
grid_like = None
for cand in ("sph_grid", "mbkm_grid_df", "grid"):
    if cand in globals():
        grid_like = globals()[cand]
        break

if isinstance(grid_like, pd.DataFrame) and {"k","silhouette","inertia"}.issubset(grid_like.columns):
    g = grid_like.sort_values("k")
    fig, ax = plt.subplots(figsize=(6.8, 3.6), dpi=120)
    ax.plot(g["k"], g["silhouette"], marker="o")
    ax.set_title("Spherical K-Means — Silhouette vs k")
    ax.set_xlabel("k"); ax.set_ylabel("silhouette")
    fig.tight_layout(); fig.savefig(outdir / "sph_silhouette_vs_k.png", dpi=220, bbox_inches="tight")
    plt.show()

    fig, ax = plt.subplots(figsize=(6.8, 3.6), dpi=120)
    ax.plot(g["k"], g["inertia"], marker="o")
    ax.set_title("Spherical K-Means — Inertia vs k")
    ax.set_xlabel("k"); ax.set_ylabel("inertia")
    fig.tight_layout(); fig.savefig(outdir / "sph_inertia_vs_k.png", dpi=220, bbox_inches="tight")
    plt.show()
else:
    print("ℹ️ Silhouette/Inertia grid not found; skipping the k-sweep plots.")

# ---------- Summary table you can paste into the report ----------
rows = []
if 'res_sph' in locals():
    rows.append(["Spherical K-Means (k=7)", res_sph["acc"], res_sph["f1m"], res_sph["auc"], "outputs_unsup/sph_confusion.png"])
if 'res_km' in locals():
    rows.append(["K-Modes (k=3)", res_km["acc"], res_km["f1m"], res_km["auc"], "outputs_unsup/kmodes_confusion.png"])
if rows:
    summary_df = pd.DataFrame(rows, columns=["Method", "Accuracy", "Macro-F1", "ROC-AUC", "Confusion image"])
    print("\nUnsupervised summary:\n", summary_df.to_string(index=False))


In [ ]:
# Build spherical K-Means k-sweep table -> sph_grid
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD

# 1) Ensure we have a normalized embedding Zs
if 'Zs' not in globals():
    if 'Xs' not in globals():
        raise RuntimeError("Need Xs (sparse features) or Zs (embedding). Define one before running.")
    svd = TruncatedSVD(n_components=80, random_state=42)
    Zs = svd.fit_transform(Xs)
    Zs = normalize(Zs, norm='l2')

# 2) Sweep k and compute silhouette (cosine) + inertia
k_list = [3,4,5,6,7,8,9,10]
rows = []
rng = np.random.RandomState(42)
n_sil = min(12000, Zs.shape[0])                      # subsample for speed
sil_idx = rng.choice(Zs.shape[0], size=n_sil, replace=False)

for k in k_list:
    mbkm = MiniBatchKMeans(
        n_clusters=k, random_state=42, batch_size=2048,
        n_init=10, max_iter=100, reassignment_ratio=0.01
    )
    labels = mbkm.fit_predict(Zs)
    inertia = float(mbkm.inertia_)
    sil = float(silhouette_score(Zs[sil_idx], labels[sil_idx], metric="cosine"))
    rows.append((k, sil, inertia))

sph_grid = pd.DataFrame(rows, columns=["k","silhouette","inertia"]).sort_values("k")
print("Spherical K-Means grid:\n", sph_grid.to_string(index=False))

# 3) (Optional) Persist/update best-k clustering to df['cluster_sph']
best_k = int(sph_grid.sort_values("silhouette", ascending=False).iloc[0].k)
print(f"\nBest k by silhouette: {best_k}")
if 'df' in globals():
    mbkm_best = MiniBatchKMeans(
        n_clusters=best_k, random_state=42, batch_size=2048,
        n_init=10, max_iter=100, reassignment_ratio=0.01
    ).fit(Zs)
    df['cluster_sph'] = mbkm_best.labels_
    print("Updated df['cluster_sph'] with best-k labels.")


In [ ]:
# Plots for spherical K-Means k-sweep (silhouette & inertia)
import matplotlib.pyplot as plt
from pathlib import Path

assert 'sph_grid' in globals(), "Run the k-sweep cell first to create `sph_grid`."

g = sph_grid.sort_values('k').reset_index(drop=True)
outdir = Path("figs"); outdir.mkdir(exist_ok=True)

# Best k by silhouette
best_k = int(g.loc[g['silhouette'].idxmax(), 'k'])
best_sil = float(g['silhouette'].max())

# 1) Silhouette vs k
fig, ax = plt.subplots()
ax.plot(g['k'], g['silhouette'], marker='o')
ax.axvline(best_k, linestyle='--')
ax.set_xlabel('k')
ax.set_ylabel('Silhouette (cosine)')
ax.set_title('Spherical K-Means: Silhouette vs k')
ax.grid(True, alpha=0.3)
fig.savefig(outdir / 'sph_silhouette_vs_k.png', dpi=220, bbox_inches='tight')
plt.show()
print(f"Saved: {outdir / 'sph_silhouette_vs_k.png'}  |  best k={best_k} (silhouette={best_sil:.3f})")

# 2) Inertia vs k
fig, ax = plt.subplots()
ax.plot(g['k'], g['inertia'], marker='o')
ax.set_xlabel('k')
ax.set_ylabel('Inertia')
ax.set_title('Spherical K-Means: Inertia vs k')
ax.grid(True, alpha=0.3)
fig.savefig(outdir / 'sph_inertia_vs_k.png', dpi=220, bbox_inches='tight')
plt.show()
print(f"Saved: {outdir / 'sph_inertia_vs_k.png'}")


In [ ]:
# ==== Quick ensemble check: HGB on dense Zs + clusters ====
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# --- Build a compact, dense feature set ---
assert 'cluster_sph' in df.columns, "cluster_sph not found in df"
if 'cluster_kmodes' not in df.columns:
    df['cluster_kmodes'] = -1  # fallback if not present

# Zs must be a numpy array (n_samples, d). If you used 60 or 80 dims earlier, reuse it.
assert 'Zs' in globals(), "Zs (SVD embedding) is not in memory"

Z = pd.DataFrame(Zs, index=df.index, columns=[f"z{i:02d}" for i in range(Zs.shape[1])])
C = pd.get_dummies(df[['cluster_sph','cluster_kmodes']].astype('category'),
                   drop_first=False, prefix=['sph','km'])
Xb = pd.concat([Z, C], axis=1)

Xtr, Xte, ytr, yte = train_test_split(Xb, y, test_size=0.20, random_state=42, stratify=y)

hgb = HistGradientBoostingClassifier(
    learning_rate=0.07,
    max_leaf_nodes=31,
    min_samples_leaf=20,
    max_iter=500,
    l2_regularization=1e-4,
    class_weight='balanced',
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)

hgb.fit(Xtr, ytr)
proba = hgb.predict_proba(Xte)[:, 1]
pred  = (proba >= 0.50).astype(int)

acc = accuracy_score(yte, pred)
f1m = f1_score(yte, pred, average="macro")
auc = roc_auc_score(yte, proba)
cm  = confusion_matrix(yte, pred)

print(f"HGB(Zs+clusters) — hold-out @0.50  acc={acc:.3f}  macro-F1={f1m:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @0.50:\n", cm)
print("\nReport:\n", classification_report(yte, pred, digits=3))

# quick 3×CV for sanity (uses all rows)
cv = StratifiedKFold(3, shuffle=True, random_state=42)
scores = cross_validate(
    hgb, Xb, y,
    scoring={"acc":"accuracy","f1m":"f1_macro","auc":"roc_auc"},
    cv=cv, n_jobs=-1, return_train_score=False
)
print("CV means: acc={:.3f}  f1m={:.3f}  auc={:.3f}".format(
    scores["test_acc"].mean(), scores["test_f1m"].mean(), scores["test_auc"].mean()
))


In [ ]:
# === HGB: threshold sweep + ROC/PR + confusion visuals ===
import os, numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score,
    f1_score, confusion_matrix, classification_report, accuracy_score
)

# --- Safety checks & probability vector ---------------------------------------
if 'yte' not in globals():
    raise RuntimeError("`yte` not found. Run the HGB train/test split cell first.")

try:
    _ = proba  # reuse if it exists
except NameError:
    if 'hgb' not in globals() or 'Xte' not in globals():
        raise RuntimeError("Missing `hgb` and/or `Xte`. Run the HGB training cell first.")
    proba = hgb.predict_proba(Xte)[:, 1]

# --- Metrics @ fixed threshold (0.50) -----------------------------------------
pred_050 = (proba >= 0.50).astype(int)
acc_050 = accuracy_score(yte, pred_050)
f1m_050 = f1_score(yte, pred_050, average="macro")
auc_val = roc_auc_score(yte, proba)   # same for any threshold
cm_050  = confusion_matrix(yte, pred_050)

print(f"HGB — hold-out @0.50  acc={acc_050:.3f}  macro-F1={f1m_050:.3f}  ROC-AUC={auc_val:.3f}")
print("Confusion @0.50:\n", cm_050)
print("\nReport @0.50:\n", classification_report(yte, pred_050, digits=3))

# --- Threshold sweep for macro-F1 ---------------------------------------------
ths = np.linspace(0.01, 0.99, 99)
f1s = [f1_score(yte, (proba >= t).astype(int), average="macro") for t in ths]
best_idx = int(np.argmax(f1s))
t_star   = float(ths[best_idx])
f1m_star = float(f1s[best_idx])
pred_star = (proba >= t_star).astype(int)
acc_star  = accuracy_score(yte, pred_star)
cm_star   = confusion_matrix(yte, pred_star)

print(f"\nBest threshold for macro-F1: {t_star:.3f}")
print(f"Hold-out @{t_star:.3f}  acc={acc_star:.3f}  macro-F1={f1m_star:.3f}  ROC-AUC={auc_val:.3f}")
print("Confusion @t*:\n", cm_star)

# --- Prepare output dir --------------------------------------------------------
out_dir = "figs"
os.makedirs(out_dir, exist_ok=True)

# --- Helper: plot confusion matrix (counts) -----------------------------------
def plot_confusion(cm, title, out_path):
    fig, ax = plt.subplots(figsize=(5, 4))
    im = ax.imshow(cm)
    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_xticks([0, 1]); ax.set_yticks([0, 1])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, f"{int(cm[i, j])}", ha="center", va="center")
    fig.tight_layout()
    fig.savefig(out_path, dpi=220, bbox_inches="tight")
    plt.show()

# --- 1) ROC curve --------------------------------------------------------------
fpr, tpr, _ = roc_curve(yte, proba)
fig = plt.figure(figsize=(5, 4))
plt.plot(fpr, tpr, label=f"AUC={auc_val:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("HGB (Zs+clusters) — ROC")
plt.legend()
plt.tight_layout()
fig.savefig(f"{out_dir}/hgb_roc.png", dpi=220, bbox_inches="tight")
plt.show()

# --- 2) Precision–Recall curve -------------------------------------------------
prec, rec, _ = precision_recall_curve(yte, proba)
ap = average_precision_score(yte, proba)
fig = plt.figure(figsize=(5, 4))
plt.plot(rec, prec, label=f"AP={ap:.3f}")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title("HGB (Zs+clusters) — Precision–Recall")
plt.legend()
plt.tight_layout()
fig.savefig(f"{out_dir}/hgb_pr.png", dpi=220, bbox_inches="tight")
plt.show()

# --- 3) Threshold sweep plot ---------------------------------------------------
fig = plt.figure(figsize=(6, 4))
plt.plot(ths, f1s)
plt.axvline(0.50, linestyle="--")
plt.axvline(t_star, linestyle=":")
plt.xlabel("Threshold"); plt.ylabel("Macro-F1")
plt.title("HGB — Macro-F1 vs Threshold")
plt.tight_layout()
fig.savefig(f"{out_dir}/hgb_threshold_sweep.png", dpi=220, bbox_inches="tight")
plt.show()

# --- 4) Confusion matrices (@0.50 and @t*) ------------------------------------
plot_confusion(cm_050, "HGB — Confusion (hold-out @0.50)", f"{out_dir}/hgb_confusion_050.png")
plot_confusion(cm_star, f"HGB — Confusion (hold-out @{t_star:.3f})", f"{out_dir}/hgb_confusion_best.png")

print("\nSaved figure files:")
for f in ["hgb_roc.png", "hgb_pr.png", "hgb_threshold_sweep.png", "hgb_confusion_050.png", "hgb_confusion_best.png"]:
    p = os.path.join(out_dir, f)
    print(" -", p)


In [ ]:
import os, json, joblib
from datetime import datetime
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score

os.makedirs("artifacts/hgb", exist_ok=True)

# assume yte and proba from your last cell
metrics = {
    "model": "HGB (hybrid features)",
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "threshold_default": 0.50,
    "threshold_star": float(0.530),   # from your output
    "acc@0.50": float(accuracy_score(yte, (proba>=0.50).astype(int))),
    "f1_macro@0.50": float(f1_score(yte, (proba>=0.50).astype(int), average="macro")),
    "acc@t*": float(accuracy_score(yte, (proba>=0.530).astype(int))),
    "f1_macro@t*": float(f1_score(yte, (proba>=0.530).astype(int), average="macro")),
    "roc_auc": float(roc_auc_score(yte, proba))
}

with open("artifacts/hgb/hgb_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Save the estimator object (pipeline or bare model), whichever you used
to_save = None
if 'pipe_hgb' in globals():
    to_save = pipe_hgb
elif 'hgb' in globals():
    to_save = hgb
else:
    raise RuntimeError("No HGB estimator found (hgb/pipe_hgb).")

joblib.dump(to_save, "artifacts/hgb/hgb_model.joblib")

# Also keep a one-row CSV for your report table
import pandas as pd
row = pd.DataFrame([{
    "Model":"HGB (hybrid)",
    "Tuned?":"No (baseline)",
    "Threshold":"0.530",
    "Accuracy":metrics["acc@t*"],
    "Macro-F1":metrics["f1_macro@t*"],
    "ROC-AUC":metrics["roc_auc"]
}])
row.to_csv("artifacts/hgb/hgb_table_row.csv", index=False)

print("Saved: artifacts/hgb/hgb_model.joblib, hgb_metrics.json, hgb_table_row.csv")


In [ ]:
# --- HGB with SVD compression (recommended) ---
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score,
                             confusion_matrix)

# pre_h = your existing ColumnTransformer (impute + OHE that yields sparse)
# Xh, y, and your holdout split (Xte, yte) already exist from earlier cells.

pipe_hgb_svd = Pipeline([
    ("pre", pre_h),                                    # sparse out
    ("svd", TruncatedSVD(n_components=120, random_state=42)),
    ("sc",  StandardScaler()),                         # dense scaling
    ("clf", HistGradientBoostingClassifier(
        loss="log_loss",
        early_stopping=True,
        validation_fraction=0.1,
        class_weight="balanced",
        random_state=42
    ))
])

param_dist = {
    "svd__n_components": [80, 120, 160],
    "clf__learning_rate": list(np.logspace(-2.2, -0.5, 6)),  # ~0.006–0.316
    "clf__max_iter": [200, 300, 400],
    "clf__max_depth": [None, 3, 4],
    "clf__max_leaf_nodes": [31, 63],
    "clf__min_samples_leaf": [20, 50],
    "clf__l2_regularization": [0.0, 0.1, 1.0],
    "clf__max_bins": [255],
}

rs_hgb = RandomizedSearchCV(
    pipe_hgb_svd,
    param_distributions=param_dist,
    n_iter=12, cv=3, scoring="f1_macro",
    n_jobs=-1, verbose=2, random_state=42, refit=True
)
rs_hgb.fit(Xh, y)

print("Best params:", rs_hgb.best_params_)
print("Best CV macro-F1:", rs_hgb.best_score_)

# ---- Hold-out evaluation (same style as before) ----
proba = rs_hgb.predict_proba(Xte)[:, 1]
pred50 = (proba >= 0.50).astype(int)

ths = np.linspace(0.01, 0.99, 99)
f1s = [f1_score(yte, (proba >= t).astype(int), average="macro") for t in ths]
t_star = float(ths[int(np.argmax(f1s))])
pred_star = (proba >= t_star).astype(int)

print(f"\nTUNED HGB(SVD) — @0.50  acc={accuracy_score(yte,pred50):.3f}  "
      f"f1m={f1_score(yte,pred50,average='macro'):.3f}  auc={roc_auc_score(yte,proba):.3f}")
print(f"TUNED HGB(SVD) — @{t_star:.3f}  acc={accuracy_score(yte,pred_star):.3f}  "
      f"f1m={f1_score(yte,pred_star,average='macro'):.3f}")
print("Confusion @t*:\n", confusion_matrix(yte, pred_star))


In [ ]:
# --- Evaluate tuned HGB(SVD) on a consistent hold-out -----------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np

# 1) Make a FRESH split from the SAME Xh,y you used for RandomizedSearchCV
Xtr2, Xte2, ytr2, yte2 = train_test_split(
    Xh, y, test_size=0.20, random_state=42, stratify=y
)

# 2) Grab the tuned pipeline and refit on TRAIN ONLY
best_hgb = rs_hgb.best_estimator_      # this is your Pipeline(pre -> svd -> sc -> HGB)
best_hgb.fit(Xtr2, ytr2)

# 3) Predict on the aligned test slice
proba = best_hgb.predict_proba(Xte2)[:, 1]
pred50 = (proba >= 0.50).astype(int)

# threshold sweep for best macro-F1
ths = np.linspace(0.01, 0.99, 99)
f1s = [f1_score(yte2, (proba >= t).astype(int), average="macro") for t in ths]
t_star = float(ths[int(np.argmax(f1s))])
pred_star = (proba >= t_star).astype(int)

# 4) Metrics
acc50 = accuracy_score(yte2, pred50)
f1m50 = f1_score(yte2, pred50, average="macro")
auc   = roc_auc_score(yte2, proba)

accs  = accuracy_score(yte2, pred_star)
f1ms  = f1_score(yte2, pred_star, average="macro")

print("Best params (CV):", rs_hgb.best_params_)
print(f"Best CV macro-F1: {rs_hgb.best_score_:.3f}")

print(f"\nTUNED HGB(SVD) — hold-out @0.50  acc={acc50:.3f}  macro-F1={f1m50:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @0.50:\n", confusion_matrix(yte2, pred50))

print(f"\nBest threshold for macro-F1: {t_star:.3f}")
print(f"Hold-out @{t_star:.3f}  acc={accs:.3f}  macro-F1={f1ms:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @t*:\n", confusion_matrix(yte2, pred_star))


In [ ]:
# ===== Bulletproof eval for tuned HGB(SVD): hard NA sanitization + eval =====
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

best = rs_hgb.best_estimator_         # Pipeline(pre -> svd -> hgb)
pre  = best.named_steps["pre"]         # ColumnTransformer

# --- 1) Recover exact training columns and split (num/cat) ---
try:
    expected_cols = list(pre.feature_names_in_)
except AttributeError:
    expected_cols = []
    for name, trans, cols in pre.transformers_:
        expected_cols.extend(list(cols))

num_cols, cat_cols = [], []
for name, trans, cols in pre.transformers_:
    if name == "num":
        num_cols = list(cols)
    elif name == "cat":
        cat_cols = list(cols)

# --- 2) Build DataFrame with EXACT columns; HARD sanitize all pd.NA -> np.nan ---
Xall = df.reindex(columns=expected_cols)

# 2a) Nuke all extension dtypes by going to object first (so replacements stick)
Xall = Xall.astype("object")

# 2b) Replace ANY missing-like value (incl. pd.NA) with real np.nan in one shot
Xall = Xall.where(pd.notna(Xall), np.nan)

# 2c) Restore numeric branch to float64; coerce stray strings to NaN
if num_cols:
    Xall.loc[:, num_cols] = (
        Xall[num_cols]
        .apply(pd.to_numeric, errors="coerce")
        .astype("float64")
    )

# 2d) Categorical branch stays as plain 'object'

# --- 3) Safety net: if any pd.NA still survives, force-replace per-cell ---
stubborn_cols = []
for c in Xall.columns:
    s = Xall[c]
    if s.dtype == "object":
        if any((v is pd.NA) for v in s.values):
            stubborn_cols.append(c)
if stubborn_cols:
    # Slow but surgical: guaranteed pd.NA -> np.nan conversion
    Xall.loc[:, stubborn_cols] = Xall[stubborn_cols].applymap(
        lambda v: np.nan if v is pd.NA else v
    )

# --- 4) Use your existing hold-out indices if available, else recreate ---
try:
    te_idx  = yte.index
    Xte     = Xall.loc[te_idx]
    y_eval  = yte
except NameError:
    _, Xte, _, y_eval = train_test_split(
        Xall, y, test_size=0.20, random_state=42, stratify=y
    )

# --- 5) Predict + threshold sweep (macro-F1) ---
proba  = best.predict_proba(Xte)[:, 1]
pred50 = (proba >= 0.50).astype(int)

ths = np.linspace(0.01, 0.99, 99)
f1s = [f1_score(y_eval, (proba >= t).astype(int), average="macro") for t in ths]
t_star = float(ths[int(np.argmax(f1s))])
predst = (proba >= t_star).astype(int)

auc   = float(roc_auc_score(y_eval, proba))
acc50 = float((pred50 == y_eval).mean())
f1m50 = float(f1_score(y_eval, pred50, average="macro"))
accst = float((predst == y_eval).mean())
f1mst = float(f1_score(y_eval, predst, average="macro"))

print(f"TUNED HGB(SVD) — hold-out @0.50  acc={acc50:.3f}  macro-F1={f1m50:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @0.50:\n", confusion_matrix(y_eval, pred50))
print(f"\nBest threshold for macro-F1: {t_star:.3f}")
print(f"Hold-out @{t_star:.3f}  acc={accst:.3f}  macro-F1={f1mst:.3f}  ROC-AUC={auc:.3f}")
print("Confusion @t*:\n", confusion_matrix(y_eval, predst))


In [ ]:
# ==== HGB(SVD) visuals + export (safe to rerun) ====
import os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, f1_score

# Expect: best (pipeline), Xte, y_eval exist. If not, adjust to your var names.
# Recompute proba to be safe.
proba  = best.predict_proba(Xte)[:, 1]
pred50 = (proba >= 0.50).astype(int)

# Threshold sweep (macro-F1)
ths = np.linspace(0.01, 0.99, 99)
f1s = [f1_score(y_eval, (proba >= t).astype(int), average="macro") for t in ths]
t_star = float(ths[int(np.argmax(f1s))])
predst = (proba >= t_star).astype(int)

# Metrics
cm50  = confusion_matrix(y_eval, pred50)
cmst  = confusion_matrix(y_eval, predst)
fpr, tpr, _ = roc_curve(y_eval, proba)
roc_auc     = auc(fpr, tpr)
prec, rec, _= precision_recall_curve(y_eval, proba)

acc50 = float((pred50 == y_eval).mean())
f1m50 = float(f1_score(y_eval, pred50, average="macro"))
accst = float((predst == y_eval).mean())
f1mst = float(f1_score(y_eval, predst, average="macro"))

print(f"HGB(SVD) @0.50  acc={acc50:.3f}  f1m={f1m50:.3f}  AUC={roc_auc:.3f}")
print(f"HGB(SVD) @{t_star:.3f}  acc={accst:.3f}  f1m={f1mst:.3f}  AUC={roc_auc:.3f}")

# --- plotting helpers ---
def plot_confusion(cm, title):
    fig, ax = plt.subplots(figsize=(4.8, 4.2))
    im = ax.imshow(cm, interpolation="nearest")
    ax.set_title(title)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, str(cm[i,j]), ha="center", va="center")
    plt.tight_layout()
    plt.show()

# 1) Confusion @0.50 and @t*
plot_confusion(cm50, "HGB(SVD) — Confusion @0.50")
plot_confusion(cmst, f"HGB(SVD) — Confusion @{t_star:.3f}")

# 2) ROC curve
fig, ax = plt.subplots(figsize=(5.2, 4.4))
ax.plot(fpr, tpr, lw=2, label=f"AUC={roc_auc:.3f}")
ax.plot([0,1], [0,1], lw=1, linestyle="--")
ax.set_title("HGB(SVD) — ROC curve")
ax.set_xlabel("FPR"); ax.set_ylabel("TPR")
ax.legend(loc="lower right")
plt.tight_layout(); plt.show()

# 3) Precision–Recall curve
fig, ax = plt.subplots(figsize=(5.2, 4.4))
ax.plot(rec, prec, lw=2)
ax.set_title("HGB(SVD) — Precision–Recall")
ax.set_xlabel("Recall"); ax.set_ylabel("Precision")
plt.tight_layout(); plt.show()

# 4) Threshold sweep (macro-F1)
fig, ax = plt.subplots(figsize=(5.2, 4.4))
ax.plot(ths, f1s, lw=2)
ax.axvline(t_star, linestyle="--")
ax.set_title("HGB(SVD) — Macro-F1 vs Threshold")
ax.set_xlabel("Decision threshold"); ax.set_ylabel("Macro-F1")
plt.tight_layout(); plt.show()

# 5) Export a tidy table row with params + metrics
outdir = Path("reports"); outdir.mkdir(exist_ok=True)
row = {
    "model": "HGB(SVD)",
    "threshold_50": 0.50,
    "acc_50": acc50, "macro_f1_50": f1m50, "roc_auc": roc_auc,
    "threshold_star": t_star,
    "acc_star": accst, "macro_f1_star": f1mst,
}
# add best params if available
try:
    for k, v in rs_hgb.best_params_.items():
        row[k] = v
except Exception:
    pass

pd.DataFrame([row]).to_csv(outdir / "hgb_svd_table_row.csv", index=False)
print(f"Saved table row -> {outdir/'hgb_svd_table_row.csv'}")


In [ ]:
# ==== CHECKPOINT NOW: persist models, metadata, splits, and results ====
from pathlib import Path
import json, joblib, numpy as np, pandas as pd
import sklearn, sys, platform, datetime as dt

ART = Path("artifacts"); ART.mkdir(exist_ok=True)
print(f"Saving to: {ART.resolve()}")

# 1) Core data (lightweight + reproducible)
if 'y' in globals():
    pd.Series(y).to_csv(ART/"y.csv", index=True)
if 'df' in globals():
    # Save only columns we actually used in hybrid features if available
    cols_to_save = None
    if 'hyb_cols' in globals():
        cols_to_save = list(dict.fromkeys(hyb_cols))
    elif 'use_cols' in globals():
        # fall back to base features; clusters/cosdist might be missing on reload
        cols_to_save = list(dict.fromkeys(use_cols))
    if cols_to_save is not None:
        df[cols_to_save].to_parquet(ART/"df_features.parquet")
        json.dump(cols_to_save, open(ART/"hyb_cols.json","w"))
        print(f"Saved df_features.parquet with {len(cols_to_save)} columns.")
    else:
        # as a last resort (larger file), save the whole frame
        df.to_parquet(ART/"df_full.parquet")
        print("Saved df_full.parquet (fallback).")

# 2) Train/hold-out split indices (so you can reproduce evaluation fast)
def _save_index(name, X):
    idx = getattr(X, "index", None)
    if idx is not None:
        pd.Index(idx).to_series(name="index").to_csv(ART/f"{name}_index.csv", index=False)

for nm in ("Xtr","Xte","ytr","y_eval"):
    if nm in globals():
        _save_index(nm, globals()[nm])

# 3) Pipelines / models
# Hybrid Logistic (elastic net) – try to save the tuned one first
if 'rs' in globals() and getattr(rs, 'best_estimator_', None) is not None:
    joblib.dump(rs.best_estimator_, ART/"hybrid_logreg_enet_best.joblib")
elif 'pipe_hybrid' in globals():
    joblib.dump(pipe_hybrid, ART/"hybrid_logreg_enet_pipe.joblib")

# Tuned HGB(SVD)
if 'rs_hgb' in globals() and getattr(rs_hgb, 'best_estimator_', None) is not None:
    joblib.dump(rs_hgb.best_estimator_, ART/"hgb_svd_best.joblib")
elif 'best' in globals():
    joblib.dump(best, ART/"hgb_svd_best.joblib")

# 4) CV results and single-row summaries if you’ve created them
if 'rs' in globals() and hasattr(rs, "cv_results_"):
    pd.DataFrame(rs.cv_results_).to_csv(ART/"cv_hybrid_logreg_enet.csv", index=False)
if 'rs_hgb' in globals() and hasattr(rs_hgb, "cv_results_"):
    pd.DataFrame(rs_hgb.cv_results_).to_csv(ART/"cv_hgb_svd.csv", index=False)

for fname in ("hybrid_logreg_enet_table_row.csv", "hgb_svd_table_row.csv",
              "hybrid_logreg_enet_summary.json"):
    p = Path(fname)
    if p.exists():
        p.replace(ART/p.name)

# 5) Minimal manifest (helps future-you)
manifest = {
    "timestamp_utc": dt.datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "python": sys.version.split()[0],
    "sklearn": sklearn.__version__,
    "platform": platform.platform(),
    "artifacts": sorted([p.name for p in ART.iterdir() if p.is_file()]),
}
json.dump(manifest, open(ART/"manifest.json","w"), indent=2)
print("Wrote manifest.json")
print("Done ✅ — you can safely stop the kernel after this cell finishes.")


In [ ]:
# ==== RESUME LATER: reload models, data, and splits (pandas>=2 friendly) ====
from pathlib import Path
import joblib, json
import numpy as np
import pandas as pd

ART = Path("artifacts")
assert ART.exists(), "No artifacts/ folder found. Did you run the checkpoint cell?"

# Load y as a Series (works whether the CSV had a header or not)
ydf = pd.read_csv(ART / "y.csv", index_col=0)
y = ydf.iloc[:, 0]
if y.name is None or y.name == "":
    y.name = "target"

# Load features DF (prefer slim feature parquet, else full)
if (ART / "df_features.parquet").exists():
    df = pd.read_parquet(ART / "df_features.parquet")
    hyb_cols = json.load(open(ART / "hyb_cols.json"))
elif (ART / "df_full.parquet").exists():
    df = pd.read_parquet(ART / "df_full.parquet")
    hyb_cols = list(df.columns)
else:
    raise FileNotFoundError("No data parquet found in artifacts/")

# Rebuild Xh exactly as saved
Xh = df[hyb_cols].copy()

# Load split indices if present
def _maybe_idx(name):
    p = ART / f"{name}_index.csv"
    if p.exists():
        return pd.read_csv(p)["index"]
    return None

idx_Xte = _maybe_idx("Xte")
idx_yte = _maybe_idx("y_eval")

if idx_Xte is not None and idx_yte is not None:
    Xte = Xh.loc[idx_Xte]
    y_eval = y.loc[idx_yte]
    print(f"Restored hold-out: Xte={Xte.shape}, y_eval={y_eval.shape}")
else:
    print("Split indices not found — you can re-run train_test_split if needed.")

# Load models
best_hgb = None
best_log = None
if (ART / "hgb_svd_best.joblib").exists():
    best_hgb = joblib.load(ART / "hgb_svd_best.joblib")
    print("Loaded: hgb_svd_best.joblib")
if (ART / "hybrid_logreg_enet_best.joblib").exists():
    best_log = joblib.load(ART / "hybrid_logreg_enet_best.joblib")
    print("Loaded: hybrid_logreg_enet_best.joblib")
elif (ART / "hybrid_logreg_enet_pipe.joblib").exists():
    best_log = joblib.load(ART / "hybrid_logreg_enet_pipe.joblib")
    print("Loaded: hybrid_logreg_enet_pipe.joblib")

print("Ready ✅ — models and data are reloaded. You can call predict_proba on Xte immediately.")


In [ ]:
import numpy as np, pandas as pd

def make_sklearn_friendly(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Normalize null sentinel first
    df = df.replace({pd.NA: np.nan})

    # 2) Extension numerics -> plain float64 so np.nan works everywhere
    for col in df.columns:
        dt = df[col].dtype
        if str(dt) in ("Int64", "UInt64", "Float64"):
            df[col] = df[col].astype("float64")

    # 3) Extension booleans / pandas string -> object (avoid pd.NA semantics)
    bool_ext = df.select_dtypes(include=["boolean"]).columns
    if len(bool_ext):
        df[bool_ext] = df[bool_ext].astype("object")

    str_ext = df.select_dtypes(include=["string"]).columns
    if len(str_ext):
        df[str_ext] = df[str_ext].astype("object")

    # 4) Force object/category cols to be writable NumPy arrays, then fix nulls
    obj_cols = df.select_dtypes(include=["object", "category"]).columns
    for c in obj_cols:
        arr = df[c].to_numpy(dtype=object, copy=True)  # ensures WRITABLE
        m = pd.isna(arr)
        if m.any():
            arr[m] = np.nan
        df[c] = arr  # assign back as a fresh, writeable column

    return df

# Apply to the frames you use for evaluation/plots
Xte = make_sklearn_friendly(Xte)
Xh  = make_sklearn_friendly(Xh)

# quick sanity checks
assert not any(dt.name == "boolean" for dt in Xte.dtypes)
assert not any(s in str(dt) for dt in Xte.dtypes for s in ("Int64","UInt64","Float64"))


In [ ]:
# --- Permutation importance + plots (robust) -------------------------------
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

# ---------- 0) Helper: sanitize a DataFrame for sklearn ----------
def make_sklearn_friendly(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # unify nulls
    df = df.replace({pd.NA: np.nan})
    # extension numerics -> float64 (so np.nan is valid)
    for c in df.columns:
        dt = df[c].dtype
        if str(dt) in ("Int64", "UInt64", "Float64"):
            df[c] = df[c].astype("float64")
    # extension booleans / pandas strings -> object
    bool_ext = df.select_dtypes(include=["boolean"]).columns
    if len(bool_ext):
        df[bool_ext] = df[bool_ext].astype("object")
    str_ext = df.select_dtypes(include=["string"]).columns
    if len(str_ext):
        df[str_ext] = df[str_ext].astype("object")
    # ensure object/category columns are writable & use real np.nan
    obj_cols = df.select_dtypes(include=["object", "category"]).columns
    for c in obj_cols:
        arr = df[c].to_numpy(dtype=object, copy=True)   # writeable
        m = pd.isna(arr)
        if m.any():
            arr[m] = np.nan
        df[c] = arr
    return df

# Apply to your frames (no-op if already clean)
Xte = make_sklearn_friendly(Xte)
Xh  = make_sklearn_friendly(Xh)

# Small safety checks
assert isinstance(Xte, pd.DataFrame), "Xte must be a DataFrame"
assert isinstance(Xh, pd.DataFrame), "Xh must be a DataFrame"
assert "y_eval" in globals(), "y_eval (hold-out y) not found"

print(f"Sanitized: Xte shape={Xte.shape}, y_eval={y_eval.shape}")

# ---------- 1) Helper to compute PI and make a tidy dataframe ----------
def perm_importance_df(estimator, X: pd.DataFrame, y, *,
                       scoring="f1_macro", n_repeats=8, n_jobs=4, random_state=42):
    pi = permutation_importance(
        estimator, X, y,
        scoring=scoring, n_repeats=n_repeats,
        n_jobs=n_jobs, random_state=random_state
    )
    out = (pd.DataFrame({
        "feature": X.columns,
        "importance_mean": pi.importances_mean,
        "importance_std":  pi.importances_std,
    })
    .sort_values("importance_mean", ascending=False)
    .reset_index(drop=True))
    return out

# ---------- 2) Plot helper ----------
def plot_topk_bars(df_imp: pd.DataFrame, title: str, k: int = 20):
    k = min(k, len(df_imp))
    top = df_imp.head(k).iloc[::-1]  # reverse so the biggest is at the top of the barh
    fig, ax = plt.subplots(figsize=(8, 0.35*k + 1.5))
    ax.barh(top["feature"], top["importance_mean"])
    ax.set_xlabel("Permutation importance (mean Δ score)")
    ax.set_title(title)
    # error bars
    for i, (m, s) in enumerate(zip(top["importance_mean"], top["importance_std"])):
        ax.errorbar(m, i, xerr=s, fmt='none', capsize=2)
    plt.tight_layout()
    plt.show()

# ---------- 3) Run for each available champion model ----------
models = []
if "best_hgb" in globals():
    models.append(("HGB(SVD) champion", best_hgb))
if "best_logit" in globals():
    models.append(("Hybrid LogReg champion", best_logit))

if not models:
    raise RuntimeError("No champion models found. Expected variables: best_hgb and/or best_logit.")

for name, est in models:
    print(f"\n▶ Permutation importance for {name}")
    imp_df = perm_importance_df(est, Xte, y_eval, scoring="f1_macro", n_repeats=8, n_jobs=4)
    display(imp_df.head(25))
    plot_topk_bars(imp_df, f"{name} — Top features (permutation)", k=25)


In [ ]:
#GOAL
#Estimate mean ± std importance per feature across several seeds, plus how often each feature appears in the top-K (stability score).

In [ ]:
# ==== 0) FIND & LOAD THE ESTIMATOR ARTIFACT ====
import os, sys, glob
from joblib import load

def find_model_candidates():
    # likely names/locations
    explicit = [
        "hgb_svd_best.joblib",
        "models/hgb_svd_best.joblib",
        "artifacts/hgb_svd_best.joblib",
        "outputs/hgb_svd_best.joblib",
        "models/hgb_svd.joblib",
    ]
    cands = [p for p in explicit if os.path.exists(p)]

    # generic search: any .joblib that looks like HGB or SVD or champion
    patterns = [
        "**/*hgb*joblib", "**/*svd*joblib", "**/*gradient*joblib",
        "**/*champion*joblib", "**/*.joblib"
    ]
    roots = [os.getcwd()]
    # add mlflow default folder if present
    if os.path.exists("mlruns"):
        roots.append("mlruns")
    # search
    for root in roots:
        for pat in patterns:
            cands.extend(glob.glob(os.path.join(root, pat), recursive=True))
    # de-dup while preserving order
    seen, uniq = set(), []
    for p in cands:
        if p not in seen:
            seen.add(p); uniq.append(p)
    return uniq

model_path = None
try:
    model_path = "hgb_svd_best.joblib"
    estimator = load(model_path)
except Exception:
    # search for alternatives
    candidates = find_model_candidates()
    if not candidates:
        raise FileNotFoundError(
            "Could not find any *.joblib model artifact. "
            "Check your save path or re-save the trained model with:\n"
            "from joblib import dump; dump(hgb_svd_best, 'hgb_svd_best.joblib')"
        )
    # prefer candidates with 'hgb' and 'svd' in name
    def score(path):
        name = os.path.basename(path).lower()
        s = 0
        for kw in ("hgb", "hist", "gradient", "boost"):
            if kw in name: s += 2
        for kw in ("svd", "pipe", "pipeline"):
            if kw in name: s += 1
        return s
    candidates.sort(key=score, reverse=True)
    model_path = candidates[0]
    estimator = load(model_path)

print(f"Loaded estimator from: {model_path}")
# optional alias expected by earlier code
hgb_svd_best = estimator

# ==== 1) STABILITY: permutation importance across seeds ====
from sklearn.inspection import permutation_importance
import numpy as np, pandas as pd

SEEDS = [0, 1, 2, 3, 4, 7, 13, 21, 42, 2025]
N_REPEATS = 10
TOP_K = 25

def perm_imp_once(est, X, y, seed, n_repeats=10, scoring=None):
    rng = np.random.RandomState(seed)
    pi = permutation_importance(
        est, X, y,
        n_repeats=n_repeats,
        random_state=rng,
        scoring=scoring,
        n_jobs=-1
    )
    idx = getattr(X, "columns", np.arange(X.shape[1]))
    s_mean = pd.Series(pi.importances_mean, index=idx, name=f"seed_{seed}")
    s_std  = pd.Series(pi.importances_std,  index=idx, name=f"seed_{seed}_std")
    return s_mean, s_std

means, stds = [], []
for seed in SEEDS:
    s_mean, s_std = perm_imp_once(estimator, Xte, y_eval, seed, n_repeats=N_REPEATS, scoring=None)
    means.append(s_mean); stds.append(s_std)

M = pd.concat(means, axis=1)   # feature x seeds (means)
S = pd.concat(stds,  axis=1)   # feature x seeds (within-seed stds)

agg = pd.DataFrame({
    "importance_mean": M.mean(axis=1),
    "importance_std":  M.std(axis=1, ddof=1),     # across-seed std
    "within_seed_std_mean": S.mean(axis=1)
}).sort_values("importance_mean", ascending=False)

# Stability metric: how often a feature is in Top-K across seeds
counts = pd.Series(0, index=M.index, dtype=float)
for col in M.columns:
    topk_idx = M[col].sort_values(ascending=False).index[:TOP_K]
    counts.loc[topk_idx] += 1
agg["topK_frequency"] = counts / len(SEEDS)

# Coefficient of variation across seeds (lower = more stable)
agg["cv_across_seeds"] = (agg["importance_std"] / agg["importance_mean"].replace(0, np.nan)).fillna(np.inf)

# Peek the top-K stable features
stable_top = agg.head(TOP_K)
print(stable_top.head(10))

# Save for your report
agg.to_csv("perm_importance_stability.csv", index=True)
print("Saved: perm_importance_stability.csv")


In [ ]:
#Grouped permutation importance

In [ ]:
# ===== Grouped Permutation Importance for HGB(SVD) champion =====
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# 1) Helper: robust AUC scorer
def auc_score(est, X, y):
    try:
        p = est.predict_proba(X)[:, 1]
    except Exception:
        s = est.decision_function(X)
        # Min-max to [0,1] as a fallback
        s_min, s_max = s.min(), s.max()
        p = (s - s_min) / (s_max - s_min + 1e-12)
    return roc_auc_score(y, p)

# 2) Build feature groups by prefix
def make_groups(cols):
    groups = {
        "Cuisine (cat__)": [c for c in cols if c.startswith("cat__")],
        "Amenities (attr_)": [c for c in cols if c.startswith("attr_")],
        "Review meta (rl_)": [c for c in cols if c.startswith("rl_")],
        "Spherical distances (sphcosdist_)": [c for c in cols if c.startswith("sphcosdist_")],
        "Clusters": [c for c in cols if c.startswith("cluster_")],
        "City dummies": [c for c in cols if c.startswith("city_")],
        "Geography (lat/lon)": [c for c in cols if c in {"latitude", "longitude"}],
        "Counts & recency": [c for c in cols if c in {"review_count_log1p", "review_count", "days_open"}],
    }
    # Optional catch-all for anything unmatched
    used = set(sum(groups.values(), []))
    remainder = [c for c in cols if c not in used]
    if remainder:
        groups["Other"] = remainder
    # Drop empty groups
    return {g: v for g, v in groups.items() if len(v) > 0}

groups = make_groups(Xte.columns)

# 3) Baseline AUC
BASELINE = auc_score(estimator, Xte, y_eval)
print(f"Baseline AUC: {BASELINE:.3f}")

# 4) Grouped permutation importance
def grouped_perm_importance(est, X, y, groups, n_repeats=30, random_state=2025):
    rng = np.random.RandomState(random_state)
    results = []
    for gname, cols in groups.items():
        drops = []
        for _ in range(n_repeats):
            Xp = X.copy()
            # Shuffle the group with a shared permutation index (preserve within-row structure)
            idx = rng.permutation(len(Xp))
            for c in cols:
                Xp[c] = Xp[c].values[idx]
            score = auc_score(est, Xp, y)
            drops.append(BASELINE - score)
        results.append((gname, float(np.mean(drops)), float(np.std(drops, ddof=1)), len(cols)))
    df = pd.DataFrame(results, columns=["group", "importance_mean", "importance_std", "n_cols"])
    return df.sort_values("importance_mean", ascending=False).reset_index(drop=True)

group_imp = grouped_perm_importance(estimator, Xte, y_eval, groups, n_repeats=30, random_state=2025)
print(group_imp)
group_imp.to_csv("grouped_permutation_importance.csv", index=False)
print("Saved: grouped_permutation_importance.csv")

# 5) Quick plot
plt.figure(figsize=(9, 6))
gi = group_imp.iloc[::-1]  # reverse for barh
plt.barh(gi["group"], gi["importance_mean"], xerr=gi["importance_std"])
plt.title("Grouped permutation importance (AUC drop)")
plt.xlabel("AUC drop when group is permuted")
plt.tight_layout()
plt.show()


In [ ]:
#Retrain ablation

In [ ]:
#“what if we removed this family entirely?”

In [ ]:
# ===== Schema-preserving group ablation (dtype-robust) =====
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import roc_auc_score

def auc_score(est, X, y):
    try:
        p = est.predict_proba(X)[:, 1]
    except Exception:
        s = est.decision_function(X)
        p = (s - s.min()) / (s.max() - s.min() + 1e-12)
    return roc_auc_score(y, p)

def neutralize_columns_dtype_aware(Xtr, Xte, cols, strategy="smart"):
    """
    Keep columns present but remove signal.
    strategy="smart": numeric -> mean constant; non-numeric -> 0
    strategy="zero": set everything to 0.0
    """
    Xtr_mut, Xte_mut = Xtr.copy(), Xte.copy()
    used = []

    for c in cols:
        if c not in Xtr_mut.columns:
            continue
        used.append(c)
        if strategy == "zero":
            const_tr = const_te = 0.0
        else:
            # smart: per-column decision
            ser = Xtr_mut[c]
            if np.issubdtype(ser.dtype, np.number):
                m = ser.mean()
                const_tr = const_te = float(0.0 if pd.isna(m) else m)
            else:
                # Try to coerce to numeric; if still non-numeric, use 0
                coerced = pd.to_numeric(ser, errors='coerce')
                m = coerced.mean()
                const = float(0.0 if pd.isna(m) else m)
                const_tr = const_te = const
        Xtr_mut[c] = const_tr
        if c in Xte_mut.columns:
            Xte_mut[c] = const_te
    return Xtr_mut, Xte_mut, used

def ablate_groups_keep_schema(est, Xtr, ytr, Xte, yte, groups, groups_to_neutralize, strategy="smart"):
    # Collect columns from the target groups that actually exist in Xtr
    cols = []
    for gname in groups_to_neutralize:
        cols.extend(groups[gname])
    cols = [c for c in cols if c in Xtr.columns]

    # Neutralize per column with dtype-aware constants
    Xtr_mut, Xte_mut, used = neutralize_columns_dtype_aware(Xtr, Xte, cols, strategy=strategy)

    est2 = clone(est)
    est2.fit(Xtr_mut, ytr)
    auc2 = auc_score(est2, Xte_mut, yte)
    return auc2, used, est2

# --- Run ablations again ---
BASELINE_AUC = auc_score(estimator, Xte, y_eval)
print(f"Baseline AUC: {BASELINE_AUC:.3f}")

to_test = ["Cuisine (cat__)", "Amenities (attr_)", "Review meta (rl_)", "Counts & recency"]
rows = []
for gname in to_test:
    auc2, used_cols, _ = ablate_groups_keep_schema(
        estimator, Xtr, y_train, Xte, y_eval, groups, [gname], strategy="smart"  # or "zero"
    )
    rows.append({
        "group": gname,
        "auc_after_neutralize": auc2,
        "auc_delta": BASELINE_AUC - auc2,
        "n_cols_neutralized": len(used_cols),
        "strategy": "smart"
    })

ablation_df = pd.DataFrame(rows).sort_values("auc_delta", ascending=False).reset_index(drop=True)
print(ablation_df)
ablation_df.to_csv("group_ablation_refit_schema_preserving.csv", index=False)
print("Saved: group_ablation_refit_schema_preserving.csv")


In [ ]:
import numpy as np, pandas as pd
from sklearn.base import clone
from sklearn.metrics import roc_auc_score

def auc_score(est, X, y):
    try:
        p = est.predict_proba(X)[:,1]
    except Exception:
        s = est.decision_function(X)
        p = (s - s.min()) / (s.max() - s.min() + 1e-12)
    return roc_auc_score(y, p)

def neutralize_columns_dtype_aware(Xtr, Xte, cols, strategy="zero"):
    Xtr_mut, Xte_mut = Xtr.copy(), Xte.copy()
    used = []
    for c in cols:
        if c not in Xtr_mut.columns: 
            continue
        used.append(c)
        if strategy == "zero":
            const_tr = const_te = 0.0
        else:  # fallback
            ser = Xtr_mut[c]
            if np.issubdtype(ser.dtype, np.number):
                m = ser.mean()
                const_tr = const_te = float(0.0 if pd.isna(m) else m)
            else:
                coerced = pd.to_numeric(ser, errors="coerce")
                m = coerced.mean()
                const_tr = const_te = float(0.0 if pd.isna(m) else m)
        Xtr_mut[c] = const_tr
        if c in Xte_mut.columns:
            Xte_mut[c] = const_te
    return Xtr_mut, Xte_mut, used

def ablate_groups_keep_schema(est, Xtr, ytr, Xte, yte, groups, groups_to_neutralize, strategy_map):
    # Collect columns and apply group-specific strategies
    Xtr_mut, Xte_mut = Xtr.copy(), Xte.copy()
    used_all = []
    for gname in groups_to_neutralize:
        cols = [c for c in groups[gname] if c in Xtr_mut.columns]
        strat = strategy_map.get(gname, "zero")
        Xtr_mut, Xte_mut, used = neutralize_columns_dtype_aware(Xtr_mut, Xte_mut, cols, strategy=strat)
        used_all.extend(used)
    est2 = clone(est)
    est2.fit(Xtr_mut, ytr)
    auc2 = auc_score(est2, Xte_mut, yte)
    return auc2, used_all, est2

# Baseline
BASELINE_AUC = auc_score(estimator, Xte, y_eval)
print(f"Baseline AUC: {BASELINE_AUC:.3f}")

# Strategy per group: zero for one-hots; mean for continuous-ish
strategy_map = {
    "Cuisine (cat__)": "zero",
    "Amenities (attr_)": "zero",
    "City dummies": "zero",
    "Clusters": "mean",
    "Counts & recency": "mean",
    "Review meta (rl_)": "mean",
    "Spherical distances (sphcosdist_)": "mean",
    "Geography (lat/lon)": "mean",
    "Other": "mean",
}

# --- Single-group ablation (schema-preserving, sharper for dummies) ---
single_groups = ["Review meta (rl_)","Amenities (attr_)","Cuisine (cat__)","Counts & recency"]
rows = []
for gname in single_groups:
    auc2, used_cols, _ = ablate_groups_keep_schema(
        estimator, Xtr, y_train, Xte, y_eval, groups, [gname], strategy_map
    )
    rows.append({
        "group": gname,
        "auc_after_neutralize": auc2,
        "auc_delta": BASELINE_AUC - auc2,
        "n_cols_neutralized": len(used_cols),
        "signal_per_feature": (BASELINE_AUC - auc2) / max(len(used_cols),1)
    })
single_df = pd.DataFrame(rows).sort_values("auc_delta", ascending=False).reset_index(drop=True)
print("\nSingle-group (zero/mean) neutralization:")
print(single_df)

# --- Pairwise ablations to check overlap/additivity ---
import itertools
pairs = list(itertools.combinations(single_groups[:3], 2))  # test top 3 pairs
pair_rows = []
for gA, gB in pairs:
    auc2, used_cols, _ = ablate_groups_keep_schema(
        estimator, Xtr, y_train, Xte, y_eval, groups, [gA, gB], strategy_map
    )
    delta = BASELINE_AUC - auc2
    pair_rows.append({
        "groups": f"{gA} + {gB}",
        "auc_after_neutralize": auc2,
        "auc_delta": delta,
        "n_cols_neutralized": len(used_cols),
        "signal_per_feature": delta / max(len(used_cols),1)
    })
pair_df = pd.DataFrame(pair_rows).sort_values("auc_delta", ascending=False).reset_index(drop=True)
print("\nPairwise neutralization:")
print(pair_df)

# Save
single_df.to_csv("group_ablation_single_zero_mean.csv", index=False)
pair_df.to_csv("group_ablation_pairs_zero_mean.csv", index=False)
print("\nSaved: group_ablation_single_zero_mean.csv, group_ablation_pairs_zero_mean.csv")


In [ ]:
# ===== STEP: Interpretability & insights → Effects (PDP/ICE) =====
# Produces PDP + ICE for top features and a 2D PDP for interaction; saves PNGs.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

# Assumes these already exist in your session:
# - estimator : your trained HGB(SVD) champion pipeline
# - Xtr, Xte  : train/eval DataFrames (same columns)
# - y_eval    : eval labels (not used here but kept available)

# Choose data for plotting grids (training is more representative for PDP grids)
X_plot = Xtr if 'Xtr' in globals() else Xte

# Select actionable features (edit if needed; code will skip missing ones)
single_features = [
    "rl_word_mean",
    "review_count_log1p",
    "attr_RestaurantsAttire",
    "attr_HasTV",
    "attr_WiFi",
    "cat__Fast Food"
]

# One 2D pair to show interaction/synergy
pair_features = [("rl_word_mean", "attr_RestaurantsAttire")]

def present(feat, X):
    return feat in X.columns

outdir = "plots_pdp_ice"
os.makedirs(outdir, exist_ok=True)

# Single-feature PDP + ICE
for feat in single_features:
    if not present(feat, X_plot):
        print(f"[skip] '{feat}' not found in columns.")
        continue
    plt.figure(figsize=(6.5, 4.5))
    PartialDependenceDisplay.from_estimator(
        estimator,
        X_plot,
        [feat],
        kind="both",           # plots both PDP (avg) and ICE (individual)
        grid_resolution=20
    )
    plt.title(f"PDP/ICE — {feat}")
    plt.tight_layout()
    fname = os.path.join(outdir, f"pdp_ice__{feat.replace(' ', '_').replace('/', '_')}.png")
    plt.savefig(fname, dpi=160)
    plt.show()
    print(f"[saved] {fname}")

# Two-way PDP (interaction surface)
for f1, f2 in pair_features:
    if not (present(f1, X_plot) and present(f2, X_plot)):
        print(f"[skip pair] '{f1}', '{f2}' — one or both missing.")
        continue
    plt.figure(figsize=(6.5, 5.5))
    PartialDependenceDisplay.from_estimator(
        estimator,
        X_plot,
        [(f1, f2)],
        kind="average",
        grid_resolution=20
    )
    plt.suptitle(f"2D PDP — {f1} × {f2}")
    plt.tight_layout()
    fname = os.path.join(outdir, f"pdp2d__{f1}__x__{f2}.png".replace(' ', '_').replace('/', '_'))
    plt.savefig(fname, dpi=160)
    plt.show()
    print(f"[saved] {fname}")

print("\nDone. All figures saved in:", outdir)


In [ ]:
# ===== STEP 1c: PDP/ICE (dtype-safe) =====
# Fixes TypeError by coercing selected columns to numeric in a copy used only for plotting.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

# Choose plotting frame (train preferred for PDP grids)
X_plot = Xtr if 'Xtr' in globals() else Xte

# Reuse previously selected features if available; otherwise auto-select from stability CSV
if 'single_features' in globals() and isinstance(single_features, (list, tuple)) and len(single_features) > 0:
    feats_single = list(single_features)
else:
    agg = pd.read_csv("perm_importance_stability.csv", index_col=0)
    cols_present = set(X_plot.columns)
    agg_present = agg[agg.index.isin(cols_present)].sort_values("importance_mean", ascending=False)

    def is_binary(col):
        try:
            vals = pd.unique(X_plot[col].dropna())
            return len(vals) <= 2
        except Exception:
            return False

    def is_numeric(col):
        return np.issubdtype(X_plot[col].dtype, np.number)

    top_numeric = [c for c in agg_present.index if is_numeric(c) and not is_binary(c)]
    top_binary  = [c for c in agg_present.index if is_binary(c)]
    feats_single = top_numeric[:4] + top_binary[:4]

# Pick a 2D pair (rl_word_mean with best amenity) if present
amenity_candidates = [c for c in feats_single if c.startswith("attr_")] or \
                     [c for c in X_plot.columns if c.startswith("attr_")]
pair_features = []
if "rl_word_mean" in X_plot.columns and len(amenity_candidates) > 0:
    pair_features = [("rl_word_mean", amenity_candidates[0])]

print("Will plot single-feature PDP/ICE for:", feats_single)
print("2D PDP pair:", pair_features if pair_features else "None")

# --- Make a dtype-safe copy just for plotting ---
X_safe = X_plot.copy()
cols_to_sanitize = set(feats_single + [f for tup in pair_features for f in (tup if isinstance(tup, tuple) else [tup])])
cols_to_sanitize = [c for c in cols_to_sanitize if c in X_safe.columns]

for c in cols_to_sanitize:
    # Try numeric conversion; fall back to 0 for non-convertible entries
    newc = pd.to_numeric(X_safe[c], errors="coerce")
    if newc.isna().any():
        # For binary-like columns use 0; otherwise median
        unique_non_na = pd.unique(newc.dropna())
        if len(unique_non_na) <= 2:
            fill_val = 0.0
        else:
            fill_val = float(np.nanmedian(newc))
        newc = newc.fillna(fill_val)
    X_safe[c] = newc.astype(float)

outdir = "plots_pdp_ice_fixed"
os.makedirs(outdir, exist_ok=True)

# --- Plot single-feature PDP + ICE ---
for feat in feats_single:
    if feat not in X_safe.columns:
        print(f"[skip] '{feat}' not found after sanitation.")
        continue
    plt.figure(figsize=(6.5, 4.5))
    PartialDependenceDisplay.from_estimator(
        estimator,
        X_safe,
        [feat],
        kind="both",
        grid_resolution=20
    )
    plt.title(f"PDP/ICE — {feat}")
    plt.tight_layout()
    fname = os.path.join(outdir, f"pdp_ice__{feat.replace(' ', '_').replace('/', '_')}.png")
    plt.savefig(fname, dpi=160)
    plt.show()
    print(f"[saved] {fname}")

# --- 2D PDP (interaction surface) ---
for (f1, f2) in pair_features:
    if f1 in X_safe.columns and f2 in X_safe.columns:
        plt.figure(figsize=(6.5, 5.5))
        PartialDependenceDisplay.from_estimator(
            estimator,
            X_safe,
            [(f1, f2)],
            kind="average",
            grid_resolution=20
        )
        plt.suptitle(f"2D PDP — {f1} × {f2}")
        plt.tight_layout()
        fname = os.path.join(outdir, f"pdp2d__{f1}__x__{f2}.png".replace(' ', '_').replace('/', '_'))
        plt.savefig(fname, dpi=160)
        plt.show()
        print(f"[saved] {fname}")
    else:
        print(f"[skip pair] Missing sanitized columns for {f1} or {f2}")

print("\nDone. All figures saved in:", outdir)


In [ ]:
# ===== STEP 2: Calibration & thresholding =====
# Outputs:
#   - plots_calibration/calibration_curve.png
#   - plots_calibration/prob_hist.png
#   - plots_calibration/threshold_vs_f1.png
#   - threshold_sweep_eval.csv (metrics vs threshold)
#   - printed best threshold by macro-F1 and metrics at 0.50, 0.53, best

import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, brier_score_loss, roc_auc_score
)
from sklearn.calibration import calibration_curve

# --- 0) Get predicted probabilities on eval (robust to estimators without predict_proba)
def get_prob(est, X):
    try:
        return est.predict_proba(X)[:, 1]
    except Exception:
        s = est.decision_function(X)
        s_min, s_max = s.min(), s.max()
        return (s - s_min) / (s_max - s_min + 1e-12)

proba = get_prob(estimator, Xte)
y_true = np.asarray(y_eval).astype(int)

# --- 1) Calibration curve + Brier score
frac_pos, mean_pred = calibration_curve(y_true, proba, n_bins=15, strategy="quantile")
brier = brier_score_loss(y_true, proba)
auc = roc_auc_score(y_true, proba)

outdir = "plots_calibration"
os.makedirs(outdir, exist_ok=True)

plt.figure(figsize=(5.5,5))
plt.plot([0,1],[0,1],'--',label="perfect")
plt.plot(mean_pred, frac_pos, marker='o', label="model")
plt.xlabel("Mean predicted probability")
plt.ylabel("Observed frequency")
plt.title(f"Calibration curve (eval) — Brier={brier:.3f}, AUC={auc:.3f}")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outdir,"calibration_curve.png"), dpi=160)
plt.show()

# Probability histogram
plt.figure(figsize=(6,4))
plt.hist(proba, bins=30, edgecolor='black')
plt.xlabel("Predicted probability (positive)")
plt.ylabel("Count")
plt.title("Predicted probability distribution (eval)")
plt.tight_layout()
plt.savefig(os.path.join(outdir,"prob_hist.png"), dpi=160)
plt.show()

# --- 2) Threshold sweep for macro-F1
def metrics_at(y_true, proba, thr):
    y_pred = (proba >= thr).astype(int)
    acc  = accuracy_score(y_true, y_pred)
    pre  = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)
    f1m  = f1_score(y_true, y_pred, average='macro', zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return acc, pre, rec, f1, f1m, tn, fp, fn, tp

thresholds = np.linspace(0.01, 0.99, 99)
rows = []
for t in thresholds:
    acc, pre, rec, f1, f1m, tn, fp, fn, tp = metrics_at(y_true, proba, t)
    rows.append({"threshold": t, "accuracy": acc, "precision": pre, "recall": rec,
                 "f1": f1, "macro_f1": f1m, "tn": tn, "fp": fp, "fn": fn, "tp": tp})
sweep = pd.DataFrame(rows)

# Best by macro-F1
best_idx = sweep["macro_f1"].idxmax()
best_row = sweep.loc[best_idx]
best_thr = float(best_row["threshold"])

# Reference thresholds
def row_for(t):
    return sweep.iloc[(sweep['threshold']-t).abs().argmin()]

row_050 = row_for(0.50)
row_053 = row_for(0.53)

print("\n=== Threshold results (eval) ===")
print(f"Best macro-F1 threshold: {best_thr:.3f} | macro-F1={best_row['macro_f1']:.3f} "
      f"| acc={best_row['accuracy']:.3f} | prec={best_row['precision']:.3f} "
      f"| rec={best_row['recall']:.3f} | F1={best_row['f1']:.3f} | "
      f"cm=[tn={int(best_row['tn'])}, fp={int(best_row['fp'])}, fn={int(best_row['fn'])}, tp={int(best_row['tp'])}]")

print(f"\nAt t=0.50: macro-F1={row_050['macro_f1']:.3f} | acc={row_050['accuracy']:.3f} "
      f"| prec={row_050['precision']:.3f} | rec={row_050['recall']:.3f} | F1={row_050['f1']:.3f}")

print(f"At t=0.53: macro-F1={row_053['macro_f1']:.3f} | acc={row_053['accuracy']:.3f} "
      f"| prec={row_053['precision']:.3f} | rec={row_053['recall']:.3f} | F1={row_053['f1']:.3f}")

# Save sweep
sweep.to_csv("threshold_sweep_eval.csv", index=False)
print("\nSaved: plots_calibration/calibration_curve.png, plots_calibration/prob_hist.png, threshold_sweep_eval.csv")

# Plot macro-F1 vs threshold
plt.figure(figsize=(6,4))
plt.plot(sweep["threshold"], sweep["macro_f1"], label="macro-F1")
plt.axvline(0.50, linestyle="--", label="0.50")
plt.axvline(0.53, linestyle="--", label="0.53")
plt.axvline(best_thr, linestyle=":", label=f"best={best_thr:.2f}")
plt.xlabel("Threshold")
plt.ylabel("Macro-F1")
plt.title("Threshold sweep (eval)")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outdir,"threshold_vs_f1.png"), dpi=160)
plt.show()
print("Saved: plots_calibration/threshold_vs_f1.png")


In [ ]:
# ===== STEP 3: Slice performance & error analysis =====
# Inputs assumed present: estimator, Xte, y_eval, and best threshold (0.53 from Step 2)

import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score
)

THRESH = 0.53  # from Step 2 best macro-F1

def get_prob(est, X):
    try:
        return est.predict_proba(X)[:,1]
    except Exception:
        s = est.decision_function(X)
        s_min, s_max = s.min(), s.max()
        return (s - s_min) / (s_max - s_min + 1e-12)

proba = get_prob(estimator, Xte)
y_true = np.asarray(y_eval).astype(int)
y_pred = (proba >= THRESH).astype(int)

eval_df = pd.DataFrame({
    "y_true": y_true,
    "proba": proba,
    "y_pred": y_pred
}, index=Xte.index)

# ---------- metrics helper ----------
def metrics_at_mask(mask, name):
    mask = mask.astype(bool)
    n = int(mask.sum())
    if n < 30:   # skip tiny slices to avoid noise; adjust if needed
        return None
    yt = y_true[mask]
    pr = proba[mask]
    yp = y_pred[mask]
    acc  = accuracy_score(yt, yp)
    prec = precision_score(yt, yp, zero_division=0)
    rec  = recall_score(yt, yp, zero_division=0)
    f1   = f1_score(yt, yp, zero_division=0)
    f1m  = f1_score(yt, yp, average="macro", zero_division=0)
    try:
        auc = roc_auc_score(yt, pr)
    except Exception:
        auc = np.nan
    tn, fp, fn, tp = confusion_matrix(yt, yp).ravel()
    pos_rate = float(yt.mean())
    return {
        "slice": name, "support": n, "pos_rate": pos_rate,
        "accuracy": acc, "precision": prec, "recall": rec,
        "f1": f1, "macro_f1": f1m, "auc": auc,
        "tn": tn, "fp": fp, "fn": fn, "tp": tp
    }

def eval_dummies(prefix, title, exclude_prefixes=(), topn_plot=12):
    cols = [c for c in Xte.columns if c.startswith(prefix) and not any(c.startswith(ep) for ep in exclude_prefixes)]
    rows = []
    for c in cols:
        res = metrics_at_mask(Xte[c] == 1, c.replace(prefix, f"{title}: "))
        if res: rows.append(res)
    if not rows:
        return pd.DataFrame()
    df = pd.DataFrame(rows).sort_values(["macro_f1","support"], ascending=[False, False])
    out = f"slice_metrics_{title.lower().replace(' ','_')}.csv"
    df.to_csv(out, index=False); print("Saved:", out)

    # quick plot: top slices by macro-F1 (min support filter)
    plot_df = df[df["support"] >= 50].head(topn_plot)
    if len(plot_df) > 0:
        plt.figure(figsize=(9, 5))
        plt.barh(plot_df["slice"][::-1], plot_df["macro_f1"][::-1])
        plt.xlabel("Macro-F1"); plt.title(f"{title} — top slices (support ≥ 50)")
        plt.tight_layout()
        plt.savefig(f"slice_{title.lower().replace(' ','_')}_top.png", dpi=160)
        plt.show()
        print("Saved:", f"slice_{title.lower().replace(' ','_')}_top.png")
    return df

# ---------- 3A. Slices by family ----------
city_df     = eval_dummies("city_", "City")
cuisine_df  = eval_dummies("cat__", "Cuisine")
# Price range one-hots often look like attr_RestaurantsPriceRange1/2/3/4
price_df    = eval_dummies("attr_RestaurantsPriceRange", "PriceRange")
# Amenities = attr_ (excluding price range columns already handled)
amen_df     = eval_dummies("attr_", "Amenity", exclude_prefixes=("attr_RestaurantsPriceRange",))

# ---------- 3B. Review-count quartiles ----------
if "review_count_log1p" in Xte.columns:
    q = pd.qcut(Xte["review_count_log1p"], q=4, duplicates="drop")
    rows = []
    for lvl in q.cat.categories:
        mask = (q == lvl)
        res = metrics_at_mask(mask.values, f"Reviews quartile: {lvl}")
        if res: rows.append(res)
    rc_df = pd.DataFrame(rows).sort_values("support", ascending=False)
    rc_df.to_csv("slice_metrics_review_count_quartiles.csv", index=False)
    print("Saved:", "slice_metrics_review_count_quartiles.csv")
else:
    rc_df = pd.DataFrame()

# ---------- 3C. Top false positives & false negatives ----------
top_features = [f for f in [
    "rl_word_mean","review_count_log1p",
    "attr_RestaurantsAttire","attr_HasTV","attr_WiFi",
    "cat__Fast Food","cat__Pizza","attr_RestaurantsPriceRange2"
] if f in Xte.columns]

# Top 20 FP (pred=1, truth=0) with a few key feature values
fp = eval_df.query("y_true==0 and y_pred==1").copy()
fp["margin"] = fp["proba"] - THRESH
fp_top = fp.sort_values("proba", ascending=False).head(20).join(Xte[top_features])
fp_top.to_csv("top_false_positives.csv"); print("Saved:", "top_false_positives.csv")

# Top 20 FN (pred=0, truth=1)
fn = eval_df.query("y_true==1 and y_pred==0").copy()
fn["margin"] = THRESH - fn["proba"]
fn_top = fn.sort_values("proba", ascending=True).head(20).join(Xte[top_features])
fn_top.to_csv("top_false_negatives.csv"); print("Saved:", "top_false_negatives.csv")

# ---------- 3D. Summary printout ----------
def brief(df, name):
    if df is None or len(df)==0: 
        print(f"{name}: no slices or too small."); return
    best = df.sort_values("macro_f1", ascending=False).head(5)[["slice","support","macro_f1","pos_rate"]]
    print(f"\nTop {name} slices by macro-F1:")
    print(best.to_string(index=False))

brief(city_df, "City")
brief(cuisine_df, "Cuisine")
brief(amen_df, "Amenity")
brief(price_df, "PriceRange")
brief(rc_df, "ReviewCount Quartiles")


In [ ]:
# ===== STEP 4: Refit & freeze (full-data) + inference with reason codes =====
import os, json, datetime as dt
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from joblib import dump, load

# --- helpers
def get_prob(est, X):
    try:
        return est.predict_proba(X)[:,1]
    except Exception:
        s = est.decision_function(X)
        s_min, s_max = s.min(), s.max()
        return (s - s_min) / (s_max - s_min + 1e-12)

THRESH = 0.53  # from STEP 2

# 1) Build full dataset (train ∪ eval) with aligned columns
cols_union = Xtr.columns.union(Xte.columns)
Xtr_aligned = Xtr.reindex(columns=cols_union, fill_value=0)
Xte_aligned = Xte.reindex(columns=cols_union, fill_value=0)

X_full = pd.concat([Xtr_aligned, Xte_aligned], axis=0, ignore_index=True)
y_full = pd.concat([pd.Series(y_train).reset_index(drop=True),
                    pd.Series(y_eval).reset_index(drop=True)], axis=0).astype(int).values

# 2) Refit champion on full data
est_full = clone(estimator)
est_full.fit(X_full, y_full)

# 3) Confirm eval metrics at chosen threshold (sanity check)
proba_eval = get_prob(est_full, Xte_aligned)
y_pred_eval = (proba_eval >= THRESH).astype(int)
metrics_eval = {
    "accuracy": float(accuracy_score(y_eval, y_pred_eval)),
    "precision": float(precision_score(y_eval, y_pred_eval, zero_division=0)),
    "recall": float(recall_score(y_eval, y_pred_eval, zero_division=0)),
    "f1": float(f1_score(y_eval, y_pred_eval, zero_division=0)),
    "macro_f1": float(f1_score(y_eval, y_pred_eval, average="macro", zero_division=0)),
    "auc": float(roc_auc_score(y_eval, proba_eval))
}
print("Eval metrics @0.53 after refit:", metrics_eval)

# 4) Save artifacts
ARTDIR = "artifacts_final"
os.makedirs(ARTDIR, exist_ok=True)
dump(est_full, f"{ARTDIR}/final_hgb_svd_v1.joblib")
json.dump(list(cols_union), open(f"{ARTDIR}/columns_v1.json","w"))

# Background means for reason-codes neutralization
bg_means = {}
for c in cols_union:
    ser = X_full[c]
    if np.issubdtype(ser.dtype, np.number):
        m = float(np.nanmean(ser.values))
        bg_means[c] = 0.0 if np.isnan(m) else m
    else:
        # try numeric coercion, else default 0
        coerced = pd.to_numeric(ser, errors="coerce")
        m = float(np.nanmean(coerced.values)) if coerced.notna().any() else 0.0
        bg_means[c] = 0.0 if np.isnan(m) else m
json.dump(bg_means, open(f"{ARTDIR}/background_means_v1.json","w"))

# Simple metadata
meta = {
    "model": "HGB(SVD) pipeline",
    "version": "v1",
    "timestamp": dt.datetime.utcnow().isoformat() + "Z",
    "threshold": THRESH,
    "eval_metrics_after_refit": metrics_eval,
    "n_train_full": int(len(X_full)),
    "n_features": int(len(cols_union))
}
json.dump(meta, open(f"{ARTDIR}/metadata_v1.json","w"), indent=2)
print("Saved artifacts in:", ARTDIR)

# 5) Lightweight inference with reason codes (top-k features)
def load_artifacts(path="artifacts_final"):
    est = load(f"{path}/final_hgb_svd_v1.joblib")
    cols = json.load(open(f"{path}/columns_v1.json"))
    bg = json.load(open(f"{path}/background_means_v1.json"))
    return est, cols, bg

def predict_with_reasons(sample_df, top_features=None, threshold=THRESH, artifacts_dir="artifacts_final"):
    """
    sample_df: DataFrame with a SINGLE row, same schema as training (missing cols OK).
    Returns dict: {'prob','class','reasons':[(feature, delta_prob),...]}
    """
    est, cols, bg = load_artifacts(artifacts_dir)
    X = sample_df.reindex(columns=cols, fill_value=0)
    base = float(get_prob(est, X)[0])

    # choose features: use stability CSV top 10 if not provided
    if top_features is None:
        agg = pd.read_csv("perm_importance_stability.csv", index_col=0)
        top_features = [c for c in agg.sort_values("importance_mean", ascending=False).index if c in X.columns][:10]

    # one-at-a-time neutralization delta (positive delta => feature increases current prob)
    deltas = []
    for f in top_features:
        Xtmp = X.copy()
        neutral = 0.0 if f.startswith(("cat__","attr_","city_")) else bg.get(f, 0.0)
        Xtmp.at[Xtmp.index[0], f] = neutral
        p = float(get_prob(est, Xtmp)[0])
        deltas.append((f, base - p))
    deltas.sort(key=lambda t: abs(t[1]), reverse=True)

    return {
        "prob": base,
        "class": int(base >= threshold),
        "threshold": threshold,
        "reasons": deltas
    }

# --- Example usage on one eval row (optional sanity check) ---
example = Xte_aligned.sample(1, random_state=7)
result = predict_with_reasons(example)
print("Example prediction:", {k:(round(v,4) if isinstance(v,float) else v) for k,v in result.items() if k!='reasons'})
print("Top reasons (feature, delta_prob):", [(f, round(d,4)) for f,d in result["reasons"][:5]])


In [ ]:
# ===== STEP 5: Deliverables pack =====
import os, glob, json, numpy as np, pandas as pd
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

THRESH = 0.53
OUTDIR = "deliverables"
os.makedirs(OUTDIR, exist_ok=True)

def get_prob(est, X):
    try:
        return est.predict_proba(X)[:,1]
    except Exception:
        s = est.decision_function(X)
        s_min, s_max = s.min(), s.max()
        return (s - s_min) / (s_max - s_min + 1e-12)

def metrics_table_row(name, est, X, y, thr):
    p = get_prob(est, X)
    ypred = (p >= thr).astype(int)
    return {
        "model": name,
        "threshold": thr,
        "accuracy": accuracy_score(y, ypred),
        "precision": precision_score(y, ypred, zero_division=0),
        "recall": recall_score(y, ypred, zero_division=0),
        "f1": f1_score(y, ypred, zero_division=0),
        "macro_f1": f1_score(y, ypred, average="macro", zero_division=0),
        "auc": roc_auc_score(y, p)
    }

rows = []
# Champion on proper hold-out (the 'estimator' you used in Steps 1–3)
rows.append(metrics_table_row("HGB(SVD) — holdout", estimator, Xte, y_eval, THRESH))

# Also include post-refit sanity (trained on full data; optimistic — label it clearly)
try:
    est_full = load("artifacts_final/final_hgb_svd_v1.joblib")
    rows.append(metrics_table_row("HGB(SVD) — post-refit (sanity only)", est_full, Xte, y_eval, THRESH))
except Exception:
    pass

# Optional: LogReg-enet if available
for cand in ["logreg_enet.joblib","artifacts/logreg_enet.joblib","models/logreg_enet.joblib"]:
    if os.path.exists(cand):
        logreg = load(cand)
        rows.append(metrics_table_row("LogReg-enet — holdout", logreg, Xte, y_eval, THRESH))
        break  # first one wins

results_df = pd.DataFrame(rows)
results_path = os.path.join(OUTDIR, "results_table.csv")
results_df.to_csv(results_path, index=False)
print("Saved:", results_path)
print(results_df)

# ---------- Actionable insights text ----------
insights_lines = []

# Pull top individual features (stability CSV)
try:
    stab = pd.read_csv("perm_importance_stability.csv", index_col=0).sort_values("importance_mean", ascending=False)
    top_feats = stab.head(8).index.tolist()
    insights_lines.append(f"Top single features by stable permutation importance: {', '.join(top_feats)}.")
except Exception:
    pass

# Pull group importance
try:
    group_imp = pd.read_csv("grouped_permutation_importance.csv").sort_values("importance_mean", ascending=False)
    top_groups = group_imp.head(3)[["group","importance_mean"]].values.tolist()
    desc = "; ".join([f"{g} (ΔAUC≈{v:.3f})" for g,v in top_groups])
    insights_lines.append(f"Most influential families (grouped permutation): {desc}.")
except Exception:
    pass

# Pull ablation (schema-preserving)
try:
    abl = pd.read_csv("group_ablation_refit_schema_preserving.csv").sort_values("auc_delta", ascending=False)
    keepers = abl.head(3)[["group","auc_delta","n_cols_neutralized"]].values.tolist()
    bullets = "; ".join([f"{g} (ΔAUC≈{d:.3f} across {n} cols)" for g,d,n in keepers])
    insights_lines.append(f"Ablation confirms uniqueness: {bullets}.")
except Exception:
    pass

# Calibration/threshold summary
try:
    sweep = pd.read_csv("threshold_sweep_eval.csv")
    best_row = sweep.loc[sweep["macro_f1"].idxmax()]
    insights_lines.append(
        f"Calibration: Brier≈{open('plots_calibration/calibration_curve.png') and 'see plot'}. "
        f"Best macro-F1 at threshold≈{float(best_row['threshold']):.2f}."
    )
except Exception:
    pass

insights_lines.append(
    "Practical levers: encourage richer reviews (rl_word_mean), maintain accurate amenity listings (WiFi, Attire), "
    "and increase authentic review volume (review_count_log1p). Treat cuisine/category as segmentation for benchmarks."
)

insights_text = "\n".join(f"- {ln}" for ln in insights_lines)
insights_path = os.path.join(OUTDIR, "actionable_insights.txt")
with open(insights_path, "w") as f:
    f.write("Actionable insights for boosting star ratings\n")
    f.write("="*48 + "\n")
    f.write(insights_text + "\n")
print("Saved:", insights_path)

# ---------- Model card ----------
card_path = os.path.join(OUTDIR, "model_card.md")
with open(card_path, "w") as f:
    f.write("# Model Card — Yelp Ratings Predictor (HGB(SVD))\n\n")
    f.write("**Objective:** Predict high star ratings and surface actionable levers to improve them.\n\n")
    f.write("**Champion:** HGB(SVD) pipeline; threshold 0.53 (macro-F1-optimal on holdout).\n\n")
    f.write("**Validation:** Train/holdout split; reporting on holdout.\n\n")
    f.write("**Key signals:** review richness (rl_word_mean), review volume (review_count_log1p), "
            "amenities (attr_*), cuisines (cat__), clusters.\n\n")
    f.write("**Limitations:** Associations, not causation; category effects are segmentation signals; "
            "avoid manipulative review incentives; monitor for drift quarterly.\n\n")
    f.write("**Artifacts:** `artifacts_final/final_hgb_svd_v1.joblib`, columns JSON, background means JSON.\n")
print("Saved:", card_path)

# ---------- Plot manifest ----------
plots = []
for pat in ["plots_*/*.png","plots_*/*/*.png","slice_*_top.png"]:
    plots.extend(glob.glob(pat))
manifest_path = os.path.join(OUTDIR, "plot_manifest.txt")
with open(manifest_path,"w") as f:
    f.write("\n".join(sorted(plots)))
print("Saved:", manifest_path)
print(f"\nTotal plots found: {len(plots)}")


In [ ]:
# ===== STEP 6: Generate executive_summary.md =====
import os, json, pandas as pd, textwrap, glob

OUTDIR = "deliverables"
os.makedirs(OUTDIR, exist_ok=True)

# 1) Load artifacts we already produced
results = pd.read_csv(f"{OUTDIR}/results_table.csv")
holdout_row = results[results["model"].str.contains("holdout", case=False)].iloc[0]

insights_txt = ""
if os.path.exists(f"{OUTDIR}/actionable_insights.txt"):
    with open(f"{OUTDIR}/actionable_insights.txt","r") as f:
        insights_txt = f.read().strip()

# optional: pick a few plot paths if they exist
plots = []
plots += [p for p in [
    "plots_calibration/calibration_curve.png",
    "plots_calibration/threshold_vs_f1.png",
] if os.path.exists(p)]
plots += sorted(glob.glob("plots_pdp_ice_fixed/pdp_ice__*.png"))[:4]  # first 4 PDP/ICE

# 2) Short bullets from slice CSVs (top-3 each if present)
def top_slices(path, col="slice", k=3):
    if not os.path.exists(path): return []
    df = pd.read_csv(path).sort_values(["macro_f1","support"], ascending=[False, False])
    df = df[df["support"]>=50].head(k)
    return [f"{r[col]} — macro-F1 {r['macro_f1']:.3f} (n={int(r['support'])})" for _,r in df.iterrows()]

city_bul = top_slices("slice_metrics_city.csv")
cui_bul  = top_slices("slice_metrics_cuisine.csv")
amen_bul = top_slices("slice_metrics_amenity.csv")
price_bul= top_slices("slice_metrics_pricerange.csv")
revq_bul = top_slices("slice_metrics_review_count_quartiles.csv")

# 3) Build markdown
md = []
md.append("# Executive Summary — Yelp Ratings Predictor\n")
md.append("**Objective:** Predict high star ratings and surface practical levers stakeholders can act on to improve them.\n")
md.append("## Champion model\n")
md.append(f"- **Model:** HGB(SVD) pipeline\n- **Holdout threshold:** 0.53 (macro-F1-optimal)\n")
md.append(f"- **Holdout metrics:** accuracy **{holdout_row['accuracy']:.3f}**, macro-F1 **{holdout_row['macro_f1']:.3f}**, AUC **{holdout_row['auc']:.3f}**\n")
md.append("*(A separate post-refit metric was computed for sanity only and is not for external reporting.)*\n")

md.append("## Key insights (actionable)\n")
if insights_txt:
    md.append(insights_txt + "\n")
else:
    md.append("- See grouped importance and ablation CSVs for evidence.\n")

def section_list(title, items):
    if not items: return ""
    s = [f"## {title}\n"]
    s += [f"- {it}" for it in items]
    s.append("") 
    return "\n".join(s)

md.append(section_list("Strong slices (City)", city_bul))
md.append(section_list("Strong slices (Cuisine)", cui_bul))
md.append(section_list("Strong slices (Amenity)", amen_bul))
md.append(section_list("Strong slices (PriceRange)", price_bul))
md.append(section_list("Performance by review volume", revq_bul))

md.append("## Plots\n")
if plots:
    md += [f"- {p}" for p in plots]
else:
    md.append("- Plots were generated in earlier steps; see `plots_*` folders.")

md.append("\n## Limitations & guardrails\n- Correlational signals; use for hypotheses and A/B tests, not causal claims.\n- Category effects guide **benchmarking** rather than direct changes.\n- Avoid manipulative review tactics; emphasize genuine, detailed feedback.\n")

path = f"{OUTDIR}/executive_summary.md"
with open(path, "w") as f:
    f.write("\n".join(md))
print("Saved:", path)


In [ ]:
# Zip & handoff (bundle artifacts) 
import os, glob, json, time, zipfile, textwrap, pandas as pd

OUTZIP = "yelp_ratings_project_v1.zip"
BUNDLE_DIRS = [
    "artifacts_final",
    "deliverables",
    "plots_calibration",
    "plots_pdp_ice",
    "plots_pdp_ice_fixed",
]
BUNDLE_FILES = [
    "perm_importance_stability.csv",
    "grouped_permutation_importance.csv",
    "group_ablation_refit_schema_preserving.csv",
    "group_ablation_pairs_zero_mean.csv",
    "group_ablation_single_zero_mean.csv",
    "threshold_sweep_eval.csv",
    "slice_metrics_city.csv",
    "slice_metrics_cuisine.csv",
    "slice_metrics_amenity.csv",
    "slice_metrics_pricerange.csv",
    "slice_metrics_review_count_quartiles.csv",
    "top_false_positives.csv",
    "top_false_negatives.csv",
    "slice_city_top.png",
    "slice_cuisine_top.png",
    "slice_amenity_top.png",
    "slice_pricerange_top.png",
]

# 1) (Re)create inference script
INFER_PY = "inference_demo.py"
infer_lines = [
    "import json, pandas as pd",
    "from joblib import load",
    "",
    "THRESH = 0.53",
    "",
    "def _prob(est, X):",
    "    try:",
    "        return est.predict_proba(X)[:,1]",
    "    except Exception:",
    "        s = est.decision_function(X)",
    "        s = (s - s.min()) / (s.max() - s.min() + 1e-12)",
    "        return s",
    "",
    "def load_artifacts(path='artifacts_final'):",
    "    est = load(f\"{path}/final_hgb_svd_v1.joblib\")",
    "    cols = json.load(open(f\"{path}/columns_v1.json\"))",
    "    bg = json.load(open(f\"{path}/background_means_v1.json\"))",
    "    return est, cols, bg",
    "",
    "def predict_with_reasons(sample_df, top_features=None, threshold=THRESH):",
    "    est, cols, bg = load_artifacts()",
    "    X = sample_df.reindex(columns=cols, fill_value=0)",
    "    base = float(_prob(est, X)[0])",
    "    if top_features is None:",
    "        try:",
    "            stab = pd.read_csv('perm_importance_stability.csv', index_col=0)",
    "            top_features = [c for c in stab.sort_values('importance_mean', ascending=False).index if c in X.columns][:10]",
    "        except Exception:",
    "            top_features = list(X.columns[:10])",
    "    deltas = []",
    "    for f in top_features:",
    "        Xtmp = X.copy()",
    "        neutral = 0.0 if f.startswith(('cat__','attr_','city_')) else bg.get(f, 0.0)",
    "        try:",
    "            Xtmp.at[Xtmp.index[0], f] = neutral",
    "        except Exception:",
    "            Xtmp[f] = neutral",
    "        p = float(_prob(est, Xtmp)[0])",
    "        deltas.append((f, base - p))",
    "    deltas.sort(key=lambda t: abs(t[1]), reverse=True)",
    "    return {'prob': base, 'class': int(base >= threshold), 'threshold': threshold, 'reasons': deltas}",
    "",
    "if __name__ == '__main__':",
    "    import sys",
    "    if len(sys.argv) < 2:",
    "        print('Usage: python inference_demo.py <single_row.csv>'); raise SystemExit(1)",
    "    df = pd.read_csv(sys.argv[1])",
    "    res = predict_with_reasons(df)",
    "    print(res)",
]
with open(INFER_PY, "w") as f:
    f.write("\n".join(infer_lines))

# 2) (Re)create README without triple-quoted strings
README = "README_SUBMISSION.md"
readme_lines = [
    "# Yelp Ratings Project — Submission (v1)",
    "",
    "**What’s inside**",
    "- `deliverables/executive_summary.md` — one-pager to share with stakeholders",
    "- `deliverables/results_table.csv` — holdout metrics (HGB(SVD) @ t=0.53)",
    "- `plots_*/*.png` — calibration, threshold sweep, PDP/ICE, top-slice charts",
    "- Analysis CSVs — permutation stability, grouped importance, ablations, slices",
    "- `artifacts_final/` — final model + columns + background means + metadata",
    "- `inference_demo.py` — quick script to score a single row with reason codes",
    "",
    "**How to run the demo**",
    "```bash",
    "pip install scikit-learn joblib pandas numpy",
    "python inference_demo.py example_row.csv",
    "```",
    "",
    "**Notes**",
    "- Report **holdout** metrics only (post-refit metrics are sanity checks).",
    "- Insights are correlational; use for hypotheses and small A/B tests.",
    "- Threshold used: **0.53** (best macro-F1 on holdout).",
]
with open(README, "w") as f:
    f.write("\n".join(readme_lines))

# 3) Build manifest list
all_paths = []
for d in BUNDLE_DIRS:
    if os.path.isdir(d):
        for root, _, files in os.walk(d):
            for fn in files:
                all_paths.append(os.path.join(root, fn))
for f in BUNDLE_FILES:
    if os.path.exists(f):
        all_paths.append(f)
# include PDP plots if present, both folders
all_paths += glob.glob("plots_pdp_ice_fixed/*.png")
all_paths += glob.glob("plots_pdp_ice/*.png")
# always include README + inference script
all_paths += [README, INFER_PY]

# 4) Manifest CSV
manifest = []
for p in sorted(set(all_paths)):
    try:
        sz = os.path.getsize(p)
        mt = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(os.path.getmtime(p)))
    except Exception:
        sz, mt = 0, ""
    manifest.append({"path": p, "size_bytes": sz, "modified": mt})
mf = pd.DataFrame(manifest)
mf_path = "submission_manifest.csv"
mf.to_csv(mf_path, index=False)
print("Saved:", mf_path)

# 5) Zip everything
with zipfile.ZipFile(OUTZIP, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in sorted(set(all_paths + [mf_path])):
        if os.path.exists(p):
            z.write(p)
print("Saved:", OUTZIP)

# 6) Preview contents
with zipfile.ZipFile(OUTZIP, "r") as z:
    names = z.namelist()
print("Zip contains", len(names), "files. First 15 entries:")
for n in names[:15]:
    print(" -", n)


In [ ]:
# ===== STEP 8A: LogReg-enet coefficient plots (NaN-safe, numeric-only) =====
import os, re, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from joblib import load

os.makedirs("plots_logreg", exist_ok=True)

# --- Resolve train vars defensively
X_train = globals().get("Xtr", globals().get("Xtr2"))
y_train_vec = globals().get("y_train", globals().get("ytr"))
assert X_train is not None and y_train_vec is not None, "Training variables not found."

# --- Try to load an existing LogReg; else fit one
logreg_path = None
for cand in ["logreg_enet.joblib", "artifacts/logreg_enet.joblib", "models/logreg_enet.joblib"]:
    if os.path.exists(cand):
        logreg_path = cand; break

if logreg_path:
    logreg_pipe = load(logreg_path)
else:
    # Numeric-only split into continuous vs binary-like
    num_cols = [c for c in X_train.columns if np.issubdtype(X_train[c].dtype, np.number)]
    cont_cols = [c for c in num_cols if X_train[c].nunique(dropna=True) > 2]
    bin_cols  = [c for c in num_cols if X_train[c].nunique(dropna=True) <= 2]

    pre = ColumnTransformer(
        transformers=[
            ("cont", Pipeline([
                ("imp", SimpleImputer(strategy="median")),
                ("sc", StandardScaler())
            ]), cont_cols),
            ("bin", Pipeline([
                ("imp", SimpleImputer(strategy="most_frequent"))
            ]), bin_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )

    logreg = LogisticRegression(
        solver="saga", penalty="elasticnet", l1_ratio=0.5, C=1.0,
        max_iter=5000, n_jobs=-1, random_state=42
    )
    logreg_pipe = Pipeline([("pre", pre), ("clf", logreg)])
    logreg_pipe.fit(X_train, y_train_vec)

# --- Get feature names and coefs
try:
    feat_names = logreg_pipe.named_steps["pre"].get_feature_names_out()
except Exception:
    # Fallback if get_feature_names_out unavailable
    # Rebuild names from the ColumnTransformer lists
    ct = logreg_pipe.named_steps["pre"]
    cont_cols = ct.transformers_[0][2]
    bin_cols  = ct.transformers_[1][2]
    feat_names = np.array(list(cont_cols) + list(bin_cols))

coefs = logreg_pipe.named_steps["clf"].coef_.ravel()
coef_df = pd.DataFrame({"feature": feat_names, "coef": coefs}).sort_values("coef", ascending=False)

# --- Group OHE back to readable fields
def group_key(feat):
    if feat.startswith("cat__"):
        return ("Cuisine", feat.replace("cat__", ""))
    if feat.startswith("city_"):
        return ("City", feat.replace("city_", ""))
    if feat.startswith("attr_"):
        # Strip trailing digits like PriceRange2 -> PriceRange
        base = re.sub(r"\d+$", "", feat.replace("attr_", ""))
        return ("Amenity", base)
    return ("Feature", feat)

coef_df[["group_type","group_name"]] = coef_df["feature"].apply(lambda f: pd.Series(group_key(f)))
group_sum = coef_df.groupby(["group_type","group_name"], as_index=False)["coef"].sum()

top_pos = group_sum.sort_values("coef", ascending=False).head(15)
top_neg = group_sum.sort_values("coef", ascending=True).head(15)

print("\nTop positive groups (sum of coeffs):")
print(top_pos[["group_type","group_name","coef"]].to_string(index=False))
print("\nTop negative groups (sum of coeffs):")
print(top_neg[["group_type","group_name","coef"]].to_string(index=False))

# --- Plot grouped bars
stack = pd.concat([top_neg.assign(side="neg"), top_pos.assign(side="pos")], axis=0)
labels = stack.apply(lambda r: f"{r['group_type']}: {r['group_name']}", axis=1)

fig, ax = plt.subplots(figsize=(10,7))
ax.barh(labels, stack["coef"])
ax.axvline(0, linestyle="--", linewidth=1)
ax.set_title("LogReg-enet — grouped coefficients (OHE aggregated)")
ax.set_xlabel("Coefficient (sum across dummies)")
plt.tight_layout()
plt.savefig("plots_logreg/coef_groups_bar.png", dpi=160)
plt.show()
print("Saved: plots_logreg/coef_groups_bar.png")

# --- Plot top individual features
sing_pos = coef_df.head(15)
sing_neg = coef_df.tail(15).sort_values("coef")
sing_stack = pd.concat([sing_neg.assign(side="neg"), sing_pos.assign(side="pos")], axis=0)

fig, ax = plt.subplots(figsize=(10,7))
ax.barh(sing_stack["feature"], sing_stack["coef"])
ax.axvline(0, linestyle="--", linewidth=1)
ax.set_title("LogReg-enet — top individual coefficients")
ax.set_xlabel("Coefficient")
plt.tight_layout()
plt.savefig("plots_logreg/coef_top_singles.png", dpi=160)
plt.show()
print("Saved: plots_logreg/coef_top_singles.png")


In [ ]:
#Prepare unsupervised representation & choose K (elbow + silhouette)

In [ ]:
# ===== STEP U1 (fixed): Unsupervised prep + K selection =====
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from joblib import dump

# -------- resolve full feature frame --------
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

assert isinstance(X_full, pd.DataFrame) and X_full.shape[1] > 0

# -------- directories --------
os.makedirs("plots_unsup", exist_ok=True)
os.makedirs("unsup_artifacts", exist_ok=True)

# -------- split columns by dtype --------
num_cols = [c for c in X_full.columns if np.issubdtype(X_full[c].dtype, np.number)]
cat_cols = [c for c in X_full.columns if c not in num_cols]  # strings, categories, bool-as-object, etc.

# Numeric: median-impute + scale (with_mean=False to be sparse-safe)
num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler(with_mean=False))
])

# Categorical: most-frequent impute + OHE (ignore unknowns at transform time)
cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
])

ct = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols)
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# Fit transform -> combined feature matrix (dense or sparse depending on mix)
X_feat = ct.fit_transform(X_full)

# -------- SVD on combined matrix --------
SVD_COMPONENTS = min(30, (X_feat.shape[1] - 1)) if X_feat.shape[1] > 1 else 1
svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=42)
Z = svd.fit_transform(X_feat)   # latent representation (n x SVD_COMPONENTS)

# Save artifacts
dump(ct,  "unsup_artifacts/ct_preproc.joblib")
dump(svd, "unsup_artifacts/svd.joblib")
pd.DataFrame(Z, columns=[f"z{i+1}" for i in range(Z.shape[1])]).to_csv("unsup_artifacts/latent_z.csv", index=False)

# -------- K selection (MiniBatchKMeans) --------
K_RANGE = list(range(2, 11))
inertias, sil_scores = [], []

rng = np.random.default_rng(42)
idx_sample = rng.choice(len(Z), size=min(5000, len(Z)), replace=False)

for k in K_RANGE:
    km = MiniBatchKMeans(
        n_clusters=k, random_state=42, n_init=10,
        batch_size=2048, max_iter=200
    )
    km.fit(Z)
    inertias.append(km.inertia_)
    sil_scores.append(silhouette_score(Z[idx_sample], km.predict(Z[idx_sample])))

# -------- plots --------
plt.figure(figsize=(6.5,4.2))
plt.plot(K_RANGE, inertias, marker="o")
plt.title("Elbow (MiniBatchKMeans on SVD space)")
plt.xlabel("k"); plt.ylabel("Inertia"); plt.xticks(K_RANGE)
plt.tight_layout(); plt.savefig("plots_unsup/elbow_kmeans.png", dpi=160); plt.show()

best_k = int(K_RANGE[int(np.argmax(sil_scores))])
best_sil = float(max(sil_scores))

plt.figure(figsize=(6.5,4.2))
plt.plot(K_RANGE, sil_scores, marker="o")
plt.axvline(best_k, linestyle="--", alpha=0.6)
plt.title(f"Silhouette by k (best k={best_k}, score={best_sil:.3f})")
plt.xlabel("k"); plt.ylabel("Silhouette"); plt.xticks(K_RANGE); plt.ylim(0,1)
plt.tight_layout(); plt.savefig("plots_unsup/silhouette_kmeans.png", dpi=160); plt.show()

# -------- summary table --------
tbl = pd.DataFrame({"k":K_RANGE, "inertia":inertias, "silhouette":sil_scores})
print("\n[STEP U1] K selection summary")
print(tbl.to_string(index=False))
print(f"\nSuggested k (max silhouette): {best_k}  |  silhouette={best_sil:.3f}")
print("Saved: plots_unsup/elbow_kmeans.png, plots_unsup/silhouette_kmeans.png, unsup_artifacts/*")


In [ ]:
# ===== STEP U2: Fit clusters, assign labels, and profile =====
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

os.makedirs("unsup_artifacts", exist_ok=True)
os.makedirs("unsup_outputs", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# --- 1) Reload preprocessing and latent space
ct = load("unsup_artifacts/ct_preproc.joblib")
svd = load("unsup_artifacts/svd.joblib")

# Build X_full again the same way we did in U1 (train∪eval union if needed)
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

# Transform to feature matrix -> latent Z
X_feat = ct.transform(X_full)
Z = svd.transform(X_feat)

# Optional post-hoc labels for interpretation (NOT used in clustering)
y_full = globals().get("y_full")
if y_full is None:
    y_tr = globals().get("y_train", globals().get("ytr"))
    y_te = globals().get("y_eval")
    if y_tr is not None and y_te is not None:
        y_full = np.concatenate([np.asarray(y_tr).ravel(), np.asarray(y_te).ravel()])
    else:
        y_full = None

# --- 2) Fit MiniBatchKMeans
K = 2  # from U1 suggestion (silhouette max). Change here if you prefer another k.
km = MiniBatchKMeans(
    n_clusters=K, random_state=42, n_init=10,
    batch_size=2048, max_iter=300
)
km.fit(Z)
labels = km.labels_
dump(km, f"unsup_artifacts/kmeans_k{K}.joblib")
pd.DataFrame({"row_id": np.arange(len(labels)), "cluster": labels}).to_csv(
    f"unsup_artifacts/cluster_labels_k{K}.csv", index=False
)

# Silhouette (overall + per-sample for quick glance)
try:
    sil = silhouette_score(Z, labels)
except Exception:
    sil = np.nan

# --- 3) Feature names of the preprocessed matrix
try:
    feat_names = ct.get_feature_names_out()
except Exception:
    # Fallback: try to infer from transformers
    feat_names = np.array([f"f{i}" for i in range(X_feat.shape[1])])

# Compute global mean (preprocessed space) and per-cluster means
# For sparse matrix, .mean(axis=0) returns 1 x d sparse; convert to dense
if hasattr(X_feat, "toarray"):
    global_mean = np.asarray(X_feat.mean(axis=0)).ravel()
else:
    global_mean = X_feat.mean(axis=0)

cluster_means = []
for c in range(K):
    idx = (labels == c)
    Xc = X_feat[idx]
    mc = np.asarray(Xc.mean(axis=0)).ravel() if hasattr(Xc, "toarray") else Xc.mean(axis=0)
    cluster_means.append(mc)
cluster_means = np.vstack(cluster_means)  # K x d

# Build a tidy profile of means
profile_rows = []
for c in range(K):
    prof = pd.DataFrame({
        "cluster": c,
        "feature": feat_names,
        "mean_in_cluster": cluster_means[c],
        "mean_global": global_mean
    })
    prof["diff_from_global"] = prof["mean_in_cluster"] - prof["mean_global"]
    profile_rows.append(prof)
profile_df = pd.concat(profile_rows, ignore_index=True)

# Save full profile means (may be large)
profile_df.to_csv("unsup_outputs/cluster_profile_means.csv", index=False)

# --- 4) Top signals per cluster (most over/under-represented)
top_signal_rows = []
TOPN = 10
for c in range(K):
    sub = profile_df[profile_df["cluster"] == c].copy()
    sub["abs_diff"] = sub["diff_from_global"].abs()
    top_pos = sub.sort_values("diff_from_global", ascending=False).head(TOPN)
    top_neg = sub.sort_values("diff_from_global", ascending=True).head(TOPN)
    top_pos["direction"] = "over"
    top_neg["direction"] = "under"
    sig = pd.concat([top_pos, top_neg], ignore_index=True)
    sig["rank_within_direction"] = sig.groupby("direction")["abs_diff"].rank(ascending=False, method="first")
    sig["cluster"] = c
    top_signal_rows.append(sig)
signals_df = pd.concat(top_signal_rows, ignore_index=True)
signals_df.to_csv("unsup_outputs/cluster_top_signals.csv", index=False)

# --- 5) Post-hoc rating (or success) by cluster
summary_rows = []
for c in range(K):
    idx = (labels == c)
    n = int(idx.sum())
    frac = n / len(labels)
    row = {"cluster": c, "size": n, "share": frac}
    if y_full is not None:
        row["avg_label"] = float(np.mean(np.asarray(y_full)[idx]))
    summary_rows.append(row)
summary_df = pd.DataFrame(summary_rows).sort_values("cluster")
summary_df.to_csv("unsup_outputs/cluster_summary.csv", index=False)

# --- 6) Quick visuals
# 2D scatter in SVD space (z1 vs z2)
plt.figure(figsize=(7.2, 5.8))
# sample to keep plotting light
rng = np.random.default_rng(42)
sample_idx = rng.choice(len(Z), size=min(6000, len(Z)), replace=False)
for c in range(K):
    m = sample_idx[labels[sample_idx] == c]
    plt.scatter(Z[m, 0], Z[m, 1], s=8, alpha=0.6, label=f"Cluster {c}")
plt.title(f"Clusters in SVD space (k={K}) — z1 vs z2")
plt.xlabel("z1"); plt.ylabel("z2"); plt.legend(markerscale=2)
plt.tight_layout(); plt.savefig(f"plots_unsup/cluster_scatter_z12_k{K}.png", dpi=160); plt.show()
print("Saved:", f"plots_unsup/cluster_scatter_z12_k{K}.png")

# Bar of post-hoc average label
if y_full is not None and "avg_label" in summary_df.columns:
    plt.figure(figsize=(6.5, 4.2))
    plt.bar(summary_df["cluster"].astype(str), summary_df["avg_label"])
    plt.xlabel("Cluster"); plt.ylabel("Avg. label (post-hoc)")
    plt.title("Average label by cluster (post-hoc; not used in training)")
    plt.tight_layout(); plt.savefig(f"plots_unsup/cluster_avg_rating_k{K}.png", dpi=160); plt.show()
    print("Saved:", f"plots_unsup/cluster_avg_rating_k{K}.png")

# --- 7) Console summaries
print("\n[STEP U2] Clustering summary")
print("Silhouette (overall):", round(float(sil), 3))
print(summary_df.to_string(index=False))

print("\nTop over/under signals per cluster (head):")
for c in range(K):
    sub = signals_df[signals_df["cluster"] == c]
    # show human-friendly top 5 per direction
    pos5 = sub[sub["direction"]=="over"].sort_values("diff_from_global", ascending=False).head(5)
    neg5 = sub[sub["direction"]=="under"].sort_values("diff_from_global", ascending=True).head(5)
    print(f"\nCluster {c} — over-represented:")
    print(pos5[["feature","diff_from_global"]].to_string(index=False))
    print(f"Cluster {c} — under-represented:")
    print(neg5[["feature","diff_from_global"]].to_string(index=False))


In [ ]:
# ===== STEP U3: Readable cluster profiles + stakeholder insights =====
import os, re, numpy as np, pandas as pd
from joblib import load
import matplotlib.pyplot as plt

os.makedirs("unsup_outputs", exist_ok=True)

# --- Load artifacts from U1/U2
ct  = load("unsup_artifacts/ct_preproc.joblib")
svd = load("unsup_artifacts/svd.joblib")
km  = load("unsup_artifacts/kmeans_k2.joblib")
labels_df = pd.read_csv("unsup_artifacts/cluster_labels_k2.csv")
labels = labels_df["cluster"].to_numpy()
K = km.n_clusters

# Rebuild X_full as before
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

# Transform to preprocessed feature matrix
X_feat = ct.transform(X_full)

# --- Reconstruct readable feature names
# Numeric block names (first transformer)
num_cols = ct.transformers_[0][2]
# Categorical block names (second transformer) using OHE
cat_cols = ct.transformers_[1][2]
ohe      = ct.named_transformers_["cat"].named_steps["ohe"]
cat_names = ohe.get_feature_names_out(cat_cols)

feat_names = np.array(list(num_cols) + list(cat_names))

# --- Compute global & per-cluster means in preprocessed space
if hasattr(X_feat, "toarray"):
    global_mean = np.asarray(X_feat.mean(axis=0)).ravel()
else:
    global_mean = X_feat.mean(axis=0)

cluster_means = []
for c in range(K):
    idx = (labels == c)
    Xc = X_feat[idx]
    mc = np.asarray(Xc.mean(axis=0)).ravel() if hasattr(Xc, "toarray") else Xc.mean(axis=0)
    cluster_means.append(mc)
cluster_means = np.vstack(cluster_means)

profile_rows = []
for c in range(K):
    prof = pd.DataFrame({
        "cluster": c,
        "feature": feat_names,
        "mean_in_cluster": cluster_means[c],
        "mean_global": global_mean
    })
    prof["diff_from_global"] = prof["mean_in_cluster"] - prof["mean_global"]
    profile_rows.append(prof)
profile_readable = pd.concat(profile_rows, ignore_index=True)

# Save full profile with readable names
profile_path = "unsup_outputs/cluster_profile_means_readable.csv"
profile_readable.to_csv(profile_path, index=False)
print("Saved:", profile_path)

# --- Top signals per cluster (readable)
def base_feature_name(f):
    """
    Try to collapse OHE names like 'attr_HasTV_False' -> 'attr_HasTV'
    Leave continuous & already-atomic names untouched.
    """
    # If it's an OHE name, it looks like "<origcol>_<category>"
    # Heuristic: if the base column exists in original X_full columns, keep it;
    # otherwise strip off the last underscore chunk for readability.
    if f in X_full.columns:
        return f
    # Try stripping last '_category' chunk
    parts = f.split("_")
    if len(parts) > 1:
        return "_".join(parts[:-1])
    return f

TOPN = 10
signals_rows = []
for c in range(K):
    sub = profile_readable[profile_readable["cluster"] == c].copy()
    sub["abs_diff"] = sub["diff_from_global"].abs()
    top_pos = sub.sort_values("diff_from_global", ascending=False).head(TOPN)
    top_neg = sub.sort_values("diff_from_global", ascending=True).head(TOPN)
    top_pos["direction"] = "over"
    top_neg["direction"] = "under"
    sig = pd.concat([top_pos, top_neg], ignore_index=True)
    # Add a 'family' tag for readability (attr_, cat__, rl_, city_, etc.)
    def family(f):
        if f.startswith("attr_"): return "Amenity"
        if f.startswith("cat__"): return "Cuisine"
        if f.startswith("rl_"): return "Review meta"
        if f.startswith("city_"): return "City"
        if f.startswith("sph") or f in {"latitude","longitude"}: return "Geo/Distance"
        if f.startswith("cluster_"): return "Cluster label"
        return "Numeric/Other"
    sig["base_feature"] = sig["feature"].apply(base_feature_name)
    sig["family"] = sig["base_feature"].apply(family)
    sig["cluster"] = c
    signals_rows.append(sig)
signals_readable = pd.concat(signals_rows, ignore_index=True)

signals_path = "unsup_outputs/cluster_top_signals_readable.csv"
signals_readable.to_csv(signals_path, index=False)
print("Saved:", signals_path)

# --- Post-hoc label (share of high rating) by cluster for the report
y_full = globals().get("y_full")
if y_full is None:
    y_tr = globals().get("y_train", globals().get("ytr"))
    y_te = globals().get("y_eval")
    if y_tr is not None and y_te is not None:
        y_full = np.concatenate([np.asarray(y_tr).ravel(), np.asarray(y_te).ravel()])
    else:
        y_full = None

summary_rows = []
for c in range(K):
    idx = (labels == c)
    n = int(idx.sum())
    frac = n / len(labels)
    row = {"cluster": c, "size": n, "share": frac}
    if y_full is not None:
        row["share_high_rating"] = float(np.mean(np.asarray(y_full)[idx]))
    summary_rows.append(row)
summary_df = pd.DataFrame(summary_rows)

# --- Write stakeholder-facing insights (Markdown)
md_lines = []
md_lines.append("# Unsupervised Segments — Summary (k=2)\n")
md_lines.append("**Method:** TruncatedSVD → MiniBatchKMeans (k=2). Features include amenities, cuisines, review meta, geo, counts. Ratings used **post-hoc** only for interpretation.\n")
md_lines.append("## Segment sizes\n")
for _, r in summary_df.iterrows():
    size = int(r["size"]); share = r["share"]
    if "share_high_rating" in r:
        md_lines.append(f"- **Cluster {int(r['cluster'])}** — n={size} ({share:.1%}) · share high-rating={r['share_high_rating']:.1%}")
    else:
        md_lines.append(f"- **Cluster {int(r['cluster'])}** — n={size} ({share:.1%})")

md_lines.append("\n## Characteristic signals (top ±5)\n")
for c in range(K):
    sub = signals_readable[signals_readable["cluster"]==c]
    pos5 = sub[sub["direction"]=="over"].sort_values("diff_from_global", ascending=False).head(5)
    neg5 = sub[sub["direction"]=="under"].sort_values("diff_from_global", ascending=True).head(5)
    md_lines.append(f"### Cluster {c}")
    md_lines.append("**Over-represented:** " + "; ".join([f"`{row['feature']}`" for _, row in pos5.iterrows()]) )
    md_lines.append("**Under-represented:** " + "; ".join([f"`{row['feature']}`" for _, row in neg5.iterrows()]) )
    # Optional: quick heuristic action line
    fams = pos5["family"].value_counts().to_dict()
    if fams:
        topfam = max(fams, key=fams.get)
        md_lines.append(f"_Action hint_: This segment leans on **{topfam}** features; tailor ops/marketing accordingly.\n")

insights_md = "unsup_outputs/unsup_insights.md"
with open(insights_md, "w") as f:
    f.write("\n".join(md_lines))
print("Saved:", insights_md)

# --- Small console preview
print("\n[STEP U3] Preview — cluster sizes")
print(summary_df.to_string(index=False))
print("\nTop readable signals — head():")
print(signals_readable.head(12).to_string(index=False))


In [ ]:
# ===== STEP U4A: IsolationForest anomalies on SVD space =====
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.ensemble import IsolationForest

os.makedirs("unsup_outputs", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# Reload artifacts
ct  = load("unsup_artifacts/ct_preproc.joblib")
svd = load("unsup_artifacts/svd.joblib")
km  = load("unsup_artifacts/kmeans_k2.joblib")
labels = pd.read_csv("unsup_artifacts/cluster_labels_k2.csv")["cluster"].to_numpy()

# Build X_full same way
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

# Latent space
X_feat = ct.transform(X_full)
Z = svd.transform(X_feat)

# Isolation Forest on latent Z
iso = IsolationForest(
    n_estimators=300, max_samples='auto', contamination=0.01,
    random_state=42, n_jobs=-1
)
iso.fit(Z)
raw_score = iso.score_samples(Z)              # higher = less anomalous
anom_score = -raw_score                       # higher = more anomalous
pred = iso.predict(Z)                         # -1 anomalous, 1 normal

# Assemble dataframe with ranks
df = pd.DataFrame({
    "row_id": np.arange(len(Z)),
    "cluster": labels,
    "anom_score": anom_score,
    "iso_pred": pred
})
df["rank_global"] = df["anom_score"].rank(ascending=False, method="first")
# within-cluster rank
df["rank_in_cluster"] = df.groupby("cluster")["anom_score"].rank(ascending=False, method="first")
# top 1% flags
top_n = max(1, int(0.01 * len(df)))
df["top1pct_global"] = df["rank_global"] <= top_n
df["top1pct_in_cluster"] = df.groupby("cluster")["rank_in_cluster"].transform(
    lambda r: r <= max(1, int(0.01 * len(r)))
).astype(bool)

out_csv = "unsup_outputs/anomalies.csv"
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)

# Plots
plt.figure(figsize=(6.5,4.2))
plt.hist(df["anom_score"], bins=50)
plt.title("Anomaly score distribution (higher = more anomalous)")
plt.xlabel("anom_score"); plt.ylabel("count")
plt.tight_layout(); plt.savefig("plots_unsup/anomaly_score_hist.png", dpi=160); plt.show()
print("Saved: plots_unsup/anomaly_score_hist.png")

plt.figure(figsize=(6.5,4.2))
data = [df.loc[df["cluster"]==c, "anom_score"] for c in sorted(df["cluster"].unique())]
plt.boxplot(data, labels=[str(c) for c in sorted(df["cluster"].unique())])
plt.title("Anomaly scores by cluster")
plt.xlabel("cluster"); plt.ylabel("anom_score")
plt.tight_layout(); plt.savefig("plots_unsup/anomaly_by_cluster_box.png", dpi=160); plt.show()
print("Saved: plots_unsup/anomaly_by_cluster_box.png")

# Console summary
summary = df.groupby("cluster").agg(
    n=("row_id","count"),
    n_top1pct_global=("top1pct_global","sum"),
    n_top1pct_in_cluster=("top1pct_in_cluster","sum")
).reset_index()
print("\n[STEP U4A] Anomaly summary")
print(summary.to_string(index=False))


In [ ]:
# STEP U4B — Generate an "Unsupervised Segment Playbook" PDF (retry, simplified)
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Image, ListFlowable, ListItem, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
import pandas as pd
import numpy as np
import os

# Paths
PLAYBOOK_PDF = "/mnt/data/unsupervised_segment_playbook.pdf"
signals_csv = "unsup_outputs/cluster_top_signals_readable.csv"
summary_csv = "unsup_outputs/cluster_summary.csv"
anoms_csv   = "unsup_outputs/anomalies.csv"
scatter_png = "plots_unsup/cluster_scatter_z12_k2.png"
avg_png     = "plots_unsup/cluster_avg_rating_k2.png"
anom_hist   = "plots_unsup/anomaly_score_hist.png"
anom_box    = "plots_unsup/anomaly_by_cluster_box.png"

# Load data (assume created in previous steps)
signals = pd.read_csv(signals_csv) if os.path.exists(signals_csv) else pd.DataFrame()
summary = pd.read_csv(summary_csv) if os.path.exists(summary_csv) else pd.DataFrame()
anoms   = pd.read_csv(anoms_csv) if os.path.exists(anoms_csv) else pd.DataFrame()

# Styles
styles = getSampleStyleSheet()
title = styles["Title"]
h1 = styles["Heading1"]
h2 = styles["Heading2"]
h3 = styles["Heading3"]
body = styles["BodyText"]
body.spaceAfter = 6

bullet = ParagraphStyle(
    'Bullet',
    parent=styles['BodyText'],
    leftIndent=14,
    bulletIndent=0,
    spaceBefore=2,
    spaceAfter=2
)

doc = SimpleDocTemplate(
    PLAYBOOK_PDF, pagesize=letter,
    leftMargin=0.7*inch, rightMargin=0.7*inch, topMargin=0.7*inch, bottomMargin=0.7*inch
)

story = []

# Cover
story.append(Paragraph("Unsupervised Segment Playbook — Yelp Ratings Predictor", title))
story.append(Spacer(1, 0.1*inch))
intro = "Method: TruncatedSVD → MiniBatchKMeans (k=2). Features include amenities, cuisines, review meta, geography, and counts. Ratings are used <b>post-hoc</b> only for interpretation."
story.append(Paragraph(intro, body))
story.append(Spacer(1, 0.15*inch))

# Figures (scatter + avg rating)
img_w = 6.2*inch
for pth, caption in [(scatter_png, "Clusters in SVD space (z1 vs z2)"),
                     (avg_png, "Post-hoc average high-rating share by cluster")]:
    if os.path.exists(pth):
        im = Image(pth, width=img_w, height=img_w*0.66)
        story.append(im)
        story.append(Paragraph(caption, styles["Italic"]))
        story.append(Spacer(1, 0.12*inch))

# Segment sizes table
if not summary.empty:
    story.append(Paragraph("Segment sizes", h2))
    tbl_df = summary.copy()
    for col in ["share", "share_high_rating"]:
        if col in tbl_df.columns:
            tbl_df[col] = (tbl_df[col]*100.0).map(lambda x: f"{x:.1f}%")
    if "cluster" in tbl_df.columns:
        tbl_df["cluster"] = tbl_df["cluster"].astype(int)
    data = [tbl_df.columns.tolist()] + tbl_df.values.tolist()
    table = Table(data, hAlign="LEFT")
    table.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ("GRID", (0,0), (-1,-1), 0.3, colors.grey),
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("ALIGN", (1,1), (-1,-1), "CENTER")
    ]))
    story.append(table)
    story.append(Spacer(1, 0.15*inch))

# Cluster profiles
if not signals.empty:
    story.append(Paragraph("Cluster profiles", h2))
    clusters = sorted(signals["cluster"].unique())
    for c in clusters:
        story.append(Paragraph(f"Cluster {int(c)}", h3))
        sub = signals[signals["cluster"]==c].copy()
        pos = sub[sub["direction"]=="over"].sort_values("diff_from_global", ascending=False).head(8)
        neg = sub[sub["direction"]=="under"].sort_values("diff_from_global", ascending=True).head(8)
        if not pos.empty:
            story.append(Paragraph("Over-represented signals:", body))
            items = [ListItem(Paragraph(f"{row['feature']}  <font size=9 color=grey>(Δ={row['diff_from_global']:.3f}; {row['family']})</font>", bullet)) for _, row in pos.iterrows()]
            story.append(ListFlowable(items, bulletType='bullet', start='•'))
        if not neg.empty:
            story.append(Paragraph("Under-represented signals:", body))
            items = [ListItem(Paragraph(f"{row['feature']}  <font size=9 color=grey>(Δ={row['diff_from_global']:.3f}; {row['family']})</font>", bullet)) for _, row in neg.iterrows()]
            story.append(ListFlowable(items, bulletType='bullet', start='•'))
        story.append(Spacer(1, 0.1*inch))
    story.append(PageBreak())

# Anomaly section
if not anoms.empty:
    story.append(Paragraph("Anomalies (IsolationForest on SVD space)", h2))
    summary_anom = anoms.groupby("cluster").agg(
        n=("row_id","count"),
        n_top1pct_global=("top1pct_global","sum"),
        n_top1pct_in_cluster=("top1pct_in_cluster","sum")
    ).reset_index()
    data = [summary_anom.columns.tolist()] + summary_anom.values.tolist()
    table = Table(data, hAlign="LEFT")
    table.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ("GRID", (0,0), (-1,-1), 0.3, colors.grey),
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("ALIGN", (1,1), (-1,-1), "CENTER")
    ]))
    story.append(table)
    story.append(Spacer(1, 0.12*inch))
    for pth, caption in [(anom_hist, "Anomaly score distribution (higher = more anomalous)"),
                         (anom_box,  "Anomaly scores by cluster")]:
        if os.path.exists(pth):
            im = Image(pth, width=img_w, height=img_w*0.62)
            story.append(im)
            story.append(Paragraph(caption, styles["Italic"]))
            story.append(Spacer(1, 0.12*inch))

# Actionable guidance
story.append(Paragraph("Actionable guidance (associational)", h2))
bullets = [
    "Use segment membership to tailor <b>listing hygiene</b>: highlight amenities over-represented in the segment; close gaps for under-represented but desirable amenities.",
    "Lean into <b>review operations</b>: prompt specific, balanced feedback; steady volume matters.",
    "Benchmark expectations by <b>cuisine/price band</b>; avoid over-interpreting geo effects.",
    "Treat outliers from the anomaly list as <b>case studies</b>—either risk (data errors) or opportunity (true standouts). Validate changes with small A/B tests."
]
story.append(ListFlowable([ListItem(Paragraph(b, bullet)) for b in bullets], bulletType='bullet', start='•'))

# Build PDF
doc.build(story)

PLAYBOOK_PDF


In [ ]:
# ===== STEP U5: Try alternative ks (k=3 and k=7) =====
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

os.makedirs("unsup_artifacts", exist_ok=True)
os.makedirs("unsup_outputs", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# --- Load preprocessing + SVD from U1
ct  = load("unsup_artifacts/ct_preproc.joblib")
svd = load("unsup_artifacts/svd.joblib")

# --- Build X_full exactly like U1/U2
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

# Transform to feature matrix -> latent space
X_feat = ct.transform(X_full)
Z = svd.transform(X_feat)

# Optional post-hoc labels for interpretation only
y_full = globals().get("y_full")
if y_full is None:
    y_tr = globals().get("y_train", globals().get("ytr"))
    y_te = globals().get("y_eval")
    if y_tr is not None and y_te is not None:
        y_full = np.concatenate([np.asarray(y_tr).ravel(), np.asarray(y_te).ravel()])
    else:
        y_full = None

# Reconstruct readable feature names (same approach as U3)
num_cols = ct.transformers_[0][2]
cat_cols = ct.transformers_[1][2]
ohe      = ct.named_transformers_["cat"].named_steps["ohe"]
cat_names = ohe.get_feature_names_out(cat_cols)
feat_names = np.array(list(num_cols) + list(cat_names))

def base_feature_name(f):
    # Collapse OHE: "attr_HasTV_False" -> "attr_HasTV"
    if f in X_full.columns:
        return f
    parts = f.split("_")
    return "_".join(parts[:-1]) if len(parts) > 1 else f

def family(f):
    if f.startswith("attr_"): return "Amenity"
    if f.startswith("cat__"): return "Cuisine"
    if f.startswith("rl_"): return "Review meta"
    if f.startswith("city_"): return "City"
    if f.startswith("sph") or f in {"latitude","longitude"}: return "Geo/Distance"
    if f.startswith("cluster_"): return "Cluster label"
    return "Numeric/Other"

def summarize_for_k(K):
    km = MiniBatchKMeans(n_clusters=K, random_state=42, n_init=10, batch_size=2048, max_iter=300)
    km.fit(Z)
    labels = km.labels_
    dump(km, f"unsup_artifacts/kmeans_k{K}.joblib")
    pd.DataFrame({"row_id": np.arange(len(labels)), "cluster": labels}).to_csv(
        f"unsup_artifacts/cluster_labels_k{K}.csv", index=False
    )
    # Silhouette on a sample for speed
    rng = np.random.default_rng(42)
    idx_sample = rng.choice(len(Z), size=min(6000, len(Z)), replace=False)
    sil = float(silhouette_score(Z[idx_sample], km.predict(Z[idx_sample])))

    # Global & per-cluster means in preprocessed space
    if hasattr(X_feat, "toarray"):
        global_mean = np.asarray(X_feat.mean(axis=0)).ravel()
    else:
        global_mean = X_feat.mean(axis=0)

    cluster_means = []
    for c in range(K):
        Xc = X_feat[labels == c]
        mc = np.asarray(Xc.mean(axis=0)).ravel() if hasattr(Xc, "toarray") else Xc.mean(axis=0)
        cluster_means.append(mc)
    cluster_means = np.vstack(cluster_means)

    # Tidy profile
    rows = []
    for c in range(K):
        prof = pd.DataFrame({
            "cluster": c,
            "feature": feat_names,
            "mean_in_cluster": cluster_means[c],
            "mean_global": global_mean
        })
        prof["diff_from_global"] = prof["mean_in_cluster"] - prof["mean_global"]
        prof["base_feature"] = prof["feature"].apply(base_feature_name)
        prof["family"] = prof["base_feature"].apply(family)
        rows.append(prof)
    profile = pd.concat(rows, ignore_index=True)
    profile.to_csv(f"unsup_outputs/cluster_profile_means_readable_k{K}.csv", index=False)

    # Top signals per cluster
    sig_rows = []
    TOPN = 10
    for c in range(K):
        sub = profile[profile["cluster"]==c].copy()
        top_pos = sub.sort_values("diff_from_global", ascending=False).head(TOPN)
        top_neg = sub.sort_values("diff_from_global", ascending=True).head(TOPN)
        top_pos["direction"] = "over"; top_neg["direction"] = "under"
        sig_rows.append(pd.concat([top_pos, top_neg], ignore_index=True))
    signals = pd.concat(sig_rows, ignore_index=True)
    signals.to_csv(f"unsup_outputs/cluster_top_signals_readable_k{K}.csv", index=False)

    # Summary (size/share/+ post-hoc rating share)
    summ_rows = []
    for c in range(K):
        idx = (labels == c)
        row = {"cluster": c, "size": int(idx.sum()), "share": float(idx.mean())}
        if y_full is not None:
            row["share_high_rating"] = float(np.mean(np.asarray(y_full)[idx]))
        summ_rows.append(row)
    summary_df = pd.DataFrame(summ_rows).sort_values("cluster")
    summary_df.to_csv(f"unsup_outputs/cluster_summary_k{K}.csv", index=False)

    # Plots
    plt.figure(figsize=(7.2,5.8))
    # light scatter of z1 vs z2
    for c in range(K):
        m = idx_sample[labels[idx_sample] == c]
        plt.scatter(Z[m,0], Z[m,1], s=8, alpha=0.6, label=f"Cluster {c}")
    plt.title(f"Clusters in SVD space (k={K}) — z1 vs z2")
    plt.xlabel("z1"); plt.ylabel("z2"); plt.legend(markerscale=2)
    plt.tight_layout(); plt.savefig(f"plots_unsup/cluster_scatter_z12_k{K}.png", dpi=160); plt.show()
    print("Saved:", f"plots_unsup/cluster_scatter_z12_k{K}.png")

    if y_full is not None and "share_high_rating" in summary_df.columns:
        plt.figure(figsize=(6.5,4.2))
        plt.bar(summary_df["cluster"].astype(str), summary_df["share_high_rating"])
        plt.xlabel("Cluster"); plt.ylabel("Avg. high-rating share (post-hoc)")
        plt.title(f"Avg. label by cluster (k={K})")
        plt.tight_layout(); plt.savefig(f"plots_unsup/cluster_avg_rating_k{K}.png", dpi=160); plt.show()
        print("Saved:", f"plots_unsup/cluster_avg_rating_k{K}.png")

    # Console summary
    print(f"\n[STEP U5] Summary for k={K}")
    print(f"Silhouette: {sil:.3f}")
    print(summary_df.to_string(index=False))

# Run for k=3 and k=7
for K in (3, 7):
    summarize_for_k(K)


In [ ]:
# ===== STEP U6: Summaries for k=3 main + k=7 spotlight =====
import os, pandas as pd, zipfile

def need(path):
    assert os.path.exists(path), f"Missing file: {path}. Did STEP U5 complete?"
    return path

os.makedirs("unsup_outputs", exist_ok=True)

# --- Load STEP U5 outputs
summ_k3 = pd.read_csv(need("unsup_outputs/cluster_summary_k3.csv"))
sig_k3  = pd.read_csv(need("unsup_outputs/cluster_top_signals_readable_k3.csv"))
summ_k7 = pd.read_csv(need("unsup_outputs/cluster_summary_k7.csv"))
sig_k7  = pd.read_csv(need("unsup_outputs/cluster_top_signals_readable_k7.csv"))

def pct(x): 
    try: return f"{100*float(x):.1f}%"
    except: return str(x)

# --- k=3 main summary (Markdown)
lines = []
lines.append("# Unsupervised Segments — Main (k=3)\n")
lines.append("**Method:** TruncatedSVD → MiniBatchKMeans (k=3). Ratings used **post-hoc** only for interpretation.\n")

tbl = summ_k3.copy()
if "share" in tbl: tbl["share"] = tbl["share"].map(pct)
if "share_high_rating" in tbl: tbl["share_high_rating"] = tbl["share_high_rating"].map(pct)

lines.append("## Segment sizes & post-hoc high-rating share")
lines.append(tbl.to_markdown(index=False))
lines.append("")

lines.append("## Cluster profiles — characteristic signals (top ±5)")
for c in sorted(sig_k3["cluster"].unique()):
    sub = sig_k3[sig_k3["cluster"]==c]
    pos5 = sub[sub["direction"]=="over"].sort_values("diff_from_global", ascending=False).head(5)
    neg5 = sub[sub["direction"]=="under"].sort_values("diff_from_global", ascending=True).head(5)
    lines.append(f"### Cluster {int(c)}")
    if not pos5.empty:
        lines.append("**Over-represented:** " + 
                     "; ".join(f"`{r.feature}` (Δ={r.diff_from_global:.3f}, {r.family})" for _, r in pos5.iterrows()))
    if not neg5.empty:
        lines.append("**Under-represented:** " + 
                     "; ".join(f"`{r.feature}` (Δ={r.diff_from_global:.3f}, {r.family})" for _, r in neg5.iterrows()))
    lines.append("")

# Include plots if available
for p, cap in [("plots_unsup/cluster_scatter_z12_k3.png","Clusters in SVD space (k=3) — z1 vs z2"),
               ("plots_unsup/cluster_avg_rating_k3.png","Post-hoc average high-rating share (k=3)")]:
    if os.path.exists(p):
        lines.append(f"**{cap}**")
        lines.append(f"![{cap}]({p})")
        lines.append("")

md_k3 = "unsup_outputs/unsup_summary_k3.md"
with open(md_k3, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))
print("Saved:", md_k3)

# --- k=7 spotlight (highest post-hoc high-rating cluster)
best = summ_k7.loc[summ_k7["share_high_rating"].idxmax()]
cstar = int(best["cluster"])
spot = []
spot.append("# Segment spotlight — Premium cohort (k=7)\n")
spot.append(f"**Cluster {cstar}** — size={int(best['size'])} (~{pct(best['share'])}), "
            f"post-hoc high-rating share ≈ **{pct(best['share_high_rating'])}**.\n")

sub7 = sig_k7[sig_k7["cluster"]==cstar]
pos8 = sub7[sub7["direction"]=="over"].sort_values("diff_from_global", ascending=False).head(8)
neg8 = sub7[sub7["direction"]=="under"].sort_values("diff_from_global", ascending=True).head(8)

spot.append("## Characteristic signals")
spot.append("**Over-represented:** " + 
            "; ".join(f"`{r.feature}` (Δ={r.diff_from_global:.3f}, {r.family})" for _, r in pos8.iterrows()))
spot.append("**Under-represented:** " + 
            "; ".join(f"`{r.feature}` (Δ={r.diff_from_global:.3f}, {r.family})" for _, r in neg8.iterrows()))
spot.append("")

for p, cap in [("plots_unsup/cluster_scatter_z12_k7.png","Clusters in SVD space (k=7) — z1 vs z2"),
               ("plots_unsup/cluster_avg_rating_k7.png","Post-hoc average high-rating share (k=7)")]:
    if os.path.exists(p):
        spot.append(f"**{cap}**")
        spot.append(f"![{cap}]({p})")
        spot.append("")

md_k7 = "unsup_outputs/unsup_spotlight_k7.md"
with open(md_k7, "w", encoding="utf-8") as f:
    f.write("\n".join(spot))
print("Saved:", md_k7)

# --- Zip addendum with markdowns + CSVs + plots
zip_path = "unsup_addendum_k3_k7.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in [
        md_k3, md_k7,
        "unsup_outputs/cluster_summary_k3.csv",
        "unsup_outputs/cluster_top_signals_readable_k3.csv",
        "unsup_outputs/cluster_profile_means_readable_k3.csv",
        "unsup_outputs/cluster_summary_k7.csv",
        "unsup_outputs/cluster_top_signals_readable_k7.csv",
        "unsup_outputs/cluster_profile_means_readable_k7.csv",
        "unsup_outputs/anomalies.csv",
        "plots_unsup/cluster_scatter_z12_k3.png",
        "plots_unsup/cluster_avg_rating_k3.png",
        "plots_unsup/cluster_scatter_z12_k7.png",
        "plots_unsup/cluster_avg_rating_k7.png",
        "plots_unsup/anomaly_score_hist.png",
        "plots_unsup/anomaly_by_cluster_box.png",
    ]:
        if os.path.exists(p):
            z.write(p, arcname=os.path.basename(p))
print("Saved:", zip_path)


In [ ]:
# ===== STEP U7a: k-Prototypes hybrid clustering (K=3) =====
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from joblib import load, dump

# 1) Try to import k-Prototypes
try:
    from kmodes.kprototypes import KPrototypes
except Exception as e:
    raise SystemExit(
        "kmodes is not installed. Run:\n  pip install kmodes\nthen re-run STEP U7a.\n"
        f"Import error: {e}"
    )

os.makedirs("unsup_kproto", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# 2) Load artifacts to recover schema + latent space for plotting
ct  = load("unsup_artifacts/ct_preproc.joblib")
svd = load("unsup_artifacts/svd.joblib")

# Build X_full exactly as in earlier steps
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

# Post-hoc label for interpretation only
y_full = globals().get("y_full")
if y_full is None:
    y_tr = globals().get("y_train", globals().get("ytr"))
    y_te = globals().get("y_eval")
    y_full = (np.concatenate([np.asarray(y_tr).ravel(), np.asarray(y_te).ravel()])
              if y_tr is not None and y_te is not None else None)

# 3) Recover numeric/categorical column lists from your ColumnTransformer
num_cols = list(ct.transformers_[0][2])
cat_cols = list(ct.transformers_[1][2])
assert len(num_cols) + len(cat_cols) > 0, "Could not recover numeric/categorical columns from ct."

# 4) Prepare matrices for k-Prototypes
# - Numerics: coerce -> float, impute median, standardize
# - Categoricals: to string categories, fillna("missing")
num_df = X_full[num_cols].copy()
for c in num_cols:
    num_df[c] = pd.to_numeric(num_df[c], errors="coerce")
num_df = num_df.fillna(num_df.median(numeric_only=True))

# standardize
num_arr = (num_df - num_df.mean()) / (num_df.std(ddof=0) + 1e-9)
num_arr = num_arr.to_numpy()

cat_df = X_full[cat_cols].copy()
for c in cat_cols:
    cat_df[c] = cat_df[c].astype("string").fillna("missing")
cat_arr = cat_df.to_numpy()

# Concatenate numerics + categoricals into a single object array as expected by k-Prototypes
X_kp = np.concatenate([num_arr, cat_arr], axis=1).astype(object)
cat_idx = list(range(num_arr.shape[1], num_arr.shape[1] + cat_arr.shape[1]))  # positions of categorical columns

# 5) Fit k-Prototypes (K=3)
K = 3
kproto = KPrototypes(n_jobs=-1, n_clusters=K, init='Cao', n_init=5, max_iter=50, random_state=42)
labels = kproto.fit_predict(X_kp, categorical=cat_idx)

# Save artifacts
dump(kproto, f"unsup_kproto/kproto_k{K}.joblib")
pd.DataFrame({"row_id": np.arange(len(labels)), "cluster": labels}).to_csv(
    f"unsup_kproto/labels_k{K}.csv", index=False
)

# 6) Summarize: cluster sizes + (optional) post-hoc high-rating share
summ_rows = []
for c in range(K):
    idx = (labels == c)
    row = {"cluster": int(c), "size": int(idx.sum()), "share": float(idx.mean())}
    if y_full is not None:
        row["share_high_rating"] = float(np.mean(np.asarray(y_full)[idx]))
    summ_rows.append(row)
summary_df = pd.DataFrame(summ_rows).sort_values("cluster")
summary_df.to_csv(f"unsup_kproto/summary_k{K}.csv", index=False)

# 7) Categorical "top signals": for each categorical column, which levels are most over/under represented?
sig_rows = []
for c in range(K):
    idx = (labels == c)
    sub = cat_df[idx]
    for col in cat_cols:
        # frequency in cluster vs global
        freq_c = sub[col].value_counts(normalize=True, dropna=False)
        freq_g = cat_df[col].value_counts(normalize=True, dropna=False)
        # align
        all_levels = freq_g.index.union(freq_c.index)
        diff = (freq_c.reindex(all_levels).fillna(0) - freq_g.reindex(all_levels).fillna(0)).sort_values(ascending=False)
        # take top +/- 5
        top_over  = diff.head(5)
        top_under = diff.tail(5)
        for lvl, dv in top_over.items():
            sig_rows.append({"cluster": c, "column": col, "level": str(lvl), "direction": "over", "diff": float(dv)})
        for lvl, dv in top_under.items():
            sig_rows.append({"cluster": c, "column": col, "level": str(lvl), "direction": "under", "diff": float(dv)})
signals_df = pd.DataFrame(sig_rows)
signals_df.to_csv(f"unsup_kproto/top_signals_k{K}.csv", index=False)

# 8) Quick visuals: project to SVD space (z1,z2) and color by k-Prototypes labels
X_feat = ct.transform(X_full)          # same preproc as before
Z = svd.transform(X_feat)              # latent for plotting
plt.figure(figsize=(7.2,5.8))
rng = np.random.default_rng(42)
sample_idx = rng.choice(len(Z), size=min(6000, len(Z)), replace=False)
for c in range(K):
    m = sample_idx[labels[sample_idx] == c]
    plt.scatter(Z[m,0], Z[m,1], s=8, alpha=0.6, label=f"Cluster {c}")
plt.title(f"k-Prototypes clusters in SVD space (k={K})")
plt.xlabel("z1"); plt.ylabel("z2"); plt.legend(markerscale=2)
plt.tight_layout(); plt.savefig(f"plots_unsup/kproto_scatter_k{K}.png", dpi=160); plt.show()
print("Saved:", f"plots_unsup/kproto_scatter_k{K}.png")

if y_full is not None and "share_high_rating" in summary_df.columns:
    plt.figure(figsize=(6.2,4.2))
    plt.bar(summary_df["cluster"].astype(str), summary_df["share_high_rating"])
    plt.xlabel("Cluster"); plt.ylabel("Post-hoc high-rating share")
    plt.title(f"k-Prototypes — avg label by cluster (k={K})")
    plt.tight_layout(); plt.savefig(f"plots_unsup/kproto_avg_rating_k{K}.png", dpi=160); plt.show()
    print("Saved:", f"plots_unsup/kproto_avg_rating_k{K}.png")

# 9) Console summary
print("\n[STEP U7a] k-Prototypes summary (k=3)")
print(summary_df.to_string(index=False))
print("\nTop categorical signals — head():")
print(signals_df.head(12).to_string(index=False))


In [ ]:
# ===== STEP U7b: k-Prototypes (K=7) + side-by-side comparison =====
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.metrics import silhouette_score

# k-Prototypes
try:
    from kmodes.kprototypes import KPrototypes
except Exception as e:
    raise SystemExit("Please install kmodes first: pip install kmodes\n" + str(e))

os.makedirs("unsup_kproto", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# --- Load preprocessing+SVD (from U1) and rebuild X_full
ct  = load("unsup_artifacts/ct_preproc.joblib")
svd = load("unsup_artifacts/svd.joblib")

X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

# Post-hoc label for interpretation only
y_full = globals().get("y_full")
if y_full is None:
    y_tr = globals().get("y_train", globals().get("ytr"))
    y_te = globals().get("y_eval")
    y_full = (np.concatenate([np.asarray(y_tr).ravel(), np.asarray(y_te).ravel()])
              if y_tr is not None and y_te is not None else None)

# --- Recover numeric/categorical columns from ct
num_cols = list(ct.transformers_[0][2])
cat_cols = list(ct.transformers_[1][2])

# Prepare k-Prototypes matrix (same as U7a)
num_df = X_full[num_cols].copy()
for c in num_cols:
    num_df[c] = pd.to_numeric(num_df[c], errors="coerce")
num_df = num_df.fillna(num_df.median(numeric_only=True))
num_arr = (num_df - num_df.mean()) / (num_df.std(ddof=0) + 1e-9)
num_arr = num_arr.to_numpy()

cat_df = X_full[cat_cols].copy()
for c in cat_cols:
    cat_df[c] = cat_df[c].astype("string").fillna("missing")
cat_arr = cat_df.to_numpy()

X_kp = np.concatenate([num_arr, cat_arr], axis=1).astype(object)
cat_idx = list(range(num_arr.shape[1], num_arr.shape[1] + cat_arr.shape[1]))

# --- Fit k-Prototypes K=7
K = 7
kproto7 = KPrototypes(n_clusters=K, init='Cao', n_init=5, max_iter=50, random_state=42, n_jobs=-1)
labels7 = kproto7.fit_predict(X_kp, categorical=cat_idx)

dump(kproto7, f"unsup_kproto/kproto_k{K}.joblib")
pd.DataFrame({"row_id": np.arange(len(labels7)), "cluster": labels7}).to_csv(
    f"unsup_kproto/labels_k{K}.csv", index=False
)

# --- Summary for K=7
summ_rows = []
for c in range(K):
    idx = (labels7 == c)
    row = {"cluster": int(c), "size": int(idx.sum()), "share": float(idx.mean())}
    if y_full is not None:
        row["share_high_rating"] = float(np.mean(np.asarray(y_full)[idx]))
    summ_rows.append(row)
summary7 = pd.DataFrame(summ_rows).sort_values("cluster")
summary7.to_csv(f"unsup_kproto/summary_k{K}.csv", index=False)

# --- Top categorical signals for K=7
sig_rows = []
for c in range(K):
    idx = (labels7 == c)
    sub = cat_df[idx]
    for col in cat_cols:
        freq_c = sub[col].value_counts(normalize=True, dropna=False)
        freq_g = cat_df[col].value_counts(normalize=True, dropna=False)
        all_levels = freq_g.index.union(freq_c.index)
        diff = (freq_c.reindex(all_levels).fillna(0) - freq_g.reindex(all_levels).fillna(0)).sort_values(ascending=False)
        for lvl, dv in diff.head(5).items():
            sig_rows.append({"cluster": c, "column": col, "level": str(lvl), "direction": "over", "diff": float(dv)})
        for lvl, dv in diff.tail(5).items():
            sig_rows.append({"cluster": c, "column": col, "level": str(lvl), "direction": "under", "diff": float(dv)})
signals7 = pd.DataFrame(sig_rows)
signals7.to_csv(f"unsup_kproto/top_signals_k{K}.csv", index=False)

# --- Plots (project to SVD space for visualization)
X_feat = ct.transform(X_full)
Z = svd.transform(X_feat)

import numpy.random as npr
rng = npr.default_rng(42)
sample_idx = rng.choice(len(Z), size=min(6000, len(Z)), replace=False)

plt.figure(figsize=(7.2,5.8))
for c in range(K):
    m = sample_idx[labels7[sample_idx] == c]
    plt.scatter(Z[m,0], Z[m,1], s=8, alpha=0.6, label=f"Cluster {c}")
plt.title(f"k-Prototypes clusters in SVD space (k={K})")
plt.xlabel("z1"); plt.ylabel("z2"); plt.legend(markerscale=2)
plt.tight_layout(); plt.savefig(f"plots_unsup/kproto_scatter_k{K}.png", dpi=160); plt.show()
print("Saved:", f"plots_unsup/kproto_scatter_k{K}.png")

if y_full is not None and "share_high_rating" in summary7.columns:
    plt.figure(figsize=(6.2,4.2))
    plt.bar(summary7["cluster"].astype(str), summary7["share_high_rating"])
    plt.xlabel("Cluster"); plt.ylabel("Post-hoc high-rating share")
    plt.title(f"k-Prototypes — avg label by cluster (k={K})")
    plt.tight_layout(); plt.savefig(f"plots_unsup/kproto_avg_rating_k{K}.png", dpi=160); plt.show()
    print("Saved:", f"plots_unsup/kproto_avg_rating_k{K}.png")

print("\n[STEP U7b] k-Prototypes summary (k=7)")
print(summary7.to_string(index=False))

# --- Side-by-side comparison table on the same SVD space
rows = []

# k-means labels from earlier (U5)
def sil_on_Z(labels_any):
    # compute silhouette on Z using a sample for speed & consistency
    lab_s = labels_any[sample_idx]
    return float(silhouette_score(Z[sample_idx], lab_s))

if os.path.exists("unsup_artifacts/cluster_labels_k3.csv"):
    km3 = pd.read_csv("unsup_artifacts/cluster_labels_k3.csv")["cluster"].to_numpy()
    sil_km3 = sil_on_Z(km3)
    sk3 = pd.read_csv("unsup_outputs/cluster_summary_k3.csv")
    rows.append({"method":"kmeans","k":3,
                 "silhouette_on_SVD": sil_km3,
                 "max_posthoc_high_share": float(sk3["share_high_rating"].max())})

if os.path.exists("unsup_artifacts/cluster_labels_k7.csv"):
    km7 = pd.read_csv("unsup_artifacts/cluster_labels_k7.csv")["cluster"].to_numpy()
    sil_km7 = sil_on_Z(km7)
    sk7 = pd.read_csv("unsup_outputs/cluster_summary_k7.csv")
    rows.append({"method":"kmeans","k":7,
                 "silhouette_on_SVD": sil_km7,
                 "max_posthoc_high_share": float(sk7["share_high_rating"].max())})

# k-prototypes k=3 (from U7a)
if os.path.exists("unsup_kproto/labels_k3.csv"):
    kp3 = pd.read_csv("unsup_kproto/labels_k3.csv")["cluster"].to_numpy()
    sil_kp3 = sil_on_Z(kp3)
    sp3 = pd.read_csv("unsup_kproto/summary_k3.csv")
    rows.append({"method":"kprototypes","k":3,
                 "silhouette_on_SVD": sil_kp3,
                 "max_posthoc_high_share": float(sp3["share_high_rating"].max())})

# k-prototypes k=7 (current)
sil_kp7 = sil_on_Z(labels7)
rows.append({"method":"kprototypes","k":7,
             "silhouette_on_SVD": sil_kp7,
             "max_posthoc_high_share": float(summary7["share_high_rating"].max())})

cmp_df = pd.DataFrame(rows).sort_values(["method","k"]).reset_index(drop=True)
cmp_path = "unsup_kproto/segmentation_comparison.csv"
cmp_df.to_csv(cmp_path, index=False)

print("\n[STEP U7b] Side-by-side segmentation comparison (silhouette on SVD)")
print(cmp_df.to_string(index=False))
print("Saved:", cmp_path)


In [ ]:
# ===== STEP U7c: Finalize unsupervised choice (k-Prototypes K=3) and export =====
import os, pandas as pd, numpy as np, matplotlib.pyplot as plt
from joblib import load

os.makedirs("unsup_outputs", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# 1) Load comparison + summaries
cmp_path   = "unsup_kproto/segmentation_comparison.csv"
summ_kp3   = "unsup_kproto/summary_k3.csv"
sig_cat_kp3= "unsup_kproto/top_signals_k3.csv"
labels_kp3 = "unsup_kproto/labels_k3.csv"

cmp_df   = pd.read_csv(cmp_path)
summ_df  = pd.read_csv(summ_kp3)
sig_cat  = pd.read_csv(sig_cat_kp3)
labels   = pd.read_csv(labels_kp3)

# 2) Make a small silhouette comparison bar chart
plt.figure(figsize=(6.5, 4.2))
show = cmp_df.copy()
show["label"] = show["method"] + " k=" + show["k"].astype(str)
plt.bar(show["label"], show["silhouette_on_SVD"])
plt.ylabel("Silhouette (on SVD space)")
plt.title("Segmentation quality comparison")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig("plots_unsup/segmentation_silhouette_comparison.png", dpi=160)
plt.show()
print("Saved: plots_unsup/segmentation_silhouette_comparison.png")

# 3) Compute NUMERIC top signals for k-Prototypes K=3 (over/under vs global)
#    (Complements categorical 'top_signals_k3.csv')
ct = load("unsup_artifacts/ct_preproc.joblib")
num_cols = list(ct.transformers_[0][2])  # numeric raw columns
# Rebuild X_full
X_full = globals().get("X_full")
if X_full is None:
    Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
    Xte_ = globals().get("Xte")
    assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
    cols_union = Xtr_.columns.union(Xte_.columns)
    X_full = pd.concat([
        Xtr_.reindex(columns=cols_union, fill_value=0),
        Xte_.reindex(columns=cols_union, fill_value=0)
    ], axis=0, ignore_index=True)

num_df = X_full[num_cols].copy()
for c in num_cols:
    num_df[c] = pd.to_numeric(num_df[c], errors="coerce")
num_df = num_df.fillna(num_df.median(numeric_only=True))

labs = labels["cluster"].to_numpy()
global_mean = num_df.mean(axis=0)
rows = []
TOP = 10
for c in sorted(np.unique(labs)):
    sub = num_df[labs == c]
    mean_c = sub.mean(axis=0)
    diff = (mean_c - global_mean).sort_values(ascending=False)
    top_over  = diff.head(TOP)
    top_under = diff.tail(TOP)
    for feat, dv in top_over.items():
        rows.append({"cluster": int(c), "feature": feat, "direction": "over", "diff_from_global": float(dv)})
    for feat, dv in top_under.items():
        rows.append({"cluster": int(c), "feature": feat, "direction": "under", "diff_from_global": float(dv)})
num_sig = pd.DataFrame(rows)
num_sig.to_csv("unsup_kproto/top_numeric_signals_k3.csv", index=False)
print("Saved: unsup_kproto/top_numeric_signals_k3.csv")

# 4) Final selection write-up (Markdown)
def pct(x): 
    try: return f"{100*float(x):.1f}%"
    except: return str(x)

choice_md = []
choice_md.append("# Final Unsupervised Selection\n")
choice_md.append("**Chosen segmentation:** k-Prototypes (hybrid k-means/k-modes), **K=3**.\n")
choice_md.append("**Why:** best separation on the same geometry (silhouette on SVD ≈ highest), clean/ interpretable segments, and categorical signals appear naturally.\n")
choice_md.append("**Also included:** k-means **K=7** premium cohort (max post-hoc high-rating share) as an appendix spotlight.\n")

# Quick tables
choice_md.append("## Quality comparison (silhouette on SVD)")
choice_md.append(cmp_df.to_markdown(index=False))

tmp = summ_df.copy()
if "share" in tmp: tmp["share"] = tmp["share"].map(pct)
if "share_high_rating" in tmp: tmp["share_high_rating"] = tmp["share_high_rating"].map(pct)
choice_md.append("\n## k-Prototypes K=3 — segment sizes")
choice_md.append(tmp.to_markdown(index=False))

# Cat signals preview
choice_md.append("\n## k-Prototypes K=3 — sample categorical signals")
for c in sorted(sig_cat["cluster"].unique()):
    sub = sig_cat[sig_cat["cluster"]==c]
    over = sub[sub["direction"]=="over"].sort_values("diff", ascending=False).head(5)
    under= sub[sub["direction"]=="under"].sort_values("diff", ascending=True).head(5)
    choice_md.append(f"### Cluster {int(c)}")
    choice_md.append("**Over-represented:** " + "; ".join(f"`{r.column} = {r.level}` (Δ{r.diff:+.3f})" for _, r in over.iterrows()))
    choice_md.append("**Under-represented:** " + "; ".join(f"`{r.column} = {r.level}` (Δ{r.diff:+.3f})" for _, r in under.iterrows()))
    choice_md.append("")

with open("unsup_outputs/unsup_final_choice.md", "w", encoding="utf-8") as f:
    f.write("\n".join(choice_md))
print("Saved: unsup_outputs/unsup_final_choice.md")

# 5) Zip addendum (final)
import zipfile
zip_path = "unsup_addendum_final_segments.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in [
        "unsup_outputs/unsup_final_choice.md",
        "unsup_outputs/unsup_summary_k3.md",
        "unsup_outputs/unsup_spotlight_k7.md",
        "unsup_kproto/summary_k3.csv",
        "unsup_kproto/top_signals_k3.csv",
        "unsup_kproto/top_numeric_signals_k3.csv",
        "unsup_kproto/summary_k7.csv",
        "unsup_kproto/top_signals_k7.csv",
        "unsup_kproto/segmentation_comparison.csv",
        "unsup_kproto/labels_k3.csv",
        "plots_unsup/segmentation_silhouette_comparison.png",
        "plots_unsup/kproto_scatter_k3.png",
        "plots_unsup/kproto_avg_rating_k3.png",
        "plots_unsup/cluster_scatter_z12_k3.png",
        "plots_unsup/cluster_avg_rating_k3.png",
        "plots_unsup/kproto_scatter_k7.png",
        "plots_unsup/kproto_avg_rating_k7.png",
        "plots_unsup/cluster_scatter_z12_k7.png",
        "plots_unsup/cluster_avg_rating_k7.png",
    ]:
        if os.path.exists(p): z.write(p, arcname=os.path.basename(p))
print("Saved:", zip_path)


In [ ]:
# === STEP U7c (fix): finalize unsupervised choice & export ===
import os, zipfile, pandas as pd, numpy as np
import matplotlib.pyplot as plt
from joblib import load

os.makedirs("unsup_outputs", exist_ok=True)
os.makedirs("plots_unsup", exist_ok=True)

# 1) Load comparison + summaries generated earlier
cmp_df   = pd.read_csv("unsup_kproto/segmentation_comparison.csv")
summ_df  = pd.read_csv("unsup_kproto/summary_k3.csv")
sig_cat  = pd.read_csv("unsup_kproto/top_signals_k3.csv")
labels   = pd.read_csv("unsup_kproto/labels_k3.csv")

# 2) (Re)make silhouette comparison plot (idempotent)
plt.figure(figsize=(6.5, 4.2))
show = cmp_df.copy()
show["label"] = show["method"] + " k=" + show["k"].astype(str)
plt.bar(show["label"], show["silhouette_on_SVD"])
plt.ylabel("Silhouette (on SVD space)")
plt.title("Segmentation quality comparison")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig("plots_unsup/segmentation_silhouette_comparison.png", dpi=160)
plt.close()
print("Saved: plots_unsup/segmentation_silhouette_comparison.png")

# 3) Compute numeric top signals (if not already there)
if not os.path.exists("unsup_kproto/top_numeric_signals_k3.csv"):
    ct = load("unsup_artifacts/ct_preproc.joblib")
    num_cols = list(ct.transformers_[0][2])
    # rebuild X_full from train/eval
    X_full = globals().get("X_full")
    if X_full is None:
        Xtr_ = globals().get("Xtr", globals().get("Xtr2"))
        Xte_ = globals().get("Xte")
        assert Xtr_ is not None and Xte_ is not None, "Need X_full or (Xtr & Xte)."
        cols_union = Xtr_.columns.union(Xte_.columns)
        X_full = pd.concat([
            Xtr_.reindex(columns=cols_union, fill_value=0),
            Xte_.reindex(columns=cols_union, fill_value=0)
        ], axis=0, ignore_index=True)

    num_df = X_full[num_cols].copy()
    for c in num_cols:
        num_df[c] = pd.to_numeric(num_df[c], errors="coerce")
    num_df = num_df.fillna(num_df.median(numeric_only=True))

    labs = labels["cluster"].to_numpy()
    global_mean = num_df.mean(axis=0)
    rows = []
    TOP = 10
    for c in sorted(np.unique(labs)):
        sub = num_df[labs == c]
        diff = (sub.mean(axis=0) - global_mean).sort_values(ascending=False)
        for feat, dv in diff.head(TOP).items():
            rows.append({"cluster": int(c), "feature": feat, "direction": "over",  "diff_from_global": float(dv)})
        for feat, dv in diff.tail(TOP).items():
            rows.append({"cluster": int(c), "feature": feat, "direction": "under","diff_from_global": float(dv)})
    pd.DataFrame(rows).to_csv("unsup_kproto/top_numeric_signals_k3.csv", index=False)
    print("Saved: unsup_kproto/top_numeric_signals_k3.csv")

# 4) Write the final selection markdown (FIX: use row['diff'], not row.diff)
def pct(x):
    try: return f"{100*float(x):.1f}%"
    except Exception: return str(x)

choice_md = []
choice_md.append("# Final Unsupervised Selection\n")
choice_md.append("**Chosen segmentation:** k-Prototypes (hybrid k-means/k-modes), **K=3**.\n")
choice_md.append("**Why:** best separation on the same geometry (highest silhouette on SVD), clean/ interpretable segments, and categorical signals appear naturally.\n")
choice_md.append("**Also included:** k-means **K=7** premium cohort (max post-hoc high-rating share) as an appendix spotlight.\n")

choice_md.append("## Quality comparison (silhouette on SVD)")
choice_md.append(cmp_df.to_markdown(index=False))

tbl = summ_df.copy()
if "share" in tbl: tbl["share"] = tbl["share"].map(pct)
if "share_high_rating" in tbl: tbl["share_high_rating"] = tbl["share_high_rating"].map(pct)
choice_md.append("\n## k-Prototypes K=3 — segment sizes")
choice_md.append(tbl.to_markdown(index=False))

choice_md.append("\n## k-Prototypes K=3 — sample categorical signals")
for c in sorted(sig_cat["cluster"].unique()):
    sub = sig_cat[sig_cat["cluster"]==c]
    over = sub[sub["direction"]=="over"].sort_values("diff", ascending=False).head(5)
    under= sub[sub["direction"]=="under"].sort_values("diff", ascending=True).head(5)
    choice_md.append(f"### Cluster {int(c)}")
    if not over.empty:
        over_str = "; ".join(
            f"`{row['column']} = {row['level']}` (Δ{float(row['diff']):+0.3f})"
            for _, row in over.iterrows()
        )
        choice_md.append("**Over-represented:** " + over_str)
    if not under.empty:
        under_str = "; ".join(
            f"`{row['column']} = {row['level']}` (Δ{float(row['diff']):+0.3f})"
            for _, row in under.iterrows()
        )
        choice_md.append("**Under-represented:** " + under_str)
    choice_md.append("")

out_md = "unsup_outputs/unsup_final_choice.md"
with open(out_md, "w", encoding="utf-8") as f:
    f.write("\n".join(choice_md))
print("Saved:", out_md)

# 5) Final zip addendum (only add files that exist)
zip_path = "unsup_addendum_final_segments.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in [
        "unsup_outputs/unsup_final_choice.md",
        "unsup_outputs/unsup_summary_k3.md",
        "unsup_outputs/unsup_spotlight_k7.md",
        "unsup_kproto/summary_k3.csv",
        "unsup_kproto/top_signals_k3.csv",
        "unsup_kproto/top_numeric_signals_k3.csv",
        "unsup_kproto/summary_k7.csv",
        "unsup_kproto/top_signals_k7.csv",
        "unsup_kproto/segmentation_comparison.csv",
        "unsup_kproto/labels_k3.csv",
        "plots_unsup/segmentation_silhouette_comparison.png",
        "plots_unsup/kproto_scatter_k3.png",
        "plots_unsup/kproto_avg_rating_k3.png",
        "plots_unsup/cluster_scatter_z12_k3.png",
        "plots_unsup/cluster_avg_rating_k3.png",
        "plots_unsup/kproto_scatter_k7.png",
        "plots_unsup/kproto_avg_rating_k7.png",
        "plots_unsup/cluster_scatter_z12_k7.png",
        "plots_unsup/cluster_avg_rating_k7.png",
    ]:
        if os.path.exists(p):
            z.write(p, arcname=os.path.basename(p))
print("Saved:", zip_path)


In [ ]:
#Github Push

In [ ]:
!git config --global user.name "erosas9172"
!git config --global user.email "erosas9172@sdsu.edu"
!git config --global init.defaultBranch main


In [ ]:
!git config --global credential.helper osxkeychain

In [ ]:
!mkdir -p ~/jupyter-notebooks
%cd ~/jupyter-notebooks
!pwd
!ls -la
